<a href="https://colab.research.google.com/github/gonyoly/portfolio/blob/main/M_52_UK_PLACES_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRELIMINARY PROCEDURES

In [ ]:
#@title INSTALATIONS
!pip install numpy==1.26.0 --force-reinstall
!pip install flair
# !pip install spacy
# !python -m spacy download uk_core_news_lg
# !pip install rapidfuzz

import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 412.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [ ]:
#@title IMPORTS

import pathlib
pathlib.WindowsPath = pathlib.PosixPath

from flair.data import Sentence, Corpus
from flair.models import SequenceTagger
import pandas as pd
import random
import logging
import openpyxl
import tqdm
import re
import os
from google.colab import drive
import IPython.display as ipd
from IPython.display import display, HTML, Markdown
import subprocess
import sys
import numpy as np
import matplotlib.pyplot as plt
import json, hashlib


In [ ]:
# @title LOAD DATA & MODEL

# mount Google Drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# load model
tagger = SequenceTagger.load('lang-uk/flair-uk-ner')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 admin_units_extracted_fond_and_all_title.xlsx
 admun_presence_pilot.xlsx
 consolidated_data_original.xlsx
 consolidated_data_working_copy.csv
 consolidated_data_working_copy_with_title_and_provenance_places_bef_lemm.xlsx
 consolidated_data_working_copy_with_title_and_provenance_places.xlsx
 consolidated_data_working_copy.xlsx
'DRAFTS_DEBUGS_&_PILOTS_ARCHIVE'
 final_places_after_lemm_after_dedupl.xlsx
 final_places_after_lemm_bef_dedupl.xlsx
 final_places_mach_transl_with_fixed.xlsx
 final_places_mach_transl.xlsx
 final_places_with_sapir_take1.xlsx
 lowercase_terms_for_lemm_dictionary.xlsx
 lowercase_terms_with_repl_patterns.xlsx
 match_update_report.xlsx
 missing_union_table.xlsx
 nonempty_consolidated_unmatched.xlsx
 nonempty_master_unmatched.xlsx
 OLD_IRRELEVANT
 PLACE_EXTRACTION_TAKE_1
 Place_ID_final.txt
 places_missing_in_sapir_administrative_division.x

# EXTRACTION OF PLACES FROM TITLE & FOND

In [ ]:
# @title EXTRACTION OF PLACES USING NER & REGEX


# loading the data
data_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/consolidated_data_working_copy.csv"
df = pd.read_csv(data_path, low_memory=False)

# defining regex pattern for topological adjectives & more
regex_pattern = r"(?:(?:[А-ЯҐЄІЇ][а-яґєії']+(?:(?<![^Є]пи)(?<!рей)(?<![^Л]а)(?<!(?:еє)т)(?<!М[іи])(?<!подар)(?<!р[иі]че)(?:с|ц)ь?к)[аоегмуюийї]{,3})(?:[а-яґєії']+)?)|(?:(?<=м\.\s)[А-ЯҐЄІЇ][а-яґєії']+)|(?:(?<=с\.\s)[А-ЯҐЄІЇ][а-яґєії']+)|(?:(?<=(?<![0-9])(?<![0-9] )г\.\s)[А-ЯҐЄІЇ][а-яґєії']+)|(?:(?<=м\.)[А-ЯҐЄІЇ][а-яґєії']+)|(?:(?<=с\.)[А-ЯҐЄІЇ][а-яґєії']+)|(?:(?<=г\.)[А-ЯҐЄІЇ][а-яґєії']+)|(?:[А-ЯҐЄІЇ][а-яґєії']+(?:(?<!(?:ьків))(?:ч|щ)ин)[аіеоуий])"

# defining a helper function to filter redundant results


def filter_redundant_places(locations):
    filtered = []
    for loc in locations:
        # keeping location if it is not a full word inside another longer
        # location
        if not any(
    re.search(
        rf'{re.escape(loc)}',
         other) and loc != other for other in locations):
            filtered.append(loc)
    return filtered

# defining combined extraction function (NER + REGEX)


def extract_combined_locations(text, tagger):
    if not isinstance(text, str):
        return ''

    # NER-based extraction
    sentence = Sentence(text)
    tagger.predict(sentence)
    ner_locations = [entity.text for entity in sentence.get_spans(
        'ner') if entity.tag == 'LOC']

    # Regex-based extraction
    regex_locations = re.findall(regex_pattern, text)

    # Combining results and filtering duplicates
    combined_locations = list(set(ner_locations + regex_locations))
    filtered_locations = filter_redundant_places(combined_locations)

    return ', '.join(filtered_locations)


# loading the NER model
tagger = SequenceTagger.load('lang-uk/flair-uk-ner')

# adding running IDs to the DataFrame
df.insert(0, 'id', range(1, len(df) + 1))

# splitting the DataFrame into chunks of 200 rows
chunks = [df[i:i + 200] for i in range(0, len(df), 200)]

# processing chunks sequentially
processed_chunks = []
for chunk in tqdm(chunks, desc="Processing chunks"):
    chunk['places'] = chunk['all_title'].apply(
        lambda x: extract_combined_locations(x, tagger))
    chunk['provenance_places'] = chunk['fond_title'].apply(
        lambda x: extract_combined_locations(x, tagger))
    processed_chunks.append(chunk)

# concatenating all processed chunks
df_processed = pd.concat(processed_chunks, ignore_index=True)

# saving the results
output_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/consolidated_data_working_copy_with_title_and_provenance_places_.xlsx"
df_processed.to_excel(output_path, index=False)

print(f"Processing complete. Results saved to {output_path}")

# calculate and display statistics
df_places = df_processed[df_processed['places'].notna() & (
    df_processed['places'] != '')]
df_provenance_places = df_processed[df_processed['provenance_places'].notna() & (
    df_processed['provenance_places'] != '')]

total_rows = len(df_processed)
places_non_empty = len(df_places)
provenance_non_empty = len(df_provenance_places)
empty_rows_places = total_rows - places_non_empty
empty_rows_provenance = total_rows - provenance_non_empty

print(f"Total number of rows in DataFrame: {total_rows}")
print(f"Number of non-empty values in 'places' column: {places_non_empty}")
print(
    f"Number of non-empty values in 'provenance_places' column: {provenance_non_empty}")
print(f"Number of empty values in 'places' column: {empty_rows_places}")
print(
    f"Number of empty values in 'provenance_places' column: {empty_rows_provenance}")
print(
    f"Percentage of rows with extracted places: {(places_non_empty / total_rows) * 100:.2f}%")
print(
    f"Percentage of rows with extracted provenance_places: {(provenance_non_empty / total_rows) * 100:.2f}%")

# additional statistics
print("
Additional statistics for 'places': ")
print(
    f"Average number of places per non-empty row: {df_places['places'].str.count(',').mean() + 1:.2f}")
print(
    f"Maximum number of places in one row: {df_places['places'].str.count(',').max() + 1}")

print("
Additional statistics for 'provenance_places': ")
print(
    f"Average number of places per non-empty row: {df_provenance_places['provenance_places'].str.count(',').mean() + 1:.2f}")
print(
    f"Maximum number of places in one row: {df_provenance_places['provenance_places'].str.count(',').max() + 1}")

# displaying random samples
print("
Random samples of extracted places and provenance_places: ")
sample = df_processed[(df_processed['places'].notna() & (df_processed['places'] != '')) |
                      (df_processed['provenance_places'].notna() & (df_processed['provenance_places'] != ''))]
display(sample[['id', 'all_title', 'places',
        'fond_title', 'provenance_places']].sample(20))

In [ ]:
# @title VISUALISATION OF NER IN 5% SAMPLES FROM ALL_TITLE AND FOND_TITLE


# loading the excel file
file_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/consolidated_data_working_copy_with_title_and_provenance_places.xlsx"
# checking if the file exists
if not os.path.exists(file_path):
  raise FileNotFoundError(
      f"File not found: {file_path}. Please check the file path and ensure the file exists in your Google Drive.")

df = pd.read_excel(file_path)

# assuming 'sequencetagger' and 'sentence' should be 'SequenceTagger' and
# 'Sentence'
tagger = SequenceTagger.load('lang-uk/flair-uk-ner')


# defining a function to get color for each entity type
def get_color(entity_type):
    colors = {
        'PER': '#ffa500',  # orange
        'LOC': '#1e90ff',  # dodger blue
        'ORG': '#32cd32',  # lime green
        'MISC': '#ffd700'  # gold
    }
    return colors.get(entity_type, '#ffffff')


# defining a function to visualize ner with improved html formatting and
# entity count
def visualize_ner(text):
    text = str(text)
    # using Sentence from flair.data
    sent = Sentence(text)
    tagger.predict(sent)

    # initializing html string for highlighted text
    html_content = ""
    last_idx = 0
    entities = sent.get_spans('ner')

    for entity in entities:
        start, end = entity.start_position, entity.end_position
        entity_text = text[start:end]
        entity_type = entity.get_label('ner').value
        color = get_color(entity_type)

        # adding text before the entity
        html_content += text[last_idx:start]
        # adding highlighted entity with inline css styles
        html_content += f'<mark style="background-color: {color}; font-weight: bold; border-radius: 0.5em; padding: 0.1em 0.3em; margin: 0 0.25em;">{entity_text}<span style="font-size: 0.8em; font-weight: bold; vertical-align: top; margin-left: 0.5em;">{entity_type}</span></mark>'
        last_idx = end

    # adding remaining text after the last entity
    html_content += text[last_idx:]

    # wrapping the highlighted text in a styled container
    container_html = f'<div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; font-family: arial; font-size: 16px; line-height: 1.5;">{html_content}</div>'
    # adding information about the number of found entities
    info_html = f'<p style="font-family: arial; font-size: 14px; color: #555;">found {len(entities)} entities.</p>'

    # displaying the final formatted html output
    # assuming 'html' should be 'HTML' from IPython.display
    display(HTML(container_html + info_html))


# sampling and visualizing for 'all_title'
sample_size_all = int(len(df) * 0.05)
random_sample_all = df['all_title'].dropna().sample(
    n=sample_size_all, random_state=1)

print("ner visualization for 5% random sample from all_title:")
for txt in random_sample_all:
    visualize_ner(txt)


# sampling and visualizing for 'fond_title'
sample_size_fond = int(len(df) * 0.05)
random_sample_fond = df['fond_title'].dropna().sample(
    n=sample_size_fond, random_state=2)

print("
ner visualization for 5 % random sample from fond_title: ")
for txt in random_sample_fond:
    visualize_ner(txt)

2025-04-09 07:07:57,601 SequenceTagger predicts: Dictionary with 19 tags: O, S-PERS, B-PERS, E-PERS, I-PERS, S-LOC, B-LOC, E-LOC, I-LOC, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>
ner visualization for 5% random sample from all_title:



ner visualization for 5% random sample from fond_title:


In [ ]:
@title loading the processed file
processed_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_before_cleansing.xlsx"
df_processed = pd.read_excel(processed_path)

# calculating and displaying statistics
num_unique_places = len(df_processed)
num_title_only = len(df_processed[df_processed['source'] == 'title'])
num_fond_only = len(df_processed[df_processed['source'] == 'fond'])
num_title_and_fond = len(df_processed[df_processed['source'] == 'title&fond'])

# displaying the statistics
print("
statistics: ")
print(f"total unique places: {num_unique_places}")
print(
    f"places from title only: {num_title_only} ({(num_title_only / num_unique_places) * 100:.2f}%)")
print(
    f"places from fond only: {num_fond_only} ({(num_fond_only / num_unique_places) * 100:.2f}%)")
print(
    f"places from both title and fond: {num_title_and_fond} ({(num_title_and_fond / num_unique_places) * 100:.2f}%)")


statistics:
total unique places: 4153
places from title only: 4010 (96.56%)
places from fond only: 80 (1.93%)
places from both title and fond: 63 (1.52%)


#  DATA CLEANSING

In [ ]:
# @title PRIMARY CLEANSING OF EXTRACTED PLACES

# loading the necessary files
input_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_before_cleansing.xlsx"
deletion_list_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_values_deletion_list.xlsx"
components_fix_list_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_components_fix_list.xlsx"
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_after_primary_cleansing.xlsx"


data = pd.read_excel(input_file)
deletion_list = pd.read_excel(deletion_list_file)
components_fix_list = pd.read_excel(components_fix_list_file)

# adding a new column to store the original ID
data["ID_bef_cleans"] = data["ID"]

# capturing initial row count
initial_row_count = len(data)
rows_deleted = 0
rows_created = 0

# lists for logging
split_logs_success = []
split_logs_failed = []

# performing deletions based on deletion list
for index, row in deletion_list.iterrows():
    pattern = row["regex_pattern"]
    field = row["field_to_apply_pattern"]
    before_deletion = len(data)

    if pd.notna(field) and field in data.columns:
        data = data[~data[field].apply(lambda x: bool(
            re.search(pattern, str(x))) if pd.notna(x) else False)]

    data = data.reset_index(drop=True)
    rows_deleted += before_deletion - len(data)

# performing fixes based on components fix list
for index, row in components_fix_list.sort_values(
    by="execution_step_index").iterrows():
    pattern = row["regex_pattern"]
    component = row["component"]
    remove_component = row["remove_component"]
    further_action_1 = row["further_action_1"]
    further_action_2 = row["further_action_2"]
    note = row["note"]

    data = data.reset_index(drop=True)

    matching_rows = data.index[data["unique_extracted_places"].str.contains(
        pattern, na=False, regex=True)]

    # performing further action 1: splitting the string
    if further_action_1 == "insert the string after ( as a sep. value with the same titles, fonds & IDs":
        for idx in matching_rows:
            original_value = data.loc[idx, "unique_extracted_places"]
            if component in original_value:
                split_value = original_value.split(component, 1)[1]
                new_row = data.loc[idx].copy()
                new_row["unique_extracted_places"] = split_value.strip()
                data = pd.concat(
                    [data, pd.DataFrame([new_row])], ignore_index=True)
                rows_created += 1
                split_logs_success.append(
                    f"SUCCESS: Split performed for ID {data.loc[idx, 'ID_bef_cleans']}: '{original_value}' -> '{split_value.strip()}'")

        # deleting original rows after splitting
        data = data.drop(index=matching_rows).reset_index(drop=True)

    # handling classifier assignment and replacement
    if further_action_2 == "move deleted string to a sep. [classifier] column":
        corrected_component = "с." if component == "c." else component
        data.loc[matching_rows, "classifier"] = corrected_component
        data.loc[matching_rows,
    "unique_extracted_places"] = data.loc[matching_rows,
    "unique_extracted_places"].str.replace(re.escape(component),
    "",
     regex=True).str.strip()

    # removing the component if required
    if remove_component == "yes" and matching_rows.any():
        data.loc[matching_rows, "unique_extracted_places"] = data.loc[matching_rows, "unique_extracted_places"].apply(
            lambda x: re.sub(
    r"(?<![cсм])\.", "", x) if isinstance(
        x, str) else x
        )

    # handling hyphens
    data.loc[matching_rows, "unique_extracted_places"] = data.loc[matching_rows, "unique_extracted_places"].apply(
        lambda x: re.sub(
    r"(\w+)-(\w+-)",
    r"",
    x) if isinstance(
        x,
         str) else x
    )
    data.loc[matching_rows, "unique_extracted_places"] = data.loc[matching_rows, "unique_extracted_places"].apply(
        lambda x: re.sub(
    r"(?<!-)(\w+)-(\w+)(?!-)",
    r"",
    x) if isinstance(
        x,
         str) else x
    )

    # adding quotes at the beginning of unique_extracted_places if required
    if further_action_1 == 'add " at the start of unique_extracted_places' and matching_rows.any():
        data.loc[matching_rows, "unique_extracted_places"] = data.loc[matching_rows, "unique_extracted_places"].apply(
            lambda x: '"' + x if isinstance(x,
     str) and not x.startswith('"') else x
        )

# trimming whitespace from unique_extracted_places
data["unique_extracted_places"] = data["unique_extracted_places"].str.strip()

# sorting data alphabetically by unique_extracted_places
data = data.sort_values(by="unique_extracted_places").reset_index(drop=True)

# dropping the existing 'ID' column before inserting a new one
if 'ID' in data.columns:
  data = data.drop(columns=['ID'])

# assigning new sequential IDs
data.insert(0, "ID", range(1, len(data) + 1))

# filling NaN values in title_*, fond_*, and classifier with ""
title_columns = [f"title_{i}" for i in range(1, 7)]
fond_columns = [f"fond_{i}" for i in range(1, 7)]
data[["classifier"] +
    title_columns +
    fond_columns] = data[["classifier"] +
    title_columns +
     fond_columns].fillna("")

# ensuring the correct column order
column_order = ["ID", "ID_bef_cleans", "classifier", "unique_extracted_places",
                "source", "title_1", "title_2", "title_3", "title_4", "title_5", "title_6",
                "fond_1", "fond_2", "fond_3", "fond_4", "fond_5", "fond_6", "source_ids"]
data = data[column_order]

# saving the processed data to the output file
data.to_excel(output_file, index=False)


# generating detailed output
final_row_count = len(data)
deleted_percent = round((rows_deleted / initial_row_count) * 100, 2)
created_percent = round((rows_created / initial_row_count) * 100, 2)
changed_rows_sample = data.sample(n=10)
# counting split records
split_count = len(split_logs_success)

# counting modified records per component
updated_records = {}
for component in components_fix_list["component"].unique():
    escaped_component = re.escape(component)
    count = data["unique_extracted_places"].str.contains(
        escaped_component, na=False, regex=True).sum()
    if count > 0:
        updated_records[component] = count

# total updated records (excluding splits)
total_updated = sum(updated_records.values())

# computing percentages
split_percent = round((split_count / final_row_count) * 100, 2)
updated_percent = round((total_updated / final_row_count) * 100, 2)

# detailed update stats per component
print("Processing complete!")
print(f"Initial rows: {initial_row_count} (100%)")
print(f"Rows deleted: {rows_deleted} ({deleted_percent}%)")
print(f"Rows created (from split): {rows_created} ({created_percent}%)")
print(
    f"Final rows: {final_row_count} ({round((final_row_count / initial_row_count) * 100, 2)}%)")

print(f"
Records modified due to splitting: {split_count} ({split_percent} %)")
print(
    f"Total updated records (excluding splits): {total_updated} ({updated_percent}%)")

# per-component breakdown
print("
--- UPDATED RECORDS PER COMPONENT - --")
for component, count in updated_records.items():
    percent = round((count / final_row_count) * 100, 2)
    print(f"Component '{component}': {count} records ({percent}%)")

# displaying logs
print("
--- SPLIT SUCCESS LOGS - --")
for log in split_logs_success:
    print(log)

print("
--- SPLIT FAILED LOGS - --")
for log in split_logs_failed:
    print(log)

# displaying random sample of changed rows
changed_rows_sample = data.sample(n=10)  # random sample of changed rows
print("
Random sample of changed rows: ")
display(changed_rows_sample)

Processing complete!
Initial rows: 4153 (100%)
Rows deleted: 957 (23.04%)
Rows created (from split): 6 (0.14%)
Final rows: 3196 (76.96%)

Records modified due to splitting: 6 (0.19%)
Total updated records (excluding splits): 339 (10.61%)

--- UPDATED RECORDS PER COMPONENT ---
Component ')': 5 records (0.16%)
Component 'суч': 3 records (0.09%)
Component '.': 99 records (3.1%)
Component 'Том': 30 records (0.94%)
Component 'Том 1': 18 records (0.56%)
Component 'Том 2': 5 records (0.16%)
Component 'Том 3': 2 records (0.06%)
Component 'с.': 1 records (0.03%)
Component 'м.': 59 records (1.85%)
Component '"': 23 records (0.72%)
Component '-': 94 records (2.94%)

--- SPLIT SUCCESS LOGS ---
SUCCESS: Split performed for ID 2377: 'с. Нагірянка (Нагоряни' -> 'Нагоряни'
SUCCESS: Split performed for ID 2631: 'с.   Нагірянка (Нагоряни' -> 'Нагоряни'
SUCCESS: Split performed for ID 2901: 'м. Тлусте (Товсте' -> 'Товсте'
SUCCESS: Split performed for ID 3676: 'с. Янів (Долина' -> 'Долина'
SUCCESS: Split 

,ID,ID_bef_cleans,classifier,unique_extracted_places,source,title_1,title_2,title_3,title_4,title_5,title_6,fond_1,fond_2,fond_3,fond_4,fond_5,fond_6,source_ids
1995,1996,3213,,Печерская,title,Печерская Фейга Самуиловна,,,,,,,,,,,,2688
2298,2299,1324,,Свалявському районі,title,Відомості про нерухоме майно у Свалявському ра...,Відомості про власне нерухоме майно заявників ...,Відомості про нерухоме майно у Свалявському ра...,,,,,,,,,,"7871, 7873, 7874"
1170,1171,1641,,Карпат,title,"Воспоминание Щербака А.С., участника ВОВ, прин...",,,,,,,,,,,,7630
2699,2700,3255,,Тягинським,title,Протоколи загальних зборів громадян колонії Ль...,,,,,,,,,,,,16562
3009,3010,2469,,Шаргородский,title,Шаргородский Иван Арсентьевич,,,,,,,,,,,,3365
1456,1457,629,,Кутківці,title,"Те саме, сіл Кутківці, Купчинці, том 6 (№ 1074...",,,,,,,,,,,,6430
1158,1159,3660,,Кананської,title,Заява російського емігранта Кананської Фрейди ...,,,,,,,,,,,,4232
2960,2961,222,,Чкалівської сільрад,title,"Те саме Новогреднівської, Хрущовської та Чкалі...","Те ж Куйбишівської, Новогреднівської та Чкалів...","Списки репатріантів, котрі проживали на терито...","Списки обміру посівних площ Калінінської, Бобр...",,,,,,,,,"13090, 13291, 13354, 14488"
2232,2233,698,,Рожищах,title,Заявление Флейшеров и Гейлера и переписка с Лу...,,,,,,,,,,,,"953, 1822"
3018,3019,1319,,Шером Хаїмом,title,Справа про аренду ділянки міської землі Шером ...,,,,,,,,,,,,8961


In [ ]:
# @title REDEDUPLICATION OF UNIQUE EXTRACTED PLACES

# loading the necessary files
input_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_after_primary_cleansing.xlsx"
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_after_rededuplication.xlsx"

# reading the input file
data = pd.read_excel(input_file)

# creating a new column for the ID after cleansing
data["ID_aft_cleans"] = data["ID"]

# grouping data by unique_extracted_places and classifier to find duplicates
grouped = data.groupby(["unique_extracted_places", "classifier"], dropna=False)

# initializing lists to track changes and logs
merged_rows = []
logs = []
changes_summary = {
    "source": 0,
    "title": 0,
    "fond": 0,
    "source_ids": 0
}

# iterating through groups and processing duplicates
for (place, classifier), group in grouped:
    if len(group) > 1:
        # extracting IDs and original rows
        ids = ",".join(map(str, group["ID_aft_cleans"].tolist()))
        original_ids = ",".join(map(str, group["ID_bef_cleans"].tolist()))

        # handling source
        sources = group["source"].dropna().unique()
        if len(sources) > 1:
            merged_source = "title&fond"
            changes_summary["source"] += 1
        else:
            merged_source = sources[0] if len(sources) > 0 else ""

        # handling titles
        titles = group[[f"title_{i}" for i in range(1, 7)]].fillna("")
        merged_titles = []
        for row in titles.itertuples(index=False):
            merged_titles.extend([title for title in row if title != ""])
        changes_summary["title"] += len(merged_titles)

        # handling fonds
        fonds = group[[f"fond_{i}" for i in range(1, 7)]].fillna("")
        merged_fonds = []
        for row in fonds.itertuples(index=False):
            merged_fonds.extend([fond for fond in row if fond != ""])
        changes_summary["fond"] += len(merged_fonds)

        # handling source_ids
        source_ids = ",".join(group["source_ids"].dropna().unique())
        merged_source_ids = ",".join(sorted(set(source_ids.split(","))))
        changes_summary["source_ids"] += len(merged_source_ids.split(","))

        # creating a merged row
        merged_row = {
            "ID": None,  # to be assigned later
            "ID_bef_cleans": original_ids,
            "ID_aft_cleans": ids,
            "classifier": classifier if pd.notna(classifier) else "",
            "unique_extracted_places": place,
            "source": merged_source,
            "source_ids": merged_source_ids,
        }

        # handling merged titles and fonds
        for i in range(1, 7):
            merged_row[f"title_{i}"] = merged_titles[i -
                1] if i - 1 < len(merged_titles) else ""
            merged_row[f"fond_{i}"] = merged_fonds[i -
                1] if i - 1 < len(merged_fonds) else ""

        merged_rows.append(merged_row)
        logs.append(f"Merged IDs {ids} with unique_extracted_places '{place}'")

# creating a DataFrame for merged rows
merged_data = pd.DataFrame(merged_rows)

# ensuring that merged rows are correctly removed from the final dataset
merged_ids = set(
    merged_data["ID_aft_cleans"].str.split(",").explode().dropna().astype(str)
)
non_merged_data = data[~data["ID_aft_cleans"].astype(str).isin(merged_ids)]
final_data = pd.concat([non_merged_data, merged_data], ignore_index=True)

# creating a new column with trimmed extracted places
final_data["normalized_extracted_places"] = final_data["unique_extracted_places"].str.replace(
    r"\s+", " ", regex=True).str.strip()

# ensuring 'classifier' column has no NaN values
final_data["classifier"] = final_data["classifier"].fillna("")

# ensuring the correct column order
column_order = [
    "ID", "source_ids", "ID_bef_cleans", "ID_aft_cleans", "classifier",
    "unique_extracted_places", "normalized_extracted_places", "source",
    "title_1", "title_2", "title_3", "title_4", "title_5", "title_6",
    "fond_1", "fond_2", "fond_3", "fond_4", "fond_5", "fond_6"
]
final_data = final_data[column_order]

# saving the processed data to the output file
final_data.to_excel(output_file, index=False)

# generating detailed output
merged_count = len(merged_rows)
total_count = len(data)
removed_count = len(data) - len(final_data)
merged_percent = round((merged_count / total_count) * 100, 2)
final_percent = round((len(final_data) / total_count) * 100, 2)

print(f"Initial rows: {total_count} (100%)")
print(f"Rows merged: {merged_count} ({merged_percent}%)")
print(
    f"Rows removed due to merging: {removed_count} ({round((removed_count / total_count) * 100, 2)}%)")
print(f"Final rows: {len(final_data)} ({final_percent}%)")

# calculating statistics on normalized_extracted_places
trimmed_duplicates = final_data.groupby("normalized_extracted_places")[
                                        "unique_extracted_places"].nunique()
merged_trimmed_count = (trimmed_duplicates > 1).sum()
merged_trimmed_percent = round(
    (merged_trimmed_count / len(final_data)) * 100, 2)

print(f"
Total unique places that merged due to whitespace normalization: {merged_trimmed_count}")
print(f"Percentage of total rows affected: {merged_trimmed_percent}%")

# displaying logs for merged rows
print("
--- MERGED ROWS LOGS - --")
for log in logs[:20]:  # showing first 20 logs only
    print(log)

# displaying random sample of merged rows
print("
--- RANDOM SAMPLE OF MERGED ROWS - --")
display(merged_data.sample(n=min(20, len(final_data))))


# displaying a sample of merged trimmed places
print("
--- SAMPLE OF MERGED TRIMMED PLACES - --")
display(final_data.loc[final_data["normalized_extracted_places"].duplicated(keep=False), [
        "unique_extracted_places", "normalized_extracted_places"]].sample(n=min(20, merged_trimmed_count)))

Initial rows: 3189 (100%)
Rows merged: 44 (1.38%)
Rows removed due to merging: 50 (1.57%)
Final rows: 3139 (98.43%)

Total unique places that merged due to whitespace normalization: 23
Percentage of total rows affected: 0.73%

--- MERGED ROWS LOGS ---
Merged IDs 46,47 with unique_extracted_places 'Іршавському районі'
Merged IDs 97,98 with unique_extracted_places 'Бариш'
Merged IDs 129,130 with unique_extracted_places 'Берегівському районі'
Merged IDs 270,272 with unique_extracted_places 'Броди'
Merged IDs 291,292 with unique_extracted_places 'Буданівський'
Merged IDs 317,318 with unique_extracted_places 'Бучацький'
Merged IDs 326,327,328 with unique_extracted_places 'Бучацького повіту'
Merged IDs 336,337,338,339 with unique_extracted_places 'Бучач'
Merged IDs 406,407 with unique_extracted_places 'Великобірківський'
Merged IDs 415,416 with unique_extracted_places 'Великодедеркальського району'
Merged IDs 501,502 with unique_extracted_places 'Вишнівецький райони'
Merged IDs 504,505 with 

,ID,ID_bef_cleans,ID_aft_cleans,classifier,unique_extracted_places,source,source_ids,title_1,fond_1,title_2,fond_2,title_3,fond_3,title_4,fond_4,title_5,fond_5,title_6,fond_6
8,None,"2327,1860","406,407",,Великобірківський,title,"6518, 6521,6511,6524",Протоколи допитів затриманих бійців гарнізону ...,,Протоколи дізнань осіб зі середовища цивіл...,,пис. Протоколи допитів осіб зі середовища циві...,,Протоколи допитів виявлених секретних освідомл...,,,,,
35,None,"3319,1728","2559,2560",,Теребовлянського повіту,title,"5460,5529",Статут та періодичні відомості повітового стар...,,Періодичні відомості повітового староства про ...,,,,,,,,,
26,None,"2080,571","1560,1561",,Луцке,title,"1019, 1023, 1154, 1155, 1713, 1741, 1848, 188...",Дефектный акт и смета строительства больницы в...,,Списки граждан взятых на учет и снятых с учета...,,Устав хирургично-гинекологической больницы в Л...,,"Списки лиц, вывезенных на работу в Германию, н...",,Книга регистрации разводов в г. Луцке за 1938 год,,Переписка с Луцким староством о выдаче промышл...,
27,None,"2614,2444","1744,1748",м.,Монастириська,title,"8226,4036,8224",Статут і періодичні відомості повітового старо...,,Списки виборців по виборах\n до Сейму у м. М...,,Списки виборців по виборах\n до Сенату у \n ...,,,,,,,
29,None,"2377,2631,3698","1800,1801,1802",,Нагоряни,title,"8100,8228,8229",Списки виборців по виборах\n до Сенату у с....,,Списки виборців по виборах\n до Сейму у с. ...,,Списки виборців по виборах депутатів до Сейму\...,,,,,,,
33,None,"199,3607","2358,2359",м.,Скалат,title,"8288,8136,8191",Те саме у м. Скалат.,,"Те саме, у м. Скалат.",,"Те саме, у м. Скалат.\nТом 1.\n",,,,,,,
4,None,"484,2836","291,292",,Буданівський,title&fond,"6558, 6746, 9654, 9655, 9656, 9657, 9658, 965...",Протоколи дізнань виявлених секретних освідомл...,Буданівський міський суд,«Протоколи впавших борців революції за ...,,"Протоколи розстрілу осіб, які сприя-ли органам...",,"Список членів ОУН-УПА, полеглих у 1941-1946 ро...",,,,,
34,None,"2303,1019","2374,2375",,Скалатчина,title,"6550,6551","Хроніка подій ""Вісті з терену. Скалатчина"" із ...",,"Хроніка подій ""Вісті з терену. Ска-латчина"" із...",,,,,,,,,
22,None,"234,1041,4054","1323,1324,1327",с.,Коропець,title,"6040, 8094, 8095, 8165,5111,8096,8222",Списки виборців по виборах депутатів до Сенату...,,Списки виборців по виборах депутатів до Сейму\...,,Статут і реєстраційний лист єврейського товари...,,Списки виборців по виборах депутатів до Сейму\...,,"Те саме, у с. Коропець",,Статут і періодичні відомості повітового старо...,
1,None,"2986,626","97,98",с.,Бариш,title,"6035, 8218,5253,8083",Списки виборців по виборах депутатів до Сейму ...,,Статут та періодичні відомості повітового стар...,,Те саме у с. Бариш,,Статут і реєстраційний лист єврейського товари...,,,,,



--- SAMPLE OF MERGED TRIMMED PLACES ---


,unique_extracted_places,normalized_extracted_places
483,Вишнівчик,Вишнівчик
1841,Олесько Золочівського повіту,Олесько Золочівського повіту
1047,Калініндорф,Калініндорф
471,Вишнівець,Вишнівець
543,Воробіївка,Воробіївка
1992,Поморяни Зборівського повіту,Поморяни Зборівського повіту
1265,Копичинці,Копичинці
130,Бережанського повіту,Бережанського повіту
898,Заліщицького повіту,Заліщицького повіту
1210,Козлів Тернопільського повіту,Козлів Тернопільського повіту


In [ ]:
@title EXTRACTING LOCATIVE NOUNS(ADMINISTRATIVE UNITS) FROM TITLE & FOND
# defining the combined list of administrative unit roots and abbreviations
admin_roots = [
    # ukrainian terms
    'воєводств', 'волост', 'област', 'район', 'повіт', 'повітов', 'округ', 'окруз', 'міст', 'містечк', 'селищ', 'сільрад', 'рад', 'гмін', 'ґмін', 'колгосп', 'уїзд', 'староств', 'нацрайон', 'надрайон', 'виборч',

    # russian terms
    'воеводств', 'област', 'район', 'повет', 'уезд', 'округ', 'губерни', 'волост', 'кра', 'город', 'сел', 'поселок', 'поселк', 'совет', 'гмин', 'колхоз',

    # specific abbreviations
    r'(?<!\w)с\.', r'(?<!\w)с-щ(?:е|а)', r'(?<!\w)м\.', r'(?<!\w)г\.'
    ]

# defining position of the administrative unit terms
admin_roots_notes = {
    # ukrainian terms
    'воєводств': 'postpositive', 'волост': 'postpositive', 'област': 'postpositive', 'район': 'postpositive', 'повіт': 'postpositive', 'повітов': 'postpositive', 'округ': 'postpositive', 'окруз': 'postpositive', 'міст': 'prepositive', 'містечк': 'prepositive', 'селищ': 'prepositive', 'сільрад': 'postpositive', 'рад': 'postpositive', 'гмін': 'postpositive', 'ґмін': 'postpositive', 'колгосп': 'prepositive', 'уїзд': 'postpositive', 'староств': 'postpositive', 'нацрайон': 'postpositive', 'надрайон': 'postpositive', 'виборч': 'postpositive',
    # russian terms
    'воеводств': 'postpositive', 'област': 'postpositive', 'район': 'postpositive', 'повет': 'postpositive', 'уезд': 'postpositive', 'округ': 'postpositive', 'губерни': 'postpositive', 'волост': 'postpositive', 'кра': 'postpositive', 'город': 'prepositive', 'сел': 'prepositive', 'поселок': 'postpositive', 'поселк': 'postpositive', 'совет': 'postpositive', 'гмин': 'postpositive', 'колхоз': 'prepositive',
    # specific abbreviations
    r'(?<!\w)с\.': 'prepositive', r'(?<!\w)с-щ(?:е|а)': 'prepositive', r'(?<!\w)м\.': 'prepositive', r'(?<!\w)г\.': 'prepositive'
}


# creating a regex pattern from the list of roots
def create_regex_pattern(root_list):
    # create pattern for each root
    patterns = [r'' + root + r'\w*' for root in root_list]
    return '|'.join(patterns)  # join patterns into a single regex


# creating regex pattern for searching administrative units
admin_pattern = create_regex_pattern(admin_roots)


# finding administrative units in the text and removing duplicates
def find_admin_units(text):
    # find matches ignoring case
    matches = re.findall(admin_pattern, text, re.IGNORECASE)

    # keeping only those starting with a lowercase letter and not preceded by
    # a capital letter (to avoid proper names)
    filtered_matches = [match for match in matches if match[0].islower()]

    # excluding words that contain "нн|ск|ськ" or have uppercase letters
    return [match for match in filtered_matches if "нн" not in match and not any(char.isupper(
    ) for char in match) and "ск" not in match and "ськ" not in match and "ник" not in match]


# applying the function to the title_1 and fond_1 columns in the DataFrame
all_units = [unit for col in ['title_1', 'fond_1']
    for units in uniq_after_prelim_clean[col].apply(find_admin_units) for unit in units]

# getting unique units and sorting them
unique_units = sorted(set(all_units))

print("Extracted administrative units:")
print(unique_units)  # outputting the found administrative units

# saving results to a new excel file
output_path = '/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/admin_units_extracted_fond_and_all_title.xlsx'
df_result = pd.DataFrame(unique_units, columns=["all_admin_units"])
df_result['notes'] = df_result["all_admin_units"].apply(
    lambda unit: next(
        (val for key, val in admin_roots_notes.items() if re.search(
            key, unit, re.IGNORECASE)), ''))
df_result.to_excel(output_path, index=False)

print(f"Results saved to: {output_path}")

In [ ]:
# @title SPLITTING TOPONYMS & ADMINISTRATIVE UNITS

# loading the list of administrative units (notes == postpositive)
admin_units_df = pd.read_excel(
    "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/admin_units_extracted_fond_and_all_title.xlsx")
admin_units = set(
    admin_units_df.loc[admin_units_df["notes"] == "postpositive", "all_admin_units"])

# loading the extracted places data
places_df = pd.read_excel(
    "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_after_rededuplication.xlsx")

# storing initial count of rows
initial_count = places_df.shape[0]

# ensuring id_bef_split is present for all rows by copying id
places_df["ID_bef_split"] = places_df["ID"]

# defining regex pattern (unchanged)
pattern = r"(^[А-ЯҐЄІЇ][а-яґєії'-]+ *(?:[А-ЯҐЄІЇ](?!.*ськ)(?!.*ск)(?!.*цьк)(?!.*цк)[а-яґєії'-]+)? +)((?:[А-ЯҐЄІЇ][а-яґєії'-]+(?:(?:с|ц)ь?к[а-яґєії'-]+) *))"

# creating list for new rows and list for indices of rows that were split
new_rows = []
indices_to_drop = []

# iterating through places to find matches and perform splitting
for idx, row in places_df.iterrows():
    place = row["normalized_extracted_places"]
    match = re.match(pattern, place)
    if match:
        # first capturing group is the first part
        first_part = match.group(1).strip()
        # second part is everything after the first capturing group
        second_part = place[len(match.group(1)):].strip()

        # duplicate row for splitting
        row_first = row.copy()
        row_second = row.copy()

        # store original id in id_bef_split
        row_first["ID_bef_split"] = row["ID"]
        row_second["ID_bef_split"] = row["ID"]

        # assign split values to normalized_extracted_places
        row_first["normalized_extracted_places"] = first_part
        row_second["normalized_extracted_places"] = second_part

        # add new rows to the list
        new_rows.append(row_first)
        new_rows.append(row_second)

        # mark original row for removal
        indices_to_drop.append(idx)

# remove original rows that were split
places_df = places_df.drop(indices_to_drop)

# create dataframe from new split rows
new_rows_df = pd.DataFrame(new_rows)

# combine unsplit rows with new split rows
final_places_df = pd.concat([places_df, new_rows_df], ignore_index=True)

# trim normalized_extracted_places and sort alphabetically
final_places_df["normalized_extracted_places"] = final_places_df["normalized_extracted_places"].str.strip()
final_places_df = final_places_df.sort_values(
    by="normalized_extracted_places").reset_index(
        drop=True)

# reset id with new sequential numbers
final_places_df["ID"] = range(1, len(final_places_df) + 1)

# reposition id_bef_split column: after ID_aft_cleans and before classifier
cols = list(final_places_df.columns)
if "ID_bef_split" in cols and "ID_aft_cleans" in cols:
    cols.remove("ID_bef_split")
    insert_index = cols.index("ID_aft_cleans") + 1
    cols.insert(insert_index, "ID_bef_split")
    final_places_df = final_places_df[cols]

# fill nan values with empty string in all columns
final_places_df = final_places_df.applymap(lambda x: "" if pd.isna(x) else x)

# saving the processed data to excel file
output_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_after_split.xlsx"
final_places_df.to_excel(output_path, index=False)

# outputting statistics
final_count = final_places_df.shape[0]
split_count = len(new_rows_df)

print(f"initial number of rows: {initial_count} (100.00%)")
print(
    f"final number of rows after processing: {final_count} ({(final_count / initial_count) * 100:.2f}%)")
print(
    f"split cases (found with admun): {split_count // 2} ({((split_count // 2) / initial_count) * 100:.2f}%)")
print(
    f"resulting split rows: {split_count} ({(split_count / final_count) * 100:.2f}% of final count)")

# displaying 20 random split cases correctly
if not new_rows_df.empty:
    split_counts = new_rows_df["ID_bef_split"].value_counts()
    # only ids occurring exactly 2 times
    valid_split_ids = split_counts[split_counts == 2].index
    filtered_splits = new_rows_df[new_rows_df["ID_bef_split"].isin(
        valid_split_ids)]

    # selecting only cases where both split parts exist
    valid_pairs = filtered_splits.groupby("ID_bef_split").filter(
        lambda x: len(x) == 2 and all(pd.notna(x["normalized_extracted_places"])))

    # selecting 10 valid split pairs (20 rows total)
    sampled_splits = valid_pairs.groupby("ID_bef_split").apply(
        lambda x: x.sample(n=2, random_state=42)).reset_index(drop=True)

    display(sampled_splits)
else:
    print("no valid split cases found.")

<ipython-input-36-253738c963c9>:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_places_df = final_places_df.applymap(lambda x: "" if pd.isna(x) else x)


initial number of rows: 3139 (100.00%)
final number of rows after processing: 3270 (104.17%)
split cases (found with admun): 131 (4.17%)
resulting split rows: 262 (8.01% of final count)


<ipython-input-36-253738c963c9>:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_splits = valid_pairs.groupby("ID_bef_split").apply(lambda x: x.sample(n=2, random_state=42)).reset_index(drop=True)


,ID,source_ids,ID_bef_cleans,ID_aft_cleans,classifier,unique_extracted_places,normalized_extracted_places,source,title_1,title_2,...,title_4,title_5,title_6,fond_1,fond_2,fond_3,fond_4,fond_5,fond_6,ID_bef_split
0,22,"13852, 13863",688,53,NaN,Авангард Калінінської сільради,Калінінської сільради,title,Те саме Авангард Калінінської сільради,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
1,22,"13852, 13863",688,53,NaN,Авангард Калінінської сільради,Авангард,title,Те саме Авангард Калінінської сільради,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
2,47,3772,2387,78,с.,Баворів Тернопільського повіту,Тернопільського повіту,title,Статут і періодичні відомості повітового старо...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47
3,47,3772,2387,78,с.,Баворів Тернопільського повіту,Баворів,title,Статут і періодичні відомості повітового старо...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47
4,239,5310,4086,240,NaN,Борщів Борщівського повіту,Борщівського повіту,title,Відомості повітового староства про зміну керів...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,3115,5426,3378,3093,с.,Язловець Бучацького повіту,Язловець,title,Статут та періодичні відомості повітового стар...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3115
258,3126,5290,253,3109,с.,Янів Теребовлянського повіту,Теребовлянського повіту,title,Відомості повітового староства про зміну керів...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3126
259,3126,5290,253,3109,с.,Янів Теребовлянського повіту,Янів,title,Відомості повітового староства про зміну керів...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3126
260,3138,5009,2265,3118,NaN,Яструбове Тернопільського повіту,Тернопільського повіту,title,Періодичні відомості повітового староства про ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3138




******************************
** **(MANUAL CLEANSING)**  **
******************************




In [ ]:
# @title INSERTING CLEANSED PLACES & PROVENANCE PLACES BEFORE LEMMATIZATION (INTERIM RESULTS)
# loading the consolidated data excel file and creating a copy with
# cleared columns
consolidated_file_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/consolidated_data_working_copy_with_title_and_provenance_places.xlsx"
df_consolidated = pd.read_excel(consolidated_file_path)

# creating a copy for modifications
df_consolidated_copy = df_consolidated.copy()

# clearing the contents of 'places' and 'provenance_places'
df_consolidated_copy['places'] = ""
df_consolidated_copy['provenance_places'] = ""

# renaming the cleared columns as specified
df_consolidated_copy.rename(
    columns={
        'places': 'places_bef_lemm',
        'provenance_places': 'provenance_places_bef_lemm'},
         inplace=True)

print("consolidated data loaded, columns 'places' and 'provenance_places' cleared and renamed.")

# loading the unique extracted places excel file and creating a copy
unique_places_file_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/unique_extracted_places_after_split_manually_processed.xlsx"
df_unique = pd.read_excel(unique_places_file_path)
df_unique_copy = df_unique.copy()

print("unique extracted places data loaded and copied.")

# defining a function to merge source_ids for groups with identical
# man_processed_extracted_places and source


def merge_source_ids(series):
    # initializing an empty list to collect ids
    all_ids = []
    # iterating over each source_ids value in the series
    for s in series:
        s = str(s)  # ensuring the value is a string
        for id_str in s.split(','):  # ids are guaranteed to be comma separated
            id_str = id_str.strip()
            if id_str:
                all_ids.append(int(id_str))
    # geting unique ids and sort them in ascending order
    unique_ids = sorted(set(all_ids))
    # joining them into a comma-separated string (with a comma and space)
    return ", ".join(str(i) for i in unique_ids)


# grouping by 'man_processed_extracted_places' and 'source'
df_unique_dedup = df_unique_copy.groupby(
    ['man_processed_extracted_places', 'source'], as_index=False).agg({'source_ids': merge_source_ids})

print("unique extracted places deduplicated based on 'man_processed_extracted_places' and 'source'.")

# updating consolidated data with unique extracted places
for _, row in df_unique_dedup.iterrows():
    source = row['source']
    place_value = row['man_processed_extracted_places']
    source_ids_str = row['source_ids']
    # spliting the source_ids string into a list of integers
    id_list = [int(x.strip())
                   for x in source_ids_str.split(',') if x.strip() != ""]
    # updating each corresponding row in the consolidated dataframe copy
    for id_val in id_list:
        row_indices = df_consolidated_copy.index[df_consolidated_copy['id'] == id_val].tolist(
        )
        for i in row_indices:
            if source == "title":
                current_val = df_consolidated_copy.at[i, 'places_bef_lemm']
                if current_val == "":
                    df_consolidated_copy.at[i, 'places_bef_lemm'] = place_value
                else:
                    df_consolidated_copy.at[i,
     'places_bef_lemm'] = current_val + ", " + place_value
            elif source == "fond":
                current_val = df_consolidated_copy.at[i,
                    'provenance_places_bef_lemm']
                if current_val == "":
                    df_consolidated_copy.at[i,
     'provenance_places_bef_lemm'] = place_value
                else:
                    df_consolidated_copy.at[i,
     'provenance_places_bef_lemm'] = current_val + ", " + place_value

print("consolidated data updated with unique extracted places.")

# saving the modified consolidated dataframe to a new excel file
output_file_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/consolidated_data_working_copy_with_title_and_provenance_places_bef_lemm.xlsx"
df_consolidated_copy.to_excel(output_file_path, index=False)

print("updated consolidated data saved to excel file at:")
print(output_file_path)

# calculating detailed statistics for inserted values
total_rows = len(df_consolidated_copy)

# for title (places_bef_lemm)
df_title = df_consolidated_copy[df_consolidated_copy['places_bef_lemm'] != ""].copy(
)
df_title['num_values'] = df_title['places_bef_lemm'].apply(
    lambda x: len([val.strip() for val in x.split(",") if val.strip() != ""]))
total_title_rows = len(df_title)
total_title_values = df_title['num_values'].sum()
dist_title = df_title['num_values'].value_counts().sort_index()

# calculating unique title values
unique_title_values = set()
for val in df_title['places_bef_lemm']:
    for item in val.split(","):
        item = item.strip()
        if item:
            unique_title_values.add(item)
unique_title_count = len(unique_title_values)

# for fond (provenance_places_bef_lemm)
df_fond = df_consolidated_copy[df_consolidated_copy['provenance_places_bef_lemm'] != ""].copy(
)
df_fond['num_values'] = df_fond['provenance_places_bef_lemm'].apply(
    lambda x: len([val.strip() for val in x.split(",") if val.strip() != ""]))
total_fond_rows = len(df_fond)
total_fond_values = df_fond['num_values'].sum()
dist_fond = df_fond['num_values'].value_counts().sort_index()

# calculating unique fond values
unique_fond_values = set()
for val in df_fond['provenance_places_bef_lemm']:
    for item in val.split(","):
        item = item.strip()
        if item:
            unique_fond_values.add(item)
unique_fond_count = len(unique_fond_values)

print("
detailed statistics: ")
print("total number of rows in consolidated data:", total_rows)

print("
for title(places): ")
print("number of rows with inserted title values:", total_title_rows,
      "(", round(total_title_rows / total_rows * 100, 2), "% )")
print("total number of title values inserted:", total_title_values)
print("number of unique title values:", unique_title_count)
print("distribution of number of title values per row:")
for num, count in dist_title.items():
    print("  number of rows with", num, "value(s):", count,
          "(", round(count / total_title_rows * 100, 2), "% )")

print("
for fond(provenance_places): ")
print("number of rows with inserted fond values:", total_fond_rows,
      "(", round(total_fond_rows / total_rows * 100, 2), "% )")
print("total number of fond values inserted:", total_fond_values)
print("number of unique fond values:", unique_fond_count)
print("distribution of number of fond values per row:")
for num, count in dist_fond.items():
    print("  number of rows with", num, "value(s):", count,
          "(", round(count / total_fond_rows * 100, 2), "% )")

# displaying 25 random rows with non-empty title values (showing only 'id'
# and 'places_bef_lemm')
df_places_nonempty = df_consolidated_copy[df_consolidated_copy['places_bef_lemm'] != ""]
if len(df_places_nonempty) > 25:
    df_places_sample = df_places_nonempty.sample(25, random_state=42)
else:
    df_places_sample = df_places_nonempty
display(df_places_sample[['id', 'places_bef_lemm']])

# displaying 10 random rows with unique fond values (showing only 'id' and
# 'provenance_places_bef_lemm')
df_provenance_nonempty = df_consolidated_copy[df_consolidated_copy['provenance_places_bef_lemm'] != ""]
df_provenance_unique = df_provenance_nonempty.drop_duplicates(
    subset=['provenance_places_bef_lemm'])
if len(df_provenance_unique) > 10:
    df_provenance_sample = df_provenance_unique.sample(10, random_state=42)
else:
    df_provenance_sample = df_provenance_unique
display(df_provenance_sample[['id', 'provenance_places_bef_lemm']])

consolidated data loaded, columns 'places' and 'provenance_places' cleared and renamed.
unique extracted places data loaded and copied.
unique extracted places deduplicated based on 'man_processed_extracted_places' and 'source'.
consolidated data updated with unique extracted places.
updated consolidated data saved to excel file at:
/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/consolidated_data_working_copy_with_title_and_provenance_places_bef_lemm.xlsx

detailed statistics:
total number of rows in consolidated data: 19740

for title (places):
number of rows with inserted title values: 5934 ( 30.06 % )
total number of title values inserted: 9209
number of unique title values: 1772
distribution of number of title values per row:
  number of rows with 1 value(s): 3650 ( 61.51 % )
  number of rows with 2 value(s): 1736 ( 29.26 % )
  number of rows with 3 value(s): 369 ( 6.22 % )
  number of rows with 4 value(s): 74 ( 1.25 % )
  number of rows with 5 value(s): 4

,id,places_bef_lemm
4227,4228,Польщі
4213,4214,Польщі
5387,5388,"Перемишляни, Перемишлянського повіту"
6756,6757,"Бережанського району, Рогачин"
6529,6530,"Бережанський райони, Бучацький райони, Вишніве..."
4550,4551,Палестину
3859,3860,Тернопільського
8205,8206,Борщів
12880,12881,Бобровокутської сільради
6851,6852,УРСР


,id,provenance_places_bef_lemm
8064,8065,"Галицького, Скалат, Скалатський повітовий"
15873,15874,Бобровокутська сільська рада
12394,12395,Херсона
10120,10121,"Заліщики, Заліщицький"
9653,9654,Буданівський
12773,12774,Калініндорфська районна
12391,12392,Херсонська окружна
18580,18581,Херсонської округи
16040,16041,"Одеської губернської, Херсонський"
10151,10152,Чорткові


# (MANUAL LEMMATIZATION USING TRANSLATION TO & FROM ENGLISH)

#   POST-LEMMATIZATION PROCESSING

In [ ]:
# @title DEDUPLICATION OF LEMMATIZED PLACES

# loading the input file with columns: source_ids,
# man_processed_extracted_places, final_places, source
input_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/final_places_after_lemm_bef_dedupl.xlsx"
df = pd.read_excel(input_file)
original_count = len(df)

# group rows by final_places and source (only merge groups where source
# values are equal)
grouped = df.groupby(["final_places", "source"], as_index=False)

merged_rows = []
merged_group_sizes = []  # will store the number of records merged in each group

for name, group in grouped:
    if len(group) > 1:
        # merge source_ids: split each string by comma, convert to int, take
        # union and sort ascending
        all_ids = set()
        for ids in group["source_ids"]:
            all_ids.update([int(x.strip())
                           for x in str(ids).split(",") if x.strip().isdigit()])
        merged_source_ids = ", ".join(str(x) for x in sorted(all_ids))
        # merge man_processed_extracted_places: split each string by comma,
        # take unique trimmed parts, sort alphabetically
        all_places = set()
        for val in group["man_processed_extracted_places"]:
            parts = [x.strip() for x in str(val).split(",") if x.strip() != ""]
            all_places.update(parts)
        merged_man_places = ", ".join(sorted(all_places))
        # record the group size (number of records merged)
        merged_group_sizes.append(len(group))
        merged_row = {
            "source_ids": merged_source_ids,
            "man_processed_extracted_places": merged_man_places,
            "final_places": group["final_places"].iloc[0],
            "source": group["source"].iloc[0]
        }
        merged_rows.append(merged_row)
    else:
        merged_rows.append(group.iloc[0].to_dict())

df_dedupl = pd.DataFrame(merged_rows)
dedupl_count = len(df_dedupl)

# aggregated statistics for duplicate groups before merge (groups with
# more than one record)
group_by_final = df.groupby("final_places")
duplicate_groups = group_by_final.filter(lambda x: len(x) > 1)
total_dup_groups = duplicate_groups["final_places"].nunique()

# distribution: count frequency of group sizes (number of records merged
# into one final_places)
dist = {}
for size in merged_group_sizes:
    dist[size] = dist.get(size, 0) + 1

# count groups not merged due to differing source values:
non_merged_due_to_source = 0
for final_val, group in df.groupby("final_places"):
    if group["source"].nunique() > 1:
        non_merged_due_to_source += 1

# print detailed deduplication report
print("deduplication report:")
print(f"original number of records: {original_count}")
print(f"number of records after deduplication: {dedupl_count}")
merged_records = original_count - dedupl_count
print(
    f"number of records merged (reduction): {merged_records} ({round(merged_records/original_count*100,2)}% reduction)")

# total number of merged duplicate groups (groups with size > 1)
merged_dup_groups = len([g for g in grouped if len(g[1]) > 1])
print(f"
total duplicate groups(with more than one record): {merged_dup_groups}")

print("
distribution of merged groups by group size(number of records merged): ")
for size in sorted(dist.keys(), reverse=True):
    count_groups = dist[size]
    pct = round(
    count_groups /
    merged_dup_groups *
    100,
     2) if merged_dup_groups else 0
    print(
        f"  group size {size}: {count_groups} groups ({pct}% of merged groups)")

print("
number of duplicate groups not merged due to differing source values: ")
pct_non_merged = round(
    non_merged_due_to_source /
    total_dup_groups *
    100,
     2) if total_dup_groups else 0
print(f"  {non_merged_due_to_source} groups ({pct_non_merged}% of duplicate groups)")

# display 30 random rows from the deduplicated dataframe (for reference)
print("
random sample from deduplicated dataset: ")
display(df_dedupl.sample(30, random_state=42))

# saving the deduplicated file to output file
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/final_places_after_lemm_after_dedupl.xlsx"
df_dedupl.to_excel(output_file, index=False)
print(f"
deduplicated file saved at: {output_file}")

deduplication report:
original number of records: 2004
number of records after deduplication: 1211
number of records merged (reduction): 793 (39.57% reduction)

total duplicate groups (with more than one record): 320

distribution of merged groups by group size (number of records merged):
  group size 20: 1 groups (0.31% of merged groups)
  group size 15: 1 groups (0.31% of merged groups)
  group size 14: 1 groups (0.31% of merged groups)
  group size 12: 3 groups (0.94% of merged groups)
  group size 11: 3 groups (0.94% of merged groups)
  group size 10: 1 groups (0.31% of merged groups)
  group size 8: 5 groups (1.56% of merged groups)
  group size 7: 7 groups (2.19% of merged groups)
  group size 6: 15 groups (4.69% of merged groups)
  group size 5: 26 groups (8.12% of merged groups)
  group size 4: 44 groups (13.75% of merged groups)
  group size 3: 74 groups (23.12% of merged groups)
  group size 2: 139 groups (43.44% of merged groups)

number of duplicate groups not merged due to

,source_ids,man_processed_extracted_places,final_places,source
101,"4005, 4011, 4036, 4800, 4807, 4808, 4821, 4847...","Бучацьким повітовим, Бучацького повіту",Бучацький повіт,title
787,"4730, 4734, 4795, 4857, 4899, 4935, 4988, 5005...","Підгаєцьким повітовим, Підгаєцького повіту, Пі...",Підгаєцький повіт,title
899,6729,Солоне,Солоне,title
109,"6511, 6513, 6515, 6517, 6528, 6535, 6546, 6644...","Білобожницький район, Білобожницький райони, Б...",Білобожницький район,title
650,"4886, 5175, 5402, 5436, 5576, 5647, 5804, 6759...",Нараїв,Нараїв,title
462,"6435, 6546, 6552, 6556, 6617, 6642, 6660, 6661...","Копичинецький райони, Копичинецького району, К...",Копичинецький район,title
760,"6490, 6642",Полівці,Полівці,title
49,"16089, 16109, 16175, 16552","Березнегувате, Березнегуватського",Березнегувате,title
479,12713,Красний Бургун,Красний Бургун,title
963,6405,Торське,Торське,title



deduplicated file saved at: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/final_places_after_lemm_after_dedupl.xlsx


In [ ]:
# @title MAPPING OLD ID ON MASTER

# defining the path for datasets
path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# reading the consolidated (cons) and master files
cons_file = path + "/consolidated_data_working_copy_with_title_and_provenance_places.xlsx"
master_file = path + "/t_Master_table_draft_dana_withdates.xlsx"
cons_df = pd.read_excel(cons_file)
master_df = pd.read_excel(master_file)

# creating key columns by concatenating relevant columns after filling na
# and stripping spaces
cons_df["key_cons"] = cons_df["all_title"].fillna("").astype(
    str).str.strip() + cons_df["Act"].fillna("").astype(str).str.strip()
master_df["key_master"] = master_df["ACT_TITLE_ORIGINAL"].fillna("").astype(
    str).str.strip() + master_df["SIGNATURE_act"].fillna("").astype(str).str.strip()

# filtering out rows where all_title or ACT_TITLE_ORIGINAL are empty
# if either cons.all_title or master.ACT_TITLE_ORIGINAL is empty, set key
# to None so they won't match
cons_df.loc[cons_df["all_title"].fillna("").astype(
    str).str.strip() == "", "key_cons"] = None
master_df.loc[master_df["ACT_TITLE_ORIGINAL"].fillna(
    "").astype(str).str.strip() == "", "key_master"] = None

# printing initial record counts for reference
print("initial cons records:", cons_df.shape[0])
print("initial master records:", master_df.shape[0])

# merging master with cons on the created key columns and inserting the
# mapped 'id' as old_id in the master copy
merged_df = master_df.merge(
    cons_df[["id", "key_cons"]], how="left", left_on="key_master", right_on="key_cons")
merged_df.insert(0, "old_id", merged_df.pop("id"))
merged_df.drop(columns=["key_cons", "key_master"], inplace=True)

# saving the resulting dataframe to the target file
target_file = path + "/t_Master_table_draft_dana_withdates_mapped.xlsx"
merged_df.to_excel(target_file, index=False)
print("target file saved at:", target_file)

# calculating matching statistics
cons_total = cons_df.shape[0]
master_total = master_df.shape[0]
master_keys_set = set(master_df["key_master"].dropna().unique())
cons_df["is_matched_in_master"] = cons_df["key_cons"].apply(
    lambda x: x in master_keys_set)
cons_matched_count = cons_df["is_matched_in_master"].sum()
cons_unmatched_count = cons_total - cons_matched_count
master_matched_count = merged_df["old_id"].notna().sum()
master_unmatched_count = master_total - master_matched_count
old_id_counts = merged_df.loc[merged_df["old_id"].notna(
), "old_id"].value_counts()
duplicate_count = (old_id_counts > 1).sum()
total_matches = master_matched_count

print("
--- matching statistics - --")
print("cons total records:", cons_total)
print(
    "cons matched records:",
    cons_matched_count,
    "({:.2f}%)".format(
        100 *
        cons_matched_count /
         cons_total))
print(
    "cons unmatched records:",
    cons_unmatched_count,
    "({:.2f}%)".format(
        100 *
        cons_unmatched_count /
         cons_total))
print("
master total records: ", master_total)
print(
    "master matched records:",
    master_matched_count,
    "({:.2f}%)".format(
        100 *
        master_matched_count /
         master_total))
print(
    "master unmatched records:",
    master_unmatched_count,
    "({:.2f}%)".format(
        100 *
        master_unmatched_count /
         master_total))
print("
duplicate old_id count (each duplicate counted once): ", duplicate_count, "({: .2f} %)".format(100 * duplicate_count / total_matches if total_matches > 0 else 0))

# displaying sample groups with identical act_title_original
mapped_df = pd.read_excel(target_file)
grouped = mapped_df.groupby("ACT_TITLE_ORIGINAL")
groups_with_duplicates = [group for name,
    group in grouped if group.shape[0] > 1]

if groups_with_duplicates:
    sample_groups = random.sample(
    groups_with_duplicates, min(
        4, len(groups_with_duplicates)))
    print("
--- displaying 4 random groups with identical act_title_original - --")
    for idx, group in enumerate(sample_groups, start=1):
        print("
group", idx, " - act_title_original: ", group["ACT_TITLE_ORIGINAL"].iloc[0])
        display(group[["old_id", "ACT_TITLE_ORIGINAL", "SIGNATURE_act"]])
else:
    print("no groups with duplicate act_title_original found.")

# displaying rows with duplicate old_id values
duplicate_ids = old_id_counts[old_id_counts > 1].index.tolist()
duplicates_df = mapped_df[mapped_df["old_id"].isin(duplicate_ids)]
print("
--- displaying rows with duplicate old_id values - --")
display(duplicates_df[["old_id", "ACT_TITLE_ORIGINAL", "SIGNATURE_act"]])

initial cons records: 19740
initial master records: 18466
target file saved at: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/t_Master_table_draft_dana_withdates_mapped.xlsx

--- matching statistics ---
cons total records: 19740
cons matched records: 14630 (74.11%)
cons unmatched records: 5110 (25.89%)

master total records: 18466
master matched records: 15154 (82.06%)
master unmatched records: 3312 (17.94%)

duplicate old_id count (each duplicate counted once): 348 (2.30%)

--- displaying 4 random groups with identical act_title_original ---

group 1 - act_title_original: Тульчинский Юрий Михайлович


,old_id,ACT_TITLE_ORIGINAL,SIGNATURE_act
3010,2945.0,Тульчинский Юрий Михайлович,50
3011,2947.0,Тульчинский Юрий Михайлович,47



group 2 - act_title_original: Звітні відомості про доходи та видатки сільради


,old_id,ACT_TITLE_ORIGINAL,SIGNATURE_act
13977,15073.0,Звітні відомості про доходи та видатки сільради,26
13978,15119.0,Звітні відомості про доходи та видатки сільради,72
13979,15134.0,Звітні відомості про доходи та видатки сільради,87



group 3 - act_title_original: Накази голови заготконтори


,old_id,ACT_TITLE_ORIGINAL,SIGNATURE_act
14268,15819.0,Накази голови заготконтори,118
14269,15829.0,Накази голови заготконтори,128



group 4 - act_title_original: Дело Райза Владимира на получение шофёрских прав. 10 сентября 1928 г. – 16 апреля 1934 г. 7 листов.


,old_id,ACT_TITLE_ORIGINAL,SIGNATURE_act
2005,1079.0,Дело Райза Владимира на получение шофёрских пр...,7410
2006,1947.0,Дело Райза Владимира на получение шофёрских пр...,7410



--- displaying rows with duplicate old_id values ---


,old_id,ACT_TITLE_ORIGINAL,SIGNATURE_act
295,83.0,"Выписки из книги актов гражданского состояния,...",127
296,1094.0,"Выписки из книги актов гражданского состояния,...",127
297,1095.0,"Выписки из книги актов гражданского состояния,...",127
298,83.0,"Выписки из книги актов гражданского состояния,...",127
299,1094.0,"Выписки из книги актов гражданского состояния,...",127
...,...,...,...
18737,14715.0,Те саме,11
18738,15896.0,Те саме,11
18739,13508.0,Те саме,15
18740,13601.0,Те саме,15


In [ ]:
# @title EXTRACTING UNMATCHED ROWS

# defining the path for datasets
path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# reading the consolidated (cons) and master files
cons_file = path + "/consolidated_data_working_copy_with_title_and_provenance_places.xlsx"
master_file = path + "/t_Master_table_draft_dana_withdates.xlsx"
cons_df = pd.read_excel(cons_file)
master_df = pd.read_excel(master_file)

# creating key columns based solely on titles after filling na and
# stripping spaces
cons_df["key_cons"] = cons_df["all_title"].fillna("").astype(str).str.strip()
master_df["key_master"] = master_df["ACT_TITLE_ORIGINAL"].fillna(
    "").astype(str).str.strip()

# filtering out rows where title is empty by setting key to None
cons_df.loc[cons_df["all_title"].fillna("").astype(
    str).str.strip() == "", "key_cons"] = None
master_df.loc[master_df["ACT_TITLE_ORIGINAL"].fillna(
    "").astype(str).str.strip() == "", "key_master"] = None

# extracting sets of keys from each file (excluding None)
cons_keys = set(cons_df["key_cons"].dropna().unique())
master_keys = set(master_df["key_master"].dropna().unique())

# filtering unmatched rows:
# for master: select rows with non-empty act_title_original and key_master
# not in cons_keys
nonempty_master_unmatched = master_df[
    (master_df["ACT_TITLE_ORIGINAL"].fillna("").astype(str).str.strip() != "") &
    (master_df["key_master"].isna() |
     (~master_df["key_master"].isin(cons_keys)))
]

# for consolidated: select rows with non-empty all_title and key_cons not
# in master_keys
nonempty_consolidated_unmatched = cons_df[
    (cons_df["all_title"].fillna("").astype(str).str.strip() != "") &
    (cons_df["key_cons"].isna() | (~cons_df["key_cons"].isin(master_keys)))
]

# exporting the resulting dataframes to excel files
master_unmatched_file = path + "/nonempty_master_unmatched.xlsx"
cons_unmatched_file = path + "/nonempty_consolidated_unmatched.xlsx"

nonempty_master_unmatched.to_excel(master_unmatched_file, index=False)
nonempty_consolidated_unmatched.to_excel(cons_unmatched_file, index=False)

print("nonempty_master_unmatched saved at:", master_unmatched_file)
print("nonempty_consolidated_unmatched saved at:", cons_unmatched_file)

# calculating statistics for unmatched rows
total_master_nonempty = master_df[master_df["ACT_TITLE_ORIGINAL"].fillna(
    "").astype(str).str.strip() != ""].shape[0]
total_cons_nonempty = cons_df[cons_df["all_title"].fillna(
    "").astype(str).str.strip() != ""].shape[0]

master_unmatched_count = nonempty_master_unmatched.shape[0]
cons_unmatched_count = nonempty_consolidated_unmatched.shape[0]

print("
--- master unmatched statistics - --")
print("total master rows with non-empty title:", total_master_nonempty)
print(
    "master unmatched rows:",
    master_unmatched_count,
    "({:.2f}%)".format(
        100 *
        master_unmatched_count /
         total_master_nonempty if total_master_nonempty > 0 else 0))

print("
--- consolidated unmatched statistics - --")
print("total consolidated rows with non-empty title:", total_cons_nonempty)
print(
    "consolidated unmatched rows:",
    cons_unmatched_count,
    "({:.2f}%)".format(
        100 *
        cons_unmatched_count /
         total_cons_nonempty if total_cons_nonempty > 0 else 0))

# displaying a random sample of 20 rows from each unmatched dataframe
# using display
if master_unmatched_count > 0:
    print("
--- displaying random sample of 20 rows from nonempty_master_unmatched - --")
    display(nonempty_master_unmatched.sample(n=20, random_state=42)
            [["ACT_TITLE_ORIGINAL", "SIGNATURE_act"]].fillna(""))
else:
    print("
no unmatched rows found in master.")

if cons_unmatched_count > 0:
    print("
--- displaying random sample of 20 rows from nonempty_consolidated_unmatched - --")
    display(nonempty_consolidated_unmatched.sample(
        n=20, random_state=42)[["id", "all_title", "Act"]].fillna(""))
else:
    print("
no unmatched rows found in consolidated.")

nonempty_master_unmatched saved at: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/nonempty_master_unmatched.xlsx
nonempty_consolidated_unmatched saved at: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/nonempty_consolidated_unmatched.xlsx

--- master unmatched statistics ---
total master rows with non-empty title: 18466
master unmatched rows: 852 (4.61%)

--- consolidated unmatched statistics ---
total consolidated rows with non-empty title: 18578
consolidated unmatched rows: 140 (0.75%)

--- displaying random sample of 20 rows from nonempty_master_unmatched ---


,ACT_TITLE_ORIGINAL,SIGNATURE_act
17680,АКС на Кінзерського,
18048,Протоколи загальних зборів та засідань правлін...,60
17812,"АКС стосовно Гартена Самуеля Якубовича, т. 1",
17826,"АКС стосовно Аугонбраума Шимона Елляшовича, т. 1",
18265,Статистичні картки обліку населення по дворах ...,304
18157,Протоколи загальних зборів та засідань правлін...,236
17894,АКС стосовно Біленького Іцека Танхеновича,
17910,АКС стосовно Гершоновича Генріха Герсовича,
17979,"Справа про використання націоналізованих, кину...",116
18292,Статистичні картки обліку населення по дворах ...,333



--- displaying random sample of 20 rows from nonempty_consolidated_unmatched ---


,id,all_title,Act
16798,16799,Справа про конфіскацію контрабандних галантере...,285
6549,6550,"Хроніка подій ""Вісті з терену. Ска-латчина"" із...",
6512,6513,Протоколи дізнань виявлених агентів органів мі...,
16809,16810,Справа про конфіскацію рибальського начиння у ...,431
6523,6524,Протоколи допитів затриманих бійців гарнізону ...,
1023,1024,Переписка с Здолбуновским староством о выдаче ...,6391а
7441,7442,Мигович И.,126
6561,6562,Список підпільників полеглих на Тернопільщині ...,
16794,16795,Справа про конфіскацію контрабандних галантере...,276
16799,16800,Справа про конфіскацію контрабандного рибальсь...,340


#   MATCHING WITH SAPIR

In [ ]:
# @title PLACE MATCHING WITH SAPIR TABLE - REGEX & FUZZY MATCH

# define similarity threshold for fuzzy matching
similarity_threshold = 85

# define mapping dictionaries
base_dict = {
    'а': ['a'], 'б': ['b'], 'в': ['v', 'w'], 'г': ['h', 'g'], 'ґ': ['g'],
    'д': ['d', 'j'], 'е': ['e', 'ye', 'ie'], 'є': ['ye', 'je', 'ie'],
    'ж': ['zh', 'j', 'rz'], 'з': ['z'], 'и': ['y', 'i'], 'і': ['i'],
    'ї': ['yi', 'ji', 'ii', 'i', 'e', 'ye', 'ie', 'je'], 'й': ['y', 'j', 'i'],
    'к': ['k', 'c', 'q'], 'л': ['l'], 'м': ['m'], 'н': ['n'], 'о': ['o'],
    'п': ['p'], 'р': ['r'], 'с': ['s'], 'т': ['t'], 'у': ['u', 'ou'],
    'ф': ['f', 'ph'], 'х': ['kh', 'ch', 'h'], 'ц': ['ts', 'tz', 'z', 'c'],
    'ч': ['ch', 'tch', 'tsch', 'cz'], 'ш': ['sh', 'sch', 'sz'], 'щ': ['shch', 'sch', 'sc', 'szcz'],
    'ю': ['yu', 'ju', 'iu'], 'я': ['ya', 'ja', 'ia']
}

extended_dict = {
    'а': ['a'], 'б': ['b'], 'в': ['v', 'w', 'l'], 'г': ['h', 'g'], 'ґ': ['g'],
    'д': ['d', 'j'], 'е': ['e', 'ye', 'ie'], 'є': ['ye', 'je', 'ie', 'e'],
    'ж': ['zh', 'j', 'rz'], 'з': ['z'], 'и': ['y', 'i'], 'і': ['i', 'o', 'y', 'e'],
    'ї': ['yi', 'ji', 'ii', 'i', 'e', 'ye', 'ie', 'je'], 'й': ['y', 'j', 'i'],
    'к': ['k', 'c', 'q'], 'л': ['l', 'ł'], 'м': ['m'], 'н': ['n'], 'о': ['o'],
    'п': ['p'], 'р': ['r', 'rz'], 'с': ['s'], 'т': ['t'], 'у': ['u', 'ou', 'ó'],
    'ф': ['f', 'ph'], 'х': ['kh', 'ch', 'h'], 'ц': ['ts', 'tz', 'z', 'c'],
    'ч': ['ch', 'tch', 'tsch', 'cz'], 'ш': ['sh', 'sch', 'sz'], 'щ': ['shch', 'sch', 'sc', 'szcz'],
    'ю': ['yu', 'ju', 'iu'], 'я': ['ya', 'ja', 'ia', 'a']
}

def build_regex_pattern(query_cyr, ext_dict):
    pattern = ""
    for char in query_cyr:
        if char in ext_dict:
            alternatives = ext_dict[char]
            pattern += "(" + "|".join(re.escape(alt)
                         for alt in alternatives) + ")"
        else:
            pattern += re.escape(char)
    return pattern

# define file paths
final_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/final_places_mach_transl_with_fixed.xlsx"
sapir_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/SAPIR_TABLES/t_SC_Places_Items(ENG_HEB)_Values.xlsx"

# loading excel files
display(Markdown("**loading excel files...**"))
df_final = pd.read_excel(final_path)
df_sapir = pd.read_excel(sapir_path)

df_final["final_places_eng_norm"] = df_final["final_places_eng"].astype(
    str).str.lower().str.strip()
df_sapir["place_norm"] = df_sapir["place"].astype(str).str.lower().str.strip()

df_final_place = df_final[df_final["sapir_category"].astype(
    str).str.lower() == "place"].copy()

# precompute regex expressions
df_final_place["final_places_regex"] = df_final_place["final_places"].str.lower(
).map(lambda x: build_regex_pattern(str(x), extended_dict))

output_rows = []
match_counts = {}
regex_match_count = 0
fuzzy_match_count = 0
unique_regex_matches = set()
unique_fuzzy_matches = set()

for idx, row in df_final_place.iterrows():
    query_regex = row["final_places_regex"]
    query_eng = row["final_places_eng_norm"]

    source_first = row["final_places"][0].lower(
    ) if row["final_places"] else ""

    filtered_candidates_base = []
    for match_index, sapir_row in df_sapir.iterrows():
        candidate_first = sapir_row["place_norm"][0] if sapir_row["place_norm"] else ""
        if source_first in base_dict and candidate_first in base_dict[source_first]:
            filtered_candidates_base.append(match_index)

    filtered_candidates_regex = []
    for match_index in filtered_candidates_base:
        candidate_norm = df_sapir.iloc[match_index]["place_norm"]
        if re.fullmatch(query_regex, candidate_norm):
            filtered_candidates_regex.append(match_index)

    if filtered_candidates_regex:
        final_candidates = filtered_candidates_regex
        regex_match_count += len(filtered_candidates_regex)
        method = "regex"
    else:
        matches = process.extract(
    query_eng,
    df_sapir["place_norm"].tolist(),
    scorer=fuzz.token_sort_ratio,
    score_cutoff=similarity_threshold,
     limit=None)
        final_candidates = [match[2] for match in matches]
        fuzzy_match_count += len(final_candidates)
        method = "fuzzy"

    # filter by keywords if multiple matches exist
    if len(final_candidates) > 1:
        keyword_candidates = [
            idx for idx in final_candidates if any(
                keyword in df_sapir.iloc[idx]["TIENG"] for keyword in ["Ukraine", "Ruthenia", "Poland"]
            )
        ]
        if keyword_candidates:
            final_candidates = keyword_candidates

    match_counts[row["final_places_eng"]] = len(final_candidates)

    final_eng_val = row["final_places_eng"]
    if final_candidates:
        if method == "regex":
            unique_regex_matches.add(final_eng_val)
            unique_fuzzy_matches.discard(final_eng_val)
        else:
            if final_eng_val not in unique_regex_matches and final_eng_val not in unique_fuzzy_matches:
                unique_fuzzy_matches.add(final_eng_val)

    if not final_candidates:
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "PLACE_STRING_CYR": row["final_places"],
            "PLACE_STRING_ENG": row["final_places_eng"],
            "SAPIR_PLACE_MATCH": "",
            "SAPIR_PLACE_ID": "",
            "SAPIR_PLACE_VALUE": "",
            "COUNT_MATCHES": 0
        })

    for match_index in final_candidates:
        sapir_row = df_sapir.iloc[match_index]
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "PLACE_STRING_CYR": row["final_places"],
            "PLACE_STRING_ENG": row["final_places_eng"],
            "SAPIR_PLACE_MATCH": sapir_row["place"],
            "SAPIR_PLACE_ID": sapir_row["book_id"],
            "SAPIR_PLACE_VALUE": sapir_row["TIENG"],
            "COUNT_MATCHES": len(final_candidates)
        })

df_output = pd.DataFrame(output_rows)
df_output.insert(0, "ID", range(1, len(df_output) + 1))

output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_sapir_regex _&_fuzzy_match_place.xlsx"
df_output.to_excel(output_file, index=False)

# summary statistics for final_places_eng values (unique values in df_final_place)
total_values = df_final_place["final_places_eng"].nunique()
values_with_matches = sum(1 for count in match_counts.values() if count > 0)
values_with_no_matches = sum(1 for count in match_counts.values() if count == 0)

# build distribution of match counts
match_distribution = {}
for count in match_counts.values():
    match_distribution[count] = match_distribution.get(count, 0) + 1

# prepare summary report string for overall final_places_eng matching
report_lines = []
report_lines.append(f"total unique final_places_eng values processed: {total_values}")
report_lines.append(f"values with one or more matches: {values_with_matches} ({(values_with_matches/total_values*100):.2f}%)")
report_lines.append(f"values with no matches: {values_with_no_matches} ({(values_with_no_matches/total_values*100):.2f}%)")
for count in sorted(match_distribution.keys()):
    report_lines.append(f"values with exactly {count} match(es): {match_distribution[count]} ({(match_distribution[count]/total_values*100):.2f}%)")
report_text = "

".join(report_lines)
display(Markdown("**summary report:**"))
display(Markdown(report_text))

# unique matching method statistics for final_places_eng (without duplicates)
unique_total = len(unique_regex_matches.union(unique_fuzzy_matches))
if unique_total > 0:
    unique_regex_percentage = len(unique_regex_matches) / unique_total * 100
    unique_fuzzy_percentage = len(unique_fuzzy_matches) / unique_total * 100
else:
    unique_regex_percentage = 0
    unique_fuzzy_percentage = 0
display(Markdown("**unique match method statistics (by final_places_eng):**"))
display(Markdown(f"unique regex matches: {len(unique_regex_matches)} ({unique_regex_percentage:.2f}%)"))
display(Markdown(f"unique fuzzy matches: {len(unique_fuzzy_matches)} ({unique_fuzzy_percentage:.2f}%)"))

# display 20 random rows with matches and unique PLACE_STRING_CYR and PLACE_STRING_ENG
df_matches = df_output[df_output["COUNT_MATCHES"] > 0].copy()
# drop duplicates based on PLACE_STRING_CYR and PLACE_STRING_ENG
df_unique_places = df_matches.drop_duplicates(subset=["PLACE_STRING_CYR", "PLACE_STRING_ENG"])
if len(df_unique_places) >= 20:
    sample_df = df_unique_places.sample(n=20, random_state=42)
else:
    sample_df = df_unique_places
display(Markdown("**20 random sample rows with matches (unique PLACE_STRING_CYR and PLACE_STRING_ENG):**"))
display(sample_df[["PLACE_STRING_CYR", "PLACE_STRING_ENG", "SAPIR_PLACE_MATCH", "SAPIR_PLACE_ID", "SAPIR_PLACE_VALUE", "COUNT_MATCHES"]])

display(Markdown(f"**output file saved at:** {output_file}"))
display(Markdown("**fuzzy matching process completed successfully.**"))

**loading excel files...**

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


**summary report:**

total unique final_places_eng values processed: 760

values with one or more matches: 544 (71.58%)

values with no matches: 216 (28.42%)

values with exactly 0 match(es): 216 (28.42%)

values with exactly 1 match(es): 167 (21.97%)

values with exactly 2 match(es): 105 (13.82%)

values with exactly 3 match(es): 92 (12.11%)

values with exactly 4 match(es): 47 (6.18%)

values with exactly 5 match(es): 31 (4.08%)

values with exactly 6 match(es): 17 (2.24%)

values with exactly 7 match(es): 16 (2.11%)

values with exactly 8 match(es): 9 (1.18%)

values with exactly 9 match(es): 8 (1.05%)

values with exactly 10 match(es): 4 (0.53%)

values with exactly 11 match(es): 12 (1.58%)

values with exactly 12 match(es): 6 (0.79%)

values with exactly 13 match(es): 1 (0.13%)

values with exactly 14 match(es): 6 (0.79%)

values with exactly 15 match(es): 3 (0.39%)

values with exactly 16 match(es): 5 (0.66%)

values with exactly 17 match(es): 3 (0.39%)

values with exactly 18 match(es): 2 (0.26%)

values with exactly 20 match(es): 1 (0.13%)

values with exactly 21 match(es): 2 (0.26%)

values with exactly 22 match(es): 1 (0.13%)

values with exactly 29 match(es): 1 (0.13%)

values with exactly 30 match(es): 1 (0.13%)

values with exactly 40 match(es): 1 (0.13%)

values with exactly 42 match(es): 1 (0.13%)

values with exactly 45 match(es): 1 (0.13%)

values with exactly 55 match(es): 1 (0.13%)

**unique match method statistics (by final_places_eng):**

...

unique fuzzy matches: 175 (32.17%)

**20 random sample rows with matches (unique PLACE_STRING_CYR and PLACE_STRING_ENG):**

,PLACE_STRING_CYR,PLACE_STRING_ENG,SAPIR_PLACE_MATCH,SAPIR_PLACE_ID,SAPIR_PLACE_VALUE,COUNT_MATCHES
2648,Янковцы,Yankovtsy,Yankovtsy,14669477,"Yankovtsy,Kazatin,Vinnitsa,Ukraine (USSR)",9
362,Владимир,Vladimir,Vladimir,5440403,"Vladimir,Vladimir City,Ivanovo,Russia (USSR)",2
647,Доманинка,Domaninka,Domanin,5462908,"Domanin,Nove Mesto na Morave,Moravia-Silesia,C...",1
351,Відень,Vienna,Vienna,5556689,"Vienna,Vienna,<>,Austria",3
1346,Лішня,Lishnya,Liszna,11232908,"Liszna,Lesko,Lwow,Poland",8
1828,Первомайськ,Pervomaisk,Pervomaisk,5526714,"Pervomaisk,Belovodsk,Voroshilovgrad,Ukraine (U...",16
2467,Угринківці,Ugrinkivtsi,Uhrynkowce,5448523,"Uhrynkowce,Zaleszczyki,Tarnopol,Poland",1
2389,Топорів,Toporiv,Toporow,5439440,"Toporow,Radziechow,Tarnopol,Poland",5
364,Владимир-Волынский,Vladimir Volynskiy,Vladimir Volynskiy,5557349,"Vladimir Volynskiy,Wlodzimierz,Wolyn,Poland",3
1844,Передмірка,Peredmirka,Peredmirka,5450461,"Peredmirka,Krzemieniec,Wolyn,Poland",1


**output file saved at:** /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_sapir_regex _&_fuzzy_match_place.xlsx

**fuzzy matching process completed successfully.**

In [ ]:
#@title UNPIVOTED PLACES WITH SAPIR

# define the directory where the files are located
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# define filenames for manual data and mapping data
manual_file = "places_sapir_regex _&_fuzzy_match_place_with_man.xlsx"
mapping_file = "t_Master_table_draft_dana_withdates_mapped.xlsx"

# read the excel files
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(file_dir, mapping_file))

# convert id columns to string types in both dataframes
# for mapping file, convert numeric values to int then to string to remove decimals
df_manual["source_ids"] = df_manual["source_ids"].astype(str)
df_mapping["old_id"] = df_mapping["old_id"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")
df_mapping["ID_master"] = df_mapping["ID_master"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")

# filter out rows where status equals 'to remove'
df_manual = df_manual[df_manual["STATUS"] != "to remove"].copy()

# for place_cyr, take PLACE_STRING_CYR (only for remaining rows)
df_manual["place_cyr"] = df_manual["PLACE_STRING_CYR"]

# for place_sapir, if status equals 'to use' then take SAPIR_PLACE_MATCH, else set to ''
df_manual["place_sapir"] = df_manual["SAPIR_PLACE_MATCH"].where(df_manual["STATUS"] == "to use", "")

# for place_missing, if status equals 'to export' then take PLACE_STRING_ENG, else empty string
df_manual["place_missing"] = df_manual["PLACE_STRING_ENG"].where(df_manual["STATUS"] == "to export", "")

# full_sapir from SAPIR_PLACE_VALUE
df_manual["full_sapir"] = df_manual["SAPIR_PLACE_VALUE"]

# split source_ids by comma into a list
df_manual["id_old_list"] = df_manual["source_ids"].apply(lambda x: [item.strip() for item in x.split(",") if item.strip() != ""])

# explode the source_ids into separate rows and rename the column to id_old
df_exploded = df_manual.explode("id_old_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"id_old_list": "id_old"})
df_exploded["id_old"] = df_exploded["id_old"].astype(str)

# function to convert id values: first to float, then to int, then to string
def conv_id(x):
    try:
        return str(int(float(x)))
    except:
        return x

# convert id_old to integer then to string to avoid decimals/extra zeros
df_exploded["id_old"] = df_exploded["id_old"].apply(conv_id)

# merge to bring in ID_master based on id_old matching old_id
df_merged = pd.merge(df_exploded, df_mapping[["old_id", "ID_master"]], left_on="id_old", right_on="old_id", how="left")
df_merged["ID_master"] = df_merged["ID_master"].fillna("missing")
df_merged = df_merged.rename(columns={"ID_master": "id_master"})
if "old_id_y" in df_merged.columns:
    df_merged = df_merged.drop(columns=["old_id_y"])

# convert id_master to integer then to string
df_merged["id_master"] = df_merged["id_master"].apply(conv_id)

# prepare final unpivoted output dataframe (insert new column 'sapir_id' from SAPIR_PLACE_ID after id_master)
cols_output = ["source", "place_cyr", "place_sapir", "place_missing", "id_old", "id_master", "SAPIR_PLACE_ID", "full_sapir", "MISSING_GPS"]
df_output = df_merged[cols_output].copy()
df_output = df_output.rename(columns={"SAPIR_PLACE_ID": "sapir_id"})

# convert sapir_id to integer then to string
df_output["sapir_id"] = df_output["sapir_id"].apply(conv_id)

# prepare the missing in sapir places output file
df_missing = df_manual[df_manual["STATUS"] == "to export"][["source_ids", "source", "PLACE_STRING_CYR", "PLACE_STRING_ENG", "MISSING_GPS"]].drop_duplicates()

# computing statistics
total_manual = df_manual.shape[0]
total_exploded = df_output.shape[0]
perc_explosion = (total_exploded / total_manual) * 100 if total_manual > 0 else 0
source_counts = df_output["source"].value_counts()
source_perc = (source_counts / source_counts.sum()) * 100
unique_place_by_source = df_output.groupby("source")["place_cyr"].nunique()
total_unique_place = df_output["place_cyr"].nunique()
unique_place_perc = (unique_place_by_source / total_unique_place) * 100 if total_unique_place > 0 else 0
status_counts = df_manual["STATUS"].value_counts()
missing_id_master_count = (df_output["id_master"] == "missing").sum()
perc_missing_id_master = (missing_id_master_count / total_exploded) * 100 if total_exploded > 0 else 0
total_missing_rows = df_missing.shape[0]
missing_gps_dashes = df_missing["MISSING_GPS"].astype(str).str.contains("---").sum()
perc_missing_gps_dashes = (missing_gps_dashes / total_missing_rows) * 100 if total_missing_rows > 0 else 0

# printing summary report in a condensed format
print("
--- summary report ---
")
print("total rows in manual file (status != 'to remove'): {} rows".format(total_manual))
print("total rows in output file after explosion: {} rows ({:.2f}%)".format(total_exploded, perc_explosion))
print("source breakdown in output file: " + ", ".join(["{}: {} rows ({:.2f}%)".format(src, cnt, source_perc[src]) for src, cnt in source_counts.items()]))
print("unique place_cyr count by source: " + ", ".join(["{}: {} unique values ({:.2f}%)".format(src, cnt, unique_place_perc[src]) for src, cnt in unique_place_by_source.items()]))
print("manual file status counts: " + ", ".join(["{}: {} rows ({:.2f}%)".format(status, cnt, (cnt/total_manual*100)) for status, cnt in status_counts.items()]))
print("count of 'missing' in id_master in output file: {} rows ({:.2f}%)".format(missing_id_master_count, perc_missing_id_master))
print("missing sapir places file statistics: total rows: {} rows, rows where MISSING_GPS contains '---': {} rows ({:.2f}%)".format(total_missing_rows, missing_gps_dashes, perc_missing_gps_dashes))

# saving output files
output_file = os.path.join(file_dir, "places_unpivoted_with_sapir_places.xlsx")
missing_file = os.path.join(file_dir, "places_missing_in_sapir_places.xlsx")
df_output.to_excel(output_file, index=False)
df_missing.to_excel(missing_file, index=False)
print("
output files saved: unpivoted file -> {}, missing sapir places file -> {}".format(output_file, missing_file))

# displaying random samples
sample_output = df_output.sample(n=100, random_state=42) if total_exploded >= 100 else df_output.copy()
sample_missing = df_missing.sample(n=20, random_state=42) if total_missing_rows >= 20 else df_missing.copy()
print("
--- sample of 100 random rows from output file ---")
display(sample_output)
print("
--- sample of 20 random rows from missing file ---")
display(sample_missing)


--- summary report ---

total rows in manual file (status != 'to remove'): 1369 rows
total rows in output file after explosion: 10549 rows (770.56%)
source breakdown in output file: fond: 5401 rows (51.20%), title: 5148 rows (48.80%)
unique place_cyr count by source: fond: 41 unique values (5.39%), title: 755 unique values (99.34%)
manual file status counts: to export: 841 rows (61.43%), to use: 528 rows (38.57%)
count of 'missing' in id_master in output file: 2716 rows (25.75%)
missing sapir places file statistics: total rows: 284 rows, rows where MISSING_GPS contains '---': 111 rows (39.08%)

output files saved: unpivoted file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_unpivoted_with_sapir_places.xlsx, missing sapir places file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_missing_in_sapir_places.xlsx

--- sample of 100 random rows from output file ---


,source,place_cyr,place_sapir,place_missing,id_old,id_master,sapir_id,full_sapir,MISSING_GPS
1671,title,Варшава,Varshava,,7923,455,5555992,"Varshava,Warszawa,Warszawa,Poland",NaN
2629,fond,Тернопіль,Ternopil,,10056,2519,5547367,"Ternopil,Tarnopol,Tarnopol,Poland",NaN
5817,fond,Луцьк,Lutsk,,92,6752,5511940,"Lutsk,Łuck,Wolyn,Poland",NaN
5831,fond,Луцьк,Lutsk,,1094,6734,5511940,"Lutsk,Łuck,Wolyn,Poland",NaN
7342,title,Луцк,Lutsk,,2142,18013,5511940,"Lutsk,Łuck,Wolyn,Poland",NaN
...,...,...,...,...,...,...,...,...,...
5216,fond,Калінінське,Kalininske,,14764,10724,5500289,"Kalininske,Kalinindorf,Nikolayev,Ukraine (USSR)",NaN
1669,title,Варшава,Varshava,,7921,453,5555992,"Varshava,Warszawa,Warszawa,Poland",NaN
761,title,Ворошилове,,Voroshilove,15602,11357,9914273,"Voroshilove,Tsebrikovo,Odessa,Ukraine (USSR)",---
10461,title,Іванівка,,Ivanovka,6433,missing,5696282,"Ivanovka,Nizhnyaya Duvanka,Voroshilovgrad,Ukra...",---



--- sample of 20 random rows from missing file ---


,source_ids,source,PLACE_STRING_CYR,PLACE_STRING_ENG,MISSING_GPS
10,6450,title,Буглів,Bugliv,"49.76344874401241, 26.05663477869299"
1563,6443,title,Лішня,Lishnya,"50.17434899743098, 25.816611357925012"
207,16640,title,Черовоно-Констянтинівка,Cherovono-Konstyantynovka,---
488,6442,title,Колосове,Kolosove,"50.03529026612753, 25.69145890816239"
95,"13771, 13854, 13880, 13884",title,колгосп Комінтерн Шоломалейхімської сільради,Komintern kolkhoz of the Sholom Aleichem rural...,---
845,6397,title,Поляни,Poliany,"49.871441494967314, 25.66948775243342"
1778,13961,title,Краснослав,Krasnoslav,---
249,"8248, 8249",title,Горожанка,Horozhanka,"49.14738040165738, 24.913828776018416"
58,"5058, 5084, 5258, 5259, 5355, 5687, 6418, 6536...",title,Золотники,Zolotnyky,"49.288053343232185, 25.391236924547215"
665,"6451, 6452",title,Великі Кусківці,Velyki Kuskivtsi,"49.871624686027616, 25.950683870401367"


In [ ]:
#@title DISTRICT MATCHING WITH SAPIR TABLE - REGEX & FUZZY MATCH

# define similarity threshold for fuzzy matching
similarity_threshold = 85

# define mapping dictionaries
base_dict = {'а': ['a'], 'б': ['b'], 'в': ['v', 'w'], 'г': ['h', 'g'], 'ґ': ['g'], 'д': ['d', 'j', 'dz'], 'е': ['e', 'ye', 'ie'], 'є': ['ye', 'je', 'ie', 'e'], 'ж': ['zh', 'j', 'rz', 'ż'], 'з': ['z'], 'и': ['y', 'i'], 'і': ['i'], 'ї': ['yi', 'ji', 'ii', 'i', 'e', 'ye', 'ie', 'je'], 'й': ['y', 'j', 'i'], 'к': ['k', 'c', 'q'], 'л': ['l', 'ł'], 'м': ['m'], 'н': ['n'], 'о': ['o', 'je'], 'п': ['p'], 'р': ['r', 'rz'], 'с': ['s'], 'т': ['t'], 'у': ['u', 'ou'], 'ф': ['f', 'ph'], 'х': ['kh', 'ch', 'h'], 'ц': ['ts', 'tz', 'z', 'c'], 'ч': ['ch', 'tch', 'tsch', 'cz'], 'ш': ['sh', 'sch', 'sz'], 'щ': ['shch', 'sch', 'sc', 'szcz'], 'ю': ['yu', 'ju', 'iu'], 'я': ['ya', 'ja', 'ia', 'je']}

extended_dict = {'а': ['a'], 'б': ['b'], 'в': ['v', 'w', 'l'], 'г': ['h', 'g'], 'ґ': ['g'], 'д': ['d', 'j', 'dz'], 'е': ['e', 'ye', 'ie'], 'є': ['ye', 'je', 'ie', 'e'], 'ж': ['zh', 'j', 'rz', 'ż'], 'з': ['z'], 'и': ['y', 'i', 'e'], 'і': ['i', 'o', 'y', 'e'], 'ї': ['yi', 'ji', 'ii', 'i', 'e', 'ye', 'ie', 'je'], 'й': ['y', 'j', 'i'], 'к': ['k', 'c', 'q'], 'л': ['l', 'ł'], 'м': ['m'], 'н': ['n'], 'о': ['o'], 'п': ['p'], 'р': ['r', 'rz'], 'с': ['s'], 'т': ['t'], 'у': ['u', 'ou', 'ó'], 'ф': ['f', 'ph'], 'х': ['kh', 'ch', 'h'], 'ц': ['ts', 'tz', 'z', 'c'], 'ч': ['ch', 'tch', 'tsch', 'cz'], 'ш': ['sh', 'sch', 'sz'], 'щ': ['shch', 'sch', 'sc', 'szcz'], 'ю': ['yu', 'ju', 'iu', 'u'], 'я': ['ya', 'ja', 'ia', 'a', 'ie']}


# function to build regex pattern from cyrillic string using extended dict
def build_regex_pattern(query_cyr, ext_dict):
    pattern = ""
    for char in query_cyr:
        if char in ext_dict:
            alternatives = ext_dict[char]
            pattern += "(" + "|".join(re.escape(alt) for alt in alternatives) + ")"
        else:
            pattern += re.escape(char)
    return pattern

# define file paths
final_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/final_places_mach_transl_with_fixed.xlsx"
sapir_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/SAPIR_TABLES/t_SC_Places_Items(ENG_HEB)_Values.xlsx"

# loading excel files
display(Markdown("**loading excel files...**"))
df_final = pd.read_excel(final_path)
df_sapir = pd.read_excel(sapir_path)

# filtering sapir dataframe to include only rows where place equals "<>"
df_sapir = df_sapir[df_sapir["place"] == "<>"]
df_sapir = df_sapir.reset_index(drop=True)

# create normalized columns for matching
# fuzzy matching uses norm_admun_name_eng from final
df_final["final_places_eng_norm"] = df_final["norm_admun_name_eng"].astype(str).str.lower().str.strip()
# regex matching uses norm_admun_name_cyr from final
df_final["final_places_regex"] = df_final["norm_admun_name_cyr"].astype(str).str.lower().str.strip().map(lambda x: build_regex_pattern(str(x), extended_dict))
# create normalized district column from sapir using district
df_sapir["district_norm"] = df_sapir["district"].astype(str).str.lower().str.strip()

# filter final file for district category
df_final_district = df_final[df_final["sapir_category"].astype(str).str.lower() == "district"].copy()

output_rows = []
match_counts = {}
regex_match_count = 0
fuzzy_match_count = 0
unique_regex_matches = set()
unique_fuzzy_matches = set()

# iterate over each row in final district file
for idx, row in df_final_district.iterrows():
    query_regex = row["final_places_regex"]
    query_eng = row["final_places_eng_norm"]

    # get first character from norm_admun_name_cyr for base filtering
    source_first = row["norm_admun_name_cyr"][0].lower() if row["norm_admun_name_cyr"] else ""

    filtered_candidates_base = []
    for match_index, sapir_row in df_sapir.iterrows():
        candidate_first = sapir_row["district_norm"][0] if sapir_row["district_norm"] else ""
        if source_first in base_dict and candidate_first in base_dict[source_first]:
            filtered_candidates_base.append(match_index)

    filtered_candidates_regex = []
    for match_index in filtered_candidates_base:
        candidate_norm = df_sapir.loc[match_index]["district_norm"]
        if re.fullmatch(query_regex, candidate_norm):
            filtered_candidates_regex.append(match_index)

    if filtered_candidates_regex:
        final_candidates = filtered_candidates_regex
        regex_match_count += len(filtered_candidates_regex)
        method = "regex"
    else:
        matches = process.extract(query_eng, df_sapir["district_norm"].tolist(), scorer=fuzz.token_sort_ratio, score_cutoff=similarity_threshold, limit=None)
        final_candidates = [match[2] for match in matches]
        fuzzy_match_count += len(final_candidates)
        method = "fuzzy"

    # filter candidates by keywords if multiple matches exist
    if len(final_candidates) > 1:
        keyword_candidates = [
            idx for idx in final_candidates if any(
                keyword in df_sapir.loc[idx]["TIENG"] for keyword in ["Ukraine", "Ruthenia", "Poland"]
            )
        ]
        if keyword_candidates:
            final_candidates = keyword_candidates

    match_counts[row["norm_admun_name_eng"]] = len(final_candidates)

    final_eng_val = row["norm_admun_name_eng"]
    if final_candidates:
        if method == "regex":
            unique_regex_matches.add(final_eng_val)
            unique_fuzzy_matches.discard(final_eng_val)
        else:
            if final_eng_val not in unique_regex_matches and final_eng_val not in unique_fuzzy_matches:
                unique_fuzzy_matches.add(final_eng_val)

    # if no matches found, append row with empty match fields
    if not final_candidates:
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "ORIG_DISTRICT_STRING_CYR": row["final_places"],
            "NORM_DISTRICT_NAME_CYR": row["norm_admun_name_cyr"],
            "NORM_DISTRICT_TYPE_CYR": row["norm_admun_type_cyr"],
            "NORM_DISTRICT_NAME_ENG": row["norm_admun_name_eng"],
            "NORM_DISTRICT_TYPE_ENG": row["norm_admun_type_eng"],
            "SAPIR_DISTRICT_MATCH": "",
            "SAPIR_DISTRICT_ID": "",
            "SAPIR_DISTRICT_VALUE": "",
            "COUNT_MATCHES_BEF_MAN": 0,
            "CORRECTED": "",
            "0_MATCHES": "",
            "1_MISSMATCH": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "MISSING_GPS": ""
        })

    # for each match found, append row with match details
    for match_index in final_candidates:
        sapir_row = df_sapir.loc[match_index]
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "ORIG_DISTRICT_STRING_CYR": row["final_places"],
            "NORM_DISTRICT_NAME_CYR": row["norm_admun_name_cyr"],
            "NORM_DISTRICT_TYPE_CYR": row["norm_admun_type_cyr"],
            "NORM_DISTRICT_NAME_ENG": row["norm_admun_name_eng"],
            "NORM_DISTRICT_TYPE_ENG": row["norm_admun_type_eng"],
            "SAPIR_DISTRICT_MATCH": sapir_row["district"],
            "SAPIR_DISTRICT_ID": sapir_row["book_id"],
            "SAPIR_DISTRICT_VALUE": sapir_row["TIENG"],
            "COUNT_MATCHES_BEF_MAN": len(final_candidates),
            "CORRECTED": "",
            "0_MATCHES": "",
            "1_MISSMATCH": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "MISSING_GPS": ""
        })

df_output = pd.DataFrame(output_rows)
# insert an empty column at beginning for manual fill (id)
df_output.insert(0, "ID", "")

output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_sapir_regex _&_fuzzy_match_district_with_man.xlsx"
df_output.to_excel(output_file, index=False)

# summary statistics for norm_admun_name_eng values (unique values in final district file)
total_values = df_final_district["norm_admun_name_eng"].nunique()
values_with_matches = sum(1 for count in match_counts.values() if count > 0)
values_with_no_matches = sum(1 for count in match_counts.values() if count == 0)

# build distribution of match counts
match_distribution = {}
for count in match_counts.values():
    match_distribution[count] = match_distribution.get(count, 0) + 1

# prepare summary report string for overall norm_admun_name_eng matching
report_lines = []
report_lines.append(f"total unique norm_district_name_eng values processed: {total_values}")
report_lines.append(f"values with one or more matches: {values_with_matches} ({(values_with_matches/total_values*100):.2f}%)")
report_lines.append(f"values with no matches: {values_with_no_matches} ({(values_with_no_matches/total_values*100):.2f}%)")
for count in sorted(match_distribution.keys()):
    report_lines.append(f"values with exactly {count} match(es): {match_distribution[count]} ({(match_distribution[count]/total_values*100):.2f}%)")
report_text = "

".join(report_lines)
display(Markdown("**summary report:**"))
display(Markdown(report_text))

# unique matching method statistics for norm_admun_name_eng (without duplicates)
unique_total = len(unique_regex_matches.union(unique_fuzzy_matches))
if unique_total > 0:
    unique_regex_percentage = len(unique_regex_matches) / unique_total * 100
    unique_fuzzy_percentage = len(unique_fuzzy_matches) / unique_total * 100
else:
    unique_regex_percentage = 0
    unique_fuzzy_percentage = 0
display(Markdown("**unique match method statistics (by norm_admun_name_eng):**"))
display(Markdown(f"unique regex matches: {len(unique_regex_matches)} ({unique_regex_percentage:.2f}%)"))
display(Markdown(f"unique fuzzy matches: {len(unique_fuzzy_matches)} ({unique_fuzzy_percentage:.2f}%)"))

# display 20 random rows with matches and unique district values
df_matches = df_output[df_output["COUNT_MATCHES_BEF_MAN"] > 0].copy()
# drop duplicates based on orig_district_string_cyr and norm_district_name_eng
df_unique_districts = df_matches.drop_duplicates(subset=["ORIG_DISTRICT_STRING_CYR", "NORM_DISTRICT_NAME_ENG"])
if len(df_unique_districts) >= 20:
    sample_df = df_unique_districts.sample(n=20, random_state=42)
else:
    sample_df = df_unique_districts
display(Markdown("**20 random sample rows with matches (unique district values):**"))
display(sample_df[["ORIG_DISTRICT_STRING_CYR", "NORM_DISTRICT_NAME_CYR", "NORM_DISTRICT_NAME_ENG", "SAPIR_DISTRICT_MATCH", "SAPIR_DISTRICT_ID", "SAPIR_DISTRICT_VALUE", "COUNT_MATCHES_BEF_MAN"]])

display(Markdown(f"**output file saved at:** {output_file}"))
display(Markdown("**fuzzy matching process completed successfully.**"))

**loading excel files...**

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


**summary report:**

total unique norm_district_name_eng values processed: 133

values with one or more matches: 57 (42.86%)

values with no matches: 76 (57.14%)

values with exactly 0 match(es): 76 (57.14%)

values with exactly 1 match(es): 52 (39.10%)

values with exactly 2 match(es): 3 (2.26%)

values with exactly 3 match(es): 1 (0.75%)

values with exactly 4 match(es): 1 (0.75%)

**unique match method statistics (by norm_admun_name_eng):**

...

unique fuzzy matches: 27 (46.55%)

**20 random sample rows with matches (unique district values):**

,ORIG_DISTRICT_STRING_CYR,NORM_DISTRICT_NAME_CYR,NORM_DISTRICT_NAME_ENG,SAPIR_DISTRICT_MATCH,SAPIR_DISTRICT_ID,SAPIR_DISTRICT_VALUE,COUNT_MATCHES_BEF_MAN
83,Каховський район,Каховка,Kakhovka,Kakhovka,7562091,"<>,Kakhovka,Nikolayev,Ukraine (USSR)",1
47,Гороховское староство,Горохов,Horokhov,Horochow,7529609,"<>,Horochow,Wolyn,Poland",1
119,Луцкий уезд,Луцк,Lutsk,Łuck,5703895,"<>,Łuck,Wolyn,Poland",1
158,Потиєвський район,Потіївка,Potiivka,Potiyevka,6392544,"<>,Potiyevka,Zhitomir,Ukraine (USSR)",1
0,Берегівський район,Берегово,Beregovo,Berehovo,5703898,"<>,Berehovo,Carpathian Ruthenia,Czechoslovakia",1
57,Заліщицький район,Заліщики,Zalishchyky,Zaleszczyki,7532562,"<>,Zaleszczyki,Tarnopol,Poland",1
76,Калінінський район,Калінін,Kalinin,Kalinin,8405636,"<>,Kalinin,Tashauz,Turkmenistan (USSR)",2
141,Новотроїцький район,Новотроїцьке,Novotroitske,Novo Troitskoye,7583849,"<>,Novo Troitskoye,Zaporozhye,Ukraine (USSR)",1
15,Бучацький повіт,Бучач,Buchach,Buczacz,7526963,"<>,Buczacz,Tarnopol,Poland",1
95,Козлівський район,Козлів,Kozlov,Kozlovo,9126598,"<>,Kozlovo,Kalinin,Russia (USSR)",4


**output file saved at:** /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_sapir_regex _&_fuzzy_match_district_with_man.xlsx

**fuzzy matching process completed successfully.**

In [ ]:
#@title UNPIVOTED DISTRICTS WITH SAPIR

# define the directory where the files are located
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# define filenames for manual district data and mapping data
manual_file = "places_sapir_regex _&_fuzzy_match_district_with_man.xlsx"
mapping_file = "t_Master_table_draft_dana_withdates_mapped.xlsx"

# read the excel files
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(file_dir, mapping_file))

# convert id columns to string types in both dataframes
# for mapping file, convert numeric values to int then to string to remove decimals
df_manual["source_ids"] = df_manual["source_ids"].astype(str)
df_mapping["old_id"] = df_mapping["old_id"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")
df_mapping["ID_master"] = df_mapping["ID_master"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")

# filter out rows where STATUS equals 'to remove'
df_manual = df_manual[df_manual["STATUS"] != "to remove"].copy()

# create new columns for target district output based on manual file
# district_cyr from ORIG_DISTRICT_STRING_CYR
df_manual["district_cyr"] = df_manual["ORIG_DISTRICT_STRING_CYR"]
# norm_district_name_cyr from NORM_DISTRICT_NAME_CYR
df_manual["norm_district_name_cyr"] = df_manual["NORM_DISTRICT_NAME_CYR"]
# norm_district_type_cyr from NORM_DISTRICT_TYPE_CYR
df_manual["norm_district_type_cyr"] = df_manual["NORM_DISTRICT_TYPE_CYR"]
# district_sapir from SAPIR_DISTRICT_MATCH
df_manual["district_sapir"] = df_manual["SAPIR_DISTRICT_MATCH"]
# district_missing as concatenation of NORM_DISTRICT_NAME_ENG and NORM_DISTRICT_TYPE_ENG with a space,
# but only for rows where STATUS equals "to export"; otherwise, set as empty string
df_manual["district_missing"] = df_manual.apply(lambda row: str(row["NORM_DISTRICT_NAME_ENG"]) + " " + str(row["NORM_DISTRICT_TYPE_ENG"]) if row["STATUS"]=="to export" else "", axis=1)
# parachronistic_district_sapir: from SAPIR_DISTRICT_VALUE if SAPIR_HISTORICALLY_INCOMPATIBLE == "TRUE", else ""
df_manual["parachronistic_district_sapir"] = df_manual.apply(lambda row: row["SAPIR_DISTRICT_VALUE"] if str(row["SAPIR_HISTORICALLY_INCOMPATIBLE"]).strip().upper() == "TRUE" else "", axis=1)
# parachronistic_id_sapir: from SAPIR_DISTRICT_ID if SAPIR_HISTORICALLY_INCOMPATIBLE == "TRUE", else ""
df_manual["parachronistic_id_sapir"] = df_manual.apply(lambda row: row["SAPIR_DISTRICT_ID"] if str(row["SAPIR_HISTORICALLY_INCOMPATIBLE"]).strip().upper() == "TRUE" else "", axis=1)

# split source_ids by comma into a list
df_manual["id_old_list"] = df_manual["source_ids"].apply(lambda x: [item.strip() for item in x.split(",") if item.strip() != ""])

# explode the source_ids into separate rows and rename the column to id_old
df_exploded = df_manual.explode("id_old_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"id_old_list": "id_old"})
df_exploded["id_old"] = df_exploded["id_old"].astype(str)

# function to convert id values: first to float, then to int, then to string
def conv_id(x):
    try:
        return str(int(float(x)))
    except:
        return x

# convert id_old to integer then to string to avoid decimals/extra zeros
df_exploded["id_old"] = df_exploded["id_old"].apply(conv_id)

# merge to bring in id_master from mapping file based on id_old matching old_id
df_merged = pd.merge(df_exploded, df_mapping[["old_id", "ID_master"]], left_on="id_old", right_on="old_id", how="left")
df_merged["ID_master"] = df_merged["ID_master"].fillna("missing")
df_merged = df_merged.rename(columns={"ID_master": "id_master"})
if "old_id_y" in df_merged.columns:
    df_merged = df_merged.drop(columns=["old_id_y"])

# convert id_master to integer then to string
df_merged["id_master"] = df_merged["id_master"].apply(conv_id)

# prepare final unpivoted output dataframe with required columns
# insert sapir_id from SAPIR_DISTRICT_ID (from manual file) – convert it via conv_id
cols_output = ["source", "district_cyr", "norm_district_name_cyr", "norm_district_type_cyr", "district_sapir", "district_missing", "parachronistic_district_sapir", "parachronistic_id_sapir", "id_old", "id_master", "SAPIR_DISTRICT_ID", "SAPIR_DISTRICT_VALUE", "MISS_CENTER_GPS", "MISS_REF1", "MISS_REF2", "MISS_REF3"]
df_output = df_merged[cols_output].copy()
# rename SAPIR_DISTRICT_ID to sapir_id and SAPIR_DISTRICT_VALUE to full_sapir
df_output = df_output.rename(columns={"SAPIR_DISTRICT_ID": "sapir_id", "SAPIR_DISTRICT_VALUE": "full_sapir"})

# convert sapir_id to integer then to string
df_output["sapir_id"] = df_output["sapir_id"].apply(conv_id)

# bring STATUS from df_merged to df_output temporarily for this purpose
df_output["STATUS"] = df_merged["STATUS"]
df_output.loc[df_output["STATUS"]=="to export", "district_sapir"] = ""
df_output.loc[df_output["STATUS"]=="to export", "sapir_id"] = ""
df_output = df_output.drop(columns=["STATUS"])

# prepare the missing in sapir district output file
# include columns from manual file plus parachronistic fields
cols_missing = ["source_ids", "source", "ORIG_DISTRICT_STRING_CYR", "NORM_DISTRICT_NAME_CYR", "NORM_DISTRICT_TYPE_CYR", "NORM_DISTRICT_NAME_ENG", "NORM_DISTRICT_TYPE_ENG", "MISS_CENTER_GPS", "MISS_REF1", "MISS_REF2", "MISS_REF3", "parachronistic_district_sapir", "parachronistic_id_sapir"]
df_missing = df_manual[df_manual["STATUS"] == "to export"][cols_missing].drop_duplicates()

# computing statistics (same as before)
total_manual = df_manual.shape[0]
total_exploded = df_output.shape[0]
perc_explosion = (total_exploded / total_manual) * 100 if total_manual > 0 else 0
source_counts = df_output["source"].value_counts()
source_perc = (source_counts / source_counts.sum()) * 100
unique_place_by_source = df_output.groupby("source")["district_cyr"].nunique()
total_unique_place = df_output["district_cyr"].nunique()
unique_place_perc = (unique_place_by_source / total_unique_place) * 100 if total_unique_place > 0 else 0
status_counts = df_manual["STATUS"].value_counts()
missing_id_master_count = (df_output["id_master"] == "missing").sum()
perc_missing_id_master = (missing_id_master_count / total_exploded) * 100 if total_exploded > 0 else 0
total_missing_rows = df_missing.shape[0]
missing_gps_dashes = df_missing["MISS_CENTER_GPS"].astype(str).str.contains("---").sum()
perc_missing_gps_dashes = (missing_gps_dashes / total_missing_rows) * 100 if total_missing_rows > 0 else 0

# printing summary report in a condensed format
print("
--- summary report ---
")
print("total rows in manual file (status != 'to remove'): {} rows".format(total_manual))
print("total rows in output file after explosion: {} rows ({:.2f}%)".format(total_exploded, perc_explosion))
print("source breakdown in output file: " + ", ".join(["{}: {} rows ({:.2f}%)".format(src, cnt, source_perc[src]) for src, cnt in source_counts.items()]))
print("unique district_cyr count by source: " + ", ".join(["{}: {} unique values ({:.2f}%)".format(src, cnt, unique_place_perc[src]) for src, cnt in unique_place_by_source.items()]))
print("manual file status counts: " + ", ".join(["{}: {} rows ({:.2f}%)".format(status, cnt, (cnt/total_manual*100)) for status, cnt in status_counts.items()]))
print("count of 'missing' in id_master in output file: {} rows ({:.2f}%)".format(missing_id_master_count, perc_missing_id_master))
print("missing sapir district file statistics: total rows: {} rows, rows where MISS_CENTER_GPS contains '---': {} rows ({:.2f}%)".format(total_missing_rows, missing_gps_dashes, perc_missing_gps_dashes))

# saving output files
output_file = os.path.join(file_dir, "places_unpivoted_with_sapir_district.xlsx")
missing_file = os.path.join(file_dir, "places_missing_in_sapir_district.xlsx")
df_output.to_excel(output_file, index=False)
df_missing.to_excel(missing_file, index=False)
print("
output files saved: unpivoted file -> {}, missing district file -> {}".format(output_file, missing_file))

# displaying random samples
sample_output = df_output.sample(n=20, random_state=42) if total_exploded >= 20 else df_output.copy()
sample_missing = df_missing.sample(n=20, random_state=42) if total_missing_rows >= 20 else df_missing.copy()
print("
--- sample of 20 random rows from unpivoted district file ---")
display(sample_output)
print("
--- sample of 20 random rows from missing district file ---")
display(sample_missing)


--- summary report ---

total rows in manual file (status != 'to remove'): 177 rows
total rows in output file after explosion: 5623 rows (3176.84%)
source breakdown in output file: title: 2866 rows (50.97%), fond: 2757 rows (49.03%)
unique district_cyr count by source: fond: 23 unique values (14.11%), title: 154 unique values (94.48%)
manual file status counts: to export: 98 rows (55.37%), to use: 79 rows (44.63%)
count of 'missing' in id_master in output file: 1850 rows (32.90%)
missing sapir district file statistics: total rows: 98 rows, rows where MISS_CENTER_GPS contains '---': 1 rows (1.02%)

output files saved: unpivoted file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_unpivoted_with_sapir_district.xlsx, missing district file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_missing_in_sapir_district.xlsx

--- sample of 20 random rows from unpivoted district file ---


,source,district_cyr,norm_district_name_cyr,norm_district_type_cyr,district_sapir,district_missing,parachronistic_district_sapir,parachronistic_id_sapir,id_old,id_master,sapir_id,full_sapir,MISS_CENTER_GPS,MISS_REF1,MISS_REF2,MISS_REF3
3494,title,Коропецький район,Коропець,район,,Koropets raion,,,6763,missing,,"<>,Toropets,Kalinin,Russia (USSR)","48.93471024802256, 25.177556566435275",https://uk.wikisource.org/wiki/%D0%9A%D0%BE%D1...,NaN,NaN
1609,title,Тернопільський повіт,Тернопіль,повіт,Tarnopol,,,,3883,missing,5719882,"<>,Tarnopol,Tarnopol,Poland",NaN,NaN,NaN,NaN
1097,title,Снігурівський район,Снігурівка,район,,Snihurivka raion,,,16607,12217,,NaN,"47.075960379393685, 32.80554077675846",https://uk.wikipedia.org/wiki/%D0%A1%D0%BD%D1%...,NaN,NaN
2302,fond,Генічеський район,Генічеськ,район,Genichesk,,,,12190,8214,7583604,"<>,Genichesk,Zaporozhye,Ukraine (USSR)",NaN,NaN,NaN,NaN
4500,title,Цюрупинський район,Цюрупинськ,район,Tsyurupinsk,,,,14871,10815,7562805,"<>,Tsyurupinsk,Nikolayev,Ukraine (USSR)",NaN,NaN,NaN,NaN
2271,title,Бучацький район,Бучач,район,,Buchach raion,"<>,Buczacz,Tarnopol,Poland",7526963.0,6658,missing,,"<>,Buczacz,Tarnopol,Poland","49.06432567099221, 25.383443383940463",https://uk.wikipedia.org/wiki/%D0%91%D1%83%D1%...,NaN,NaN
2115,title,Борщівський район,Борщів,район,,Borshchiv raion,"<>,Borszczow,Tarnopol,Poland",6822483.0,6727,missing,,"<>,Borszczow,Tarnopol,Poland","48.802172382438805, 26.04922843828339",https://uk.wikipedia.org/wiki/%D0%91%D0%BE%D1%...,NaN,NaN
2600,title,Золочівський повіт,Золочів,повіт,Zolochev,,,,5269,missing,7557962,"<>,Zolochev,Kharkov,Ukraine (USSR)",NaN,NaN,NaN,NaN
1614,title,Тернопільський повіт,Тернопіль,повіт,Tarnopol,,,,3888,missing,5719882,"<>,Tarnopol,Tarnopol,Poland",NaN,NaN,NaN,NaN
3475,title,Копичинецький район,Копичинці,район,,Kopychyntsi raion,"<>,Kopyczynce,Tarnopol,Poland",9445298.0,6617,missing,,"<>,Kopyczynce,Tarnopol,Poland","49.10797919956194, 25.910343212577637",https://ternopedia.te.ua/index.php/%D0%9A%D0%B...,NaN,NaN



--- sample of 20 random rows from missing district file ---


,source_ids,source,ORIG_DISTRICT_STRING_CYR,NORM_DISTRICT_NAME_CYR,NORM_DISTRICT_TYPE_CYR,NORM_DISTRICT_NAME_ENG,NORM_DISTRICT_TYPE_ENG,MISS_CENTER_GPS,MISS_REF1,MISS_REF2,MISS_REF3,parachronistic_district_sapir,parachronistic_id_sapir
90,"6500, 6515, 6535, 6539, 6665, 6687, 6693, 6727...",title,Чортківський район,Чортків,район,Chortkiv,raion,"49.01376773097081, 25.800348180414147",https://uk.wikipedia.org/wiki/%D0%A7%D0%BE%D1%...,NaN,NaN,,
55,63,title,Полеске воєводство,Полісся,воєводство,Polesie,voivodeship,"52.09113655719164, 23.673559729590977",https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%...,NaN,NaN,,
164,"7537, 7548",title,Ужгородский район,Ужгород,район,Uzhgorod,raion,"48.62212738617726, 22.29120728462974",https://uk.wikipedia.org/wiki/%D0%A3%D0%B6%D0%...,NaN,NaN,"<>,Uzhorod,Carpathian Ruthenia,Czechoslovakia",5704074.0
23,6531,title,Заложцівський район,Заложці,район,Zalozhtsi,raion,"49.79544502073252, 25.37907878087605",https://ternopedia.te.ua/index.php/%D0%97%D0%B...,NaN,NaN,,
136,"2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017...",fond,Луцкий округ,Луцк,округ,Lutsk,okrug,"50.7518525751484, 25.33883144604442",---,NaN,NaN,"<>,Łuck,Wolyn,Poland",5703895.0
139,2007,title,Луцкий район,Луцк,район,Lutsk,raion,"50.7508442979578, 25.333643253831163",https://uk.wikipedia.org/wiki/%D0%9B%D1%83%D1%...,NaN,NaN,"<>,Łuck,Wolyn,Poland",5703895.0
92,"6211, 6212, 6239, 6301, 6318, 6319, 6362, 6363...",title,Шумський район,Шумськ,район,Shumsk,raion,"50.120791332097035, 26.115560130430108",https://uk.wikipedia.org/wiki/%D0%A8%D1%83%D0%...,NaN,NaN,,
57,16563,title,Посад-Покровська волость,Посад-Покровське,волость,Posad-Pokrovske,volost,"46.80524495953117, 32.26283090142105",---,NaN,NaN,,
13,"7758, 7831, 7834, 7835, 7838, 7839",title,Виноградівський район,Виноградів,район,Vynohradiv,raion,"48.151297657526264, 23.028160013443962",https://uk.wikipedia.org/wiki/%D0%92%D0%B8%D0%...,NaN,NaN,,
1,"6511, 6513, 6515, 6517, 6528, 6535, 6546, 6644...",title,Білобожницький район,Білобожниця,район,Belobozhnytsia,raion,"49.0494600721311, 25.67464719921591",https://uk.wikipedia.org/wiki/Білобожницький_р...,NaN,NaN,,


In [ ]:
#@title SUB-DISTRICT MATCHING WITH SAPIR TABLE - REGEX & FUZZY MATCH

# setting similarity threshold for fuzzy matching
similarity_threshold = 85

# defining mapping dictionaries for transliteration
base_dict = {'а': ['a'], 'б': ['b'], 'в': ['v', 'w'], 'г': ['h', 'g'], 'ґ': ['g'], 'д': ['d', 'j', 'dz'], 'е': ['e', 'ye', 'ie'], 'є': ['ye', 'je', 'ie', 'e'], 'ж': ['zh', 'j', 'rz', 'ż'], 'з': ['z'], 'и': ['y', 'i'], 'і': ['i'], 'ї': ['yi', 'ji', 'ii', 'i', 'e', 'ye', 'ie', 'je'], 'й': ['y', 'j', 'i'], 'к': ['k', 'c', 'q'], 'л': ['l', 'ł'], 'м': ['m'], 'н': ['n'], 'о': ['o', 'je'], 'п': ['p'], 'р': ['r', 'rz'], 'с': ['s'], 'т': ['t'], 'у': ['u', 'ou'], 'ф': ['f', 'ph'], 'х': ['kh', 'ch', 'h'], 'ц': ['ts', 'tz', 'z', 'c'], 'ч': ['ch', 'tch', 'tsch', 'cz'], 'ш': ['sh', 'sch', 'sz'], 'щ': ['shch', 'sch', 'sc', 'szcz'], 'ю': ['yu', 'ju', 'iu'], 'я': ['ya', 'ja', 'ia', 'je']}

extended_dict = {'а': ['a'], 'б': ['b'], 'в': ['v', 'w', 'l'], 'г': ['h', 'g'], 'ґ': ['g'], 'д': ['d', 'j', 'dz'], 'е': ['e', 'ye', 'ie'], 'є': ['ye', 'je', 'ie', 'e'], 'ж': ['zh', 'j', 'rz', 'ż'], 'з': ['z'], 'и': ['y', 'i', 'e'], 'і': ['i', 'o', 'y', 'e'], 'ї': ['yi', 'ji', 'ii', 'i', 'e', 'ye', 'ie', 'je'], 'й': ['y', 'j', 'i'], 'к': ['k', 'c', 'q'], 'л': ['l', 'ł'], 'м': ['m'], 'н': ['n'], 'о': ['o'], 'п': ['p'], 'р': ['r', 'rz'], 'с': ['s'], 'т': ['t'], 'у': ['u', 'ou', 'ó'], 'ф': ['f', 'ph'], 'х': ['kh', 'ch', 'h'], 'ц': ['ts', 'tz', 'z', 'c'], 'ч': ['ch', 'tch', 'tsch', 'cz'], 'ш': ['sh', 'sch', 'sz'], 'щ': ['shch', 'sch', 'sc', 'szcz'], 'ю': ['yu', 'ju', 'iu', 'u'], 'я': ['ya', 'ja', 'ia', 'a', 'ie']}

# defining a function to build regex pattern from cyrillic string using extended dict
def build_regex_pattern(query_cyr, ext_dict):
    pattern = ""
    for char in query_cyr:
        if char in ext_dict:
            alternatives = ext_dict[char]
            pattern += "(" + "|".join(re.escape(alt) for alt in alternatives) + ")"
        else:
            pattern += re.escape(char)
    return pattern

# defining file paths
final_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/final_places_mach_transl_with_fixed.xlsx"
sapir_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/SAPIR_TABLES/t_SC_Places_Items(ENG_HEB)_Values.xlsx"

# loading excel files
display(Markdown("**loading excel files...**"))
df_final = pd.read_excel(final_path)
df_sapir = pd.read_excel(sapir_path)

# note: removed filtering on sapir 'place' column since we search in all rows

# creating normalized columns for matching in final file
df_final["final_places_eng_norm"] = df_final["norm_admun_name_eng"].astype(str).str.lower().str.strip()
df_final["final_places_regex"] = df_final["norm_admun_name_cyr"].astype(str).str.lower().str.strip().map(lambda x: build_regex_pattern(str(x), extended_dict))

# creating normalized sub-district column from sapir using sub-district
df_sapir["sub-district_norm"] = df_sapir["sub-district"].astype(str).str.lower().str.strip()

# filtering final file for sub-district category instead of district
df_final_sub_district = df_final[df_final["sapir_category"].astype(str).str.lower() == "sub-district"].copy()

output_rows = []
match_counts = {}
regex_match_count = 0
fuzzy_match_count = 0
unique_regex_matches = set()
unique_fuzzy_matches = set()

# iterating over each row in final sub-district file
for idx, row in df_final_sub_district.iterrows():
    query_regex = row["final_places_regex"]
    query_eng = row["final_places_eng_norm"]

    # getting first character from norm_admun_name_cyr for base filtering
    source_first = row["norm_admun_name_cyr"][0].lower() if row["norm_admun_name_cyr"] else ""

    filtered_candidates_base = []
    # base filtering using first character matching with sapir sub-district values
    for match_index, sapir_row in df_sapir.iterrows():
        candidate_first = sapir_row["sub-district_norm"][0] if sapir_row["sub-district_norm"] else ""
        if source_first in base_dict and candidate_first in base_dict[source_first]:
            filtered_candidates_base.append(match_index)

    filtered_candidates_regex = []
    # filtering candidates with full regex match on sapir sub-district values
    for match_index in filtered_candidates_base:
        candidate_norm = df_sapir.loc[match_index]["sub-district_norm"]
        if re.fullmatch(query_regex, candidate_norm):
            filtered_candidates_regex.append(match_index)

    if filtered_candidates_regex:
        final_candidates = filtered_candidates_regex
        regex_match_count += len(filtered_candidates_regex)
        method = "regex"
    else:
        # performing fuzzy matching if regex matching fails
        matches = process.extract(query_eng, df_sapir["sub-district_norm"].tolist(), scorer=fuzz.token_sort_ratio, score_cutoff=similarity_threshold, limit=None)
        final_candidates = [match[2] for match in matches]
        fuzzy_match_count += len(final_candidates)
        method = "fuzzy"

    # filtering candidates by keywords if multiple matches exist
    if len(final_candidates) > 1:
        keyword_candidates = [
            idx for idx in final_candidates if any(
                keyword in df_sapir.loc[idx]["TIENG"] for keyword in ["Ukraine", "Ruthenia", "Poland"]
            )
        ]
        if keyword_candidates:
            final_candidates = keyword_candidates

    match_counts[row["norm_admun_name_eng"]] = len(final_candidates)

    final_eng_val = row["norm_admun_name_eng"]
    if final_candidates:
        if method == "regex":
            unique_regex_matches.add(final_eng_val)
            unique_fuzzy_matches.discard(final_eng_val)
        else:
            if final_eng_val not in unique_regex_matches and final_eng_val not in unique_fuzzy_matches:
                unique_fuzzy_matches.add(final_eng_val)

    # if no matches found, append row with empty match fields
    if not final_candidates:
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "ORIG_SUB-DISTRICT_STRING_CYR": row["final_places"],
            "NORM_SUB-DISTRICT_NAME_CYR": row["norm_admun_name_cyr"],
            "NORM_SUB-DISTRICT_TYPE_CYR": row["norm_admun_type_cyr"],
            "NORM_SUB-DISTRICT_NAME_ENG": row["norm_admun_name_eng"],
            "NORM_SUB-DISTRICT_TYPE_ENG": row["norm_admun_type_eng"],
            "SAPIR_SUB-DISTRICT_MATCH": "",
            "SAPIR_SUB-DISTRICT_ID": "",
            "SAPIR_SUB-DISTRICT_VALUE": "",
            "COUNT_MATCHES_BEF_MAN": 0,
            "CORRECTED": "",
            "0_MATCHES": "",
            "1_MISSMATCH": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "MISSING_GPS": ""
        })

    # for each match found, append row with match details
    for match_index in final_candidates:
        sapir_row = df_sapir.loc[match_index]
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "ORIG_SUB-DISTRICT_STRING_CYR": row["final_places"],
            "NORM_SUB-DISTRICT_NAME_CYR": row["norm_admun_name_cyr"],
            "NORM_SUB-DISTRICT_TYPE_CYR": row["norm_admun_type_cyr"],
            "NORM_SUB-DISTRICT_NAME_ENG": row["norm_admun_name_eng"],
            "NORM_SUB-DISTRICT_TYPE_ENG": row["norm_admun_type_eng"],
            "SAPIR_SUB-DISTRICT_MATCH": sapir_row["sub-district"],
            "SAPIR_SUB-DISTRICT_ID": sapir_row["book_id"],
            "SAPIR_SUB-DISTRICT_VALUE": sapir_row["TIENG"],
            "COUNT_MATCHES_BEF_MAN": len(final_candidates),
            "CORRECTED": "",
            "0_MATCHES": "",
            "1_MISSMATCH": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "MISSING_GPS": ""
        })

df_output = pd.DataFrame(output_rows)
# inserting an empty column at beginning for manual fill (id)
df_output.insert(0, "ID", "")

# defining output file path with sub-district in filename
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_sapir_regex _&_fuzzy_match_sub-district_with_man.xlsx"
df_output.to_excel(output_file, index=False)

# calculating summary statistics for norm_admun_name_eng values (unique values in final sub-district file)
total_values = df_final_sub_district["norm_admun_name_eng"].nunique()
values_with_matches = sum(1 for count in match_counts.values() if count > 0)
values_with_no_matches = sum(1 for count in match_counts.values() if count == 0)

# building distribution of match counts
match_distribution = {}
for count in match_counts.values():
    match_distribution[count] = match_distribution.get(count, 0) + 1

# preparing summary report string for overall norm_sub-district_name_eng matching
report_lines = []
report_lines.append(f"total unique norm_sub-district_name_eng values processed: {total_values}")
report_lines.append(f"values with one or more matches: {values_with_matches} ({(values_with_matches/total_values*100):.2f}%)")
report_lines.append(f"values with no matches: {values_with_no_matches} ({(values_with_no_matches/total_values*100):.2f}%)")
for count in sorted(match_distribution.keys()):
    report_lines.append(f"values with exactly {count} match(es): {match_distribution[count]} ({(match_distribution[count]/total_values*100):.2f}%)")
report_text = "

".join(report_lines)
display(Markdown("**summary report:**"))
display(Markdown(report_text))

# calculating unique matching method statistics for norm_sub-district_name_eng (without duplicates)
unique_total = len(unique_regex_matches.union(unique_fuzzy_matches))
if unique_total > 0:
    unique_regex_percentage = len(unique_regex_matches) / unique_total * 100
    unique_fuzzy_percentage = len(unique_fuzzy_matches) / unique_total * 100
else:
    unique_regex_percentage = 0
    unique_fuzzy_percentage = 0
display(Markdown("**unique match method statistics (by norm_sub-district_name_eng):**"))
display(Markdown(f"unique regex matches: {len(unique_regex_matches)} ({unique_regex_percentage:.2f}%)"))
display(Markdown(f"unique fuzzy matches: {len(unique_fuzzy_matches)} ({unique_fuzzy_percentage:.2f}%)"))

# displaying 20 random sample rows with matches and unique sub-district values
df_matches = df_output[df_output["COUNT_MATCHES_BEF_MAN"] > 0].copy()
# dropping duplicates based on orig_sub-district_string_cyr and norm_sub-district_name_eng
df_unique_sub_districts = df_matches.drop_duplicates(subset=["ORIG_SUB-DISTRICT_STRING_CYR", "NORM_SUB-DISTRICT_NAME_ENG"])
if len(df_unique_sub_districts) >= 20:
    sample_df = df_unique_sub_districts.sample(n=20, random_state=42)
else:
    sample_df = df_unique_sub_districts
display(Markdown("**20 random sample rows with matches (unique sub-district values):**"))
display(sample_df[["ORIG_SUB-DISTRICT_STRING_CYR", "NORM_SUB-DISTRICT_NAME_CYR", "NORM_SUB-DISTRICT_NAME_ENG", "SAPIR_SUB-DISTRICT_MATCH", "SAPIR_SUB-DISTRICT_ID", "SAPIR_SUB-DISTRICT_VALUE", "COUNT_MATCHES_BEF_MAN"]])

display(Markdown(f"**output file saved at:** {output_file}"))
display(Markdown("**fuzzy matching process completed successfully.**"))

In [ ]:
#@title UNPIVOTED SUB-DISTRICTS WITH SAPIR

# define the directory where the files are located
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# define filenames for manual sub-district data and mapping data
manual_file = "places_sapir_regex _&_fuzzy_match_sub-district_with_man.xlsx"
mapping_file = "t_Master_table_draft_dana_withdates_mapped.xlsx"

# read the excel files
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(file_dir, mapping_file))

# convert id columns to string types in both dataframes
# for mapping file,  convert numeric values to int then to string to remove decimals
df_manual["source_ids"] = df_manual["source_ids"].astype(str)
df_mapping["old_id"] = df_mapping["old_id"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")
df_mapping["ID_master"] = df_mapping["ID_master"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")

# filter out rows where STATUS equals 'to remove'
df_manual = df_manual[df_manual["STATUS"] != "to remove"].copy()

df_manual = df_manual.drop_duplicates(subset=["source", "ORIG_SUB-DISTRICT_STRING_CYR"]).copy()

# Create new columns for target sub-district output based on manual file
df_manual["sub-district_cyr"] = df_manual["ORIG_SUB-DISTRICT_STRING_CYR"]
df_manual["norm_sub-district_name_cyr"] = df_manual["NORM_SUB-DISTRICT_NAME_CYR"]
df_manual["norm_sub-district_type_cyr"] = df_manual["NORM_SUB-DISTRICT_TYPE_CYR"]
df_manual["sapir_sub-district"] = df_manual.apply(lambda row: "" if ((pd.isna(row["RELATED_SAPIR_IDS"]) or row["RELATED_SAPIR_IDS"]=="") and (pd.isna(row["SAPIR_subDistrictCode"]) or row["SAPIR_subDistrictCode"]=="")) else row["SAPIR_SUB-DISTRICT_MATCH"], axis=1)
# sub-district_missing: if STATUS equals "to export" then concatenate NORM_SUB-DISTRICT_NAME_ENG and NORM_SUB-DISTRICT_TYPE_ENG with a space; otherwise ""
df_manual["sub-district_missing"] = df_manual.apply(
    lambda row: (str(row["NORM_SUB-DISTRICT_NAME_ENG"]) + " " + str(row["NORM_SUB-DISTRICT_TYPE_ENG"]))
    if row["STATUS"]=="to export" else "", axis=1)
df_manual["sapir_related_ids"] = df_manual["RELATED_SAPIR_IDS"]
df_manual["sapir_subDistrictCode"] = df_manual["SAPIR_subDistrictCode"]

# For df_missing: create additional columns as specified
df_manual["NORM_SUB-DISTRICT_NAME_ENG_ABSENT"] = df_manual.apply(
    lambda row: row["NORM_SUB-DISTRICT_NAME_ENG"] if row["STATUS"]=="to export" else "", axis=1)
df_manual["NORM_SUB-DISTRICT_TYPE_ENG_ABSENT"] = df_manual.apply(
    lambda row: row["NORM_SUB-DISTRICT_TYPE_ENG"] if row["STATUS"]=="to export" else "", axis=1)
df_manual["SAPIR_SUB-DISTRICT_LACKING_DEDICATED_RECORD"] = df_manual.apply(
    lambda row: row["SAPIR_SUB-DISTRICT_MATCH"] if row["STATUS"]=="to use" else "", axis=1)
df_manual["SAPIR_RELATED_IDS_FOR_LACKING_DEDICATED_RECORD"] = df_manual["RELATED_SAPIR_IDS"]
df_manual["SAPIR_subDistrictCode_FOR_LACKING_DEDICATED_RECORD"] = df_manual["SAPIR_subDistrictCode"]

# split source_ids by comma into a list
df_manual["id_old_list"] = df_manual["source_ids"].apply(lambda x: [item.strip() for item in x.split(",") if item.strip() != ""])

# explode the source_ids into separate rows and rename the column to id_old
df_exploded = df_manual.explode("id_old_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"id_old_list": "id_old"})
df_exploded["id_old"] = df_exploded["id_old"].astype(str)

# function to convert id values: first to float, then to int, then to string
def conv_id(x):
    try:
        return str(int(float(x)))
    except:
        return x

# convert id_old to integer then to string to avoid decimals/extra zeros
df_exploded["id_old"] = df_exploded["id_old"].apply(conv_id)

# merge to bring in id_master from mapping file based on id_old matching old_id
df_merged = pd.merge(df_exploded, df_mapping[["old_id", "ID_master"]], left_on="id_old", right_on="old_id", how="left")
df_merged["ID_master"] = df_merged["ID_master"].fillna("missing")
df_merged = df_merged.rename(columns={"ID_master": "id_master"})
if "old_id_y" in df_merged.columns:
    df_merged = df_merged.drop(columns=["old_id_y"])

# convert id_master to integer then to string
df_merged["id_master"] = df_merged["id_master"].apply(conv_id)

# prepare final unpivoted output dataframe with required columns
# For sub-district output, the columns (in order) should be:
# source, sub-district_cyr, norm_sub-district_name_cyr, norm_sub-district_type_cyr, sapir_sub-district,
# sub-district_missing, id_old, id_master, sapir_related_ids, sapir_subDistrictCode, sapir_id, full_sapir,
# MISS_CENTER_GPS, MISS_REF1, MISS_REF2
cols_output = ["source", "sub-district_cyr", "norm_sub-district_name_cyr", "norm_sub-district_type_cyr",
               "sapir_sub-district", "sub-district_missing", "id_old", "id_master", "sapir_related_ids",
               "sapir_subDistrictCode", "SAPIR_ID_MATCH", "SAPIR_SUB-DISTRICT_VALUE",
               "MISS_CENTER_GPS", "MISS_REF1", "MISS_REF2"]
df_output = df_merged[cols_output].copy()
# rename SAPIR_ID_MATCH to sapir_id and SAPIR_SUB-DISTRICT_VALUE to full_sapir
df_output = df_output.rename(columns={"SAPIR_ID_MATCH": "sapir_id", "SAPIR_SUB-DISTRICT_VALUE": "full_sapir"})

# convert sapir_id to integer then to string if possible, but then set it to empty string as required
df_output["sapir_id"] = ""  # fixed as empty
df_output["full_sapir"] = ""  # fixed as empty


# prepare the missing in sapir sub-district output file
# The missing table should include only unique rows for unique combination of source + ORIG_SUB-DISTRICT_STRING_CYR
cols_missing = ["source_ids", "source", "ORIG_SUB-DISTRICT_STRING_CYR", "NORM_SUB-DISTRICT_NAME_CYR",
                "NORM_SUB-DISTRICT_TYPE_CYR", "NORM_SUB-DISTRICT_NAME_ENG_ABSENT", "NORM_SUB-DISTRICT_TYPE_ENG_ABSENT",
                "SAPIR_SUB-DISTRICT_LACKING_DEDICATED_RECORD", "MISS_CENTER_GPS", "MISS_REF1", "MISS_REF2",
                "SAPIR_RELATED_IDS_FOR_LACKING_DEDICATED_RECORD", "SAPIR_subDistrictCode_FOR_LACKING_DEDICATED_RECORD"]
df_missing = df_manual[df_manual["STATUS"] != "to remove"].copy()
df_missing["NORM_SUB-DISTRICT_NAME_ENG_ABSENT"] = df_missing.apply(lambda row: row["NORM_SUB-DISTRICT_NAME_ENG"] if row["STATUS"]=="to export" else "", axis=1)
df_missing["NORM_SUB-DISTRICT_TYPE_ENG_ABSENT"] = df_missing.apply(lambda row: row["NORM_SUB-DISTRICT_TYPE_ENG"] if row["STATUS"]=="to export" else "", axis=1)
df_missing["SAPIR_SUB-DISTRICT_LACKING_DEDICATED_RECORD"] = df_missing.apply(lambda row: row["SAPIR_SUB-DISTRICT_MATCH"] if row["STATUS"]=="to use" else "", axis=1)
# Select only the specified columns in the given order and drop duplicates based on unique combination of source and ORIG_SUB-DISTRICT_STRING_CYR
df_missing = df_missing[cols_missing].drop_duplicates(subset=["source", "ORIG_SUB-DISTRICT_STRING_CYR"])

# computing statistics (adapted for sub-district)
total_manual = df_manual.shape[0]
total_exploded = df_output.shape[0]
perc_explosion = (total_exploded / total_manual) * 100 if total_manual > 0 else 0
source_counts = df_output["source"].value_counts()
source_perc = (source_counts / source_counts.sum()) * 100
unique_subdistrict_by_source = df_output.groupby("source")["sub-district_cyr"].nunique()
total_unique_subdistrict = df_output["sub-district_cyr"].nunique()
unique_subdistrict_perc = (unique_subdistrict_by_source / total_unique_subdistrict) * 100 if total_unique_subdistrict > 0 else 0
status_counts = df_manual["STATUS"].value_counts()
missing_id_master_count = (df_output["id_master"] == "missing").sum()
perc_missing_id_master = (missing_id_master_count / total_exploded) * 100 if total_exploded > 0 else 0
total_missing_rows = df_missing.shape[0]
missing_gps_dashes = df_missing["MISS_CENTER_GPS"].astype(str).str.contains("---").sum()
perc_missing_gps_dashes = (missing_gps_dashes / total_missing_rows) * 100 if total_missing_rows > 0 else 0

print("\n--- summary report ---\n")
print("total rows in manual file (STATUS != 'to remove'): {} rows".format(total_manual))
print("total rows in output file after explosion: {} rows ({:.2f}%)".format(total_exploded, perc_explosion))
print("source breakdown in output file: " + ", ".join(["{}: {} rows ({:.2f}%)".format(src, cnt, source_perc[src]) for src, cnt in source_counts.items()]))
print("unique sub-district_cyr count by source: " + ", ".join(["{}: {} unique values ({:.2f}%)".format(src, cnt, unique_subdistrict_perc[src]) for src, cnt in unique_subdistrict_by_source.items()]))
print("manual file STATUS counts: " + ", ".join(["{}: {} rows ({:.2f}%)".format(status, cnt, (cnt/total_manual*100)) for status, cnt in status_counts.items()]))
print("count of 'missing' in id_master in output file: {} rows ({:.2f}%)".format(missing_id_master_count, perc_missing_id_master))
print("missing sapir sub-district file statistics: total rows: {} rows, rows where MISS_CENTER_GPS contains '---': {} rows ({:.2f}%)".format(total_missing_rows, missing_gps_dashes, perc_missing_gps_dashes))

# saving output files
output_file = os.path.join(file_dir, "places_unpivoted_with_sapir_sub-district.xlsx")
missing_file = os.path.join(file_dir, "places_missing_in_sapir_sub-district.xlsx")
df_output.to_excel(output_file, index=False)
df_missing.to_excel(missing_file, index=False)
print("\noutput files saved: unpivoted file -> {}, missing sub-district file -> {}".format(output_file, missing_file))

# displaying random samples
sample_output = df_output.sample(n=20, random_state=42) if total_exploded >= 20 else df_output.copy()
sample_missing = df_missing.sample(n=20, random_state=42) if total_missing_rows >= 20 else df_missing.copy()
print("\n--- sample of 20 random rows from unpivoted sub-district file ---")
display(sample_output)
print("\n--- sample of 20 random rows from missing sub-district file ---")
display(sample_missing)



--- summary report ---

total rows in manual file (STATUS != 'to remove'): 59 rows
total rows in output file after explosion: 808 rows (1369.49%)
source breakdown in output file: title: 442 rows (54.70%), fond: 366 rows (45.30%)
unique sub-district_cyr count by source: fond: 5 unique values (8.77%), title: 54 unique values (94.74%)
manual file STATUS counts: to export: 38 rows (64.41%), to use: 21 rows (35.59%)
count of 'missing' in id_master in output file: 47 rows (5.82%)
missing sapir sub-district file statistics: total rows: 59 rows, rows where MISS_CENTER_GPS contains '---': 6 rows (10.17%)

output files saved: unpivoted file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_unpivoted_with_sapir_sub-district.xlsx, missing sub-district file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_missing_in_sapir_sub-district.xlsx

--- sample of 20 random rows from unpivoted sub-district file ---


,source,sub-district_cyr,norm_sub-district_name_cyr,norm_sub-district_type_cyr,sapir_sub-district,sub-district_missing,id_old,id_master,sapir_related_ids,sapir_subDistrictCode,sapir_id,full_sapir,MISS_CENTER_GPS,MISS_REF1,MISS_REF2
697,fond,Шоломалейхімська сільрада,Шолом Алейхем,сільрада,SHOLOM ALEYKHEM,,15082,11026,"5435440, 5464965, 5707013, 6089197, 6089208, 7...",T9184734,,,NaN,NaN,NaN
296,fond,Львівська сільська рада,Львове,сільська рада,LVOVO,,14989,10933,"5429850, 5512611, 5512613, 5512614",T9164492,,,NaN,NaN,NaN
227,title,Куйбишівська сільрада,Куйбишеве,сільрада,,Kuibysheve rural council,13867,9868,NaN,NaN,,,"47.011379920006846, 32.642040140474414",https://uk.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...,https://uk.wikipedia.org/wiki/%D0%9A%D1%83%D0%...
336,fond,Львівська сільська рада,Львове,сільська рада,LVOVO,,15029,10973,"5429850, 5512611, 5512613, 5512614",T9164492,,,NaN,NaN,NaN
537,title,Лановецька ґміна,Ланівці,ґміна,Łanowce,,4627,missing,"7223099, 9866204, 10390950, 10390975",T9355651,,,NaN,NaN,NaN
794,title,Шоломалейхімська сільрада,Шолом Алейхем,сільрада,SHOLOM ALEYKHEM,,13771,9772,"5435440, 5464965, 5707013, 6089197, 6089208, 7...",T9184734,,,NaN,NaN,NaN
198,title,Кіровська сільрада,Кірове,сільрада,KIROVO,,13353,9348,"5453596, 9750838, 9750867, 9750878, 9750887, 9...",T9236273,,,NaN,NaN,NaN
247,title,Ленінська сільрада,Ленінське,сільрада,,Leninske rural council,13373,9368,NaN,NaN,,,---,---,NaN
448,title,Калинінська сільрада,Калінінське,сільрада,,Kalininske rural council,12985,8953,NaN,NaN,,,"47.114909224110654, 32.97937089357428",http://resource.history.org.ua/cgi-bin/eiu/his...,NaN
572,title,Хрущовська сільрада,Хрущове,сільрада,,Khrushchove rural council,12815,8783,NaN,NaN,,,---,---,NaN



--- sample of 20 random rows from missing sub-district file ---


,source_ids,source,ORIG_SUB-DISTRICT_STRING_CYR,NORM_SUB-DISTRICT_NAME_CYR,NORM_SUB-DISTRICT_TYPE_CYR,NORM_SUB-DISTRICT_NAME_ENG_ABSENT,NORM_SUB-DISTRICT_TYPE_ENG_ABSENT,SAPIR_SUB-DISTRICT_LACKING_DEDICATED_RECORD,MISS_CENTER_GPS,MISS_REF1,MISS_REF2,SAPIR_RELATED_IDS_FOR_LACKING_DEDICATED_RECORD,SAPIR_subDistrictCode_FOR_LACKING_DEDICATED_RECORD
0,6535,title,Базарська сільрада,Базар,сільрада,Bazar,rural council,,"48.948468048084045, 25.57625885462642",https://uk.wikipedia.org/wiki/%D0%91%D0%B0%D0%...,https://uk.wikipedia.org/wiki/%D0%91%D0%B0%D0%...,NaN,NaN
5,"12575, 18548",title,Великосейдеминуська сільрада,Велика Сейдеминуха,сільрада,Velyka Seydeminukha,rural council,,"47.114909224110654, 32.97937089357428",https://uk.wikisource.org/wiki/%D0%90%D1%80%D1...,NaN,NaN,NaN
34,9192,title,Тернопільська ґміна,Тернопіль,ґміна,Ternopil,gmina,,"49.55054894321392, 25.59750302470573",---,NaN,NaN,NaN
13,"12825, 12844, 12850, 12902, 12938, 12955, 1309...",title,Карломарксівська сільрада,Карла Маркса,сільрада,Karla Marksa,rural council,,---,---,NaN,NaN,NaN
50,"4597, 4627",title,Лановецька ґміна,Ланівці,ґміна,,,Łanowce,NaN,NaN,NaN,"7223099, 9866204, 10390950, 10390975",T9355651
92,12707,title,Емесівська сільрада,Емес,сільрада,,,EMES,NaN,NaN,NaN,"5456762, 6085674, 9440231, 9440333, 11374666, ...",T1421034
145,"12712, 13290, 13764, 13765, 13766, 13771, 1379...",title,Шоломалейхімська сільрада,Шолом Алейхем,сільрада,,,SHOLOM ALEYKHEM,NaN,NaN,NaN,"5435440, 5464965, 5707013, 6089197, 6089208, 7...",T9184734
25,"12713, 12829, 12853, 12886, 12906, 12942, 1295...",title,Львовська сільрада,Львове,сільрада,,,LVOVO,NaN,NaN,NaN,"5429850, 5512611, 5512613, 5512614",T9164492
58,"12832, 12846, 12856, 12909, 12945, 12962, 1299...",title,Чкалівська сільрада,Чкалове,сільрада,Chkalove,rural council,,"47.06597208519968, 33.2092939362982",https://uk.wikipedia.org/wiki/%D0%A7%D0%BA%D0%...,NaN,NaN,NaN
12,12708,title,Калініндорфська сільрада,Калініндорф,сільрада,Kalinindorf,rural council,,"47.114909224110654, 32.97937089357428",http://resource.history.org.ua/cgi-bin/eiu/his...,NaN,NaN,NaN


In [ ]:
#@title REGION MATCHING WITH SAPIR TABLE - REGEX & FUZZY MATCH

# loading similarity threshold for fuzzy matching
similarity_threshold = 85

# loading mapping dictionaries for transliteration
base_dict = {'а': ['a'], 'б': ['b'], 'в': ['v', 'w'], 'г': ['h', 'g'], 'ґ': ['g'], 'д': ['d', 'j', 'dz'], 'е': ['e', 'ye', 'ie'], 'є': ['ye', 'je', 'ie', 'e'], 'ж': ['zh', 'j', 'rz', 'ż'], 'з': ['z'], 'и': ['y', 'i'], 'і': ['i'], 'ї': ['yi', 'ji', 'ii', 'i', 'e', 'ye', 'ie', 'je'], 'й': ['y', 'j', 'i'], 'к': ['k', 'c', 'q'], 'л': ['l', 'ł'], 'м': ['m'], 'н': ['n'], 'о': ['o', 'je'], 'п': ['p'], 'р': ['r', 'rz'], 'с': ['s'], 'т': ['t'], 'у': ['u', 'ou'], 'ф': ['f', 'ph'], 'х': ['kh', 'ch', 'h'], 'ц': ['ts', 'tz', 'z', 'c'], 'ч': ['ch', 'tch', 'tsch', 'cz'], 'ш': ['sh', 'sch', 'sz'], 'щ': ['shch', 'sch', 'sc', 'szcz'], 'ю': ['yu', 'ju', 'iu'], 'я': ['ya', 'ja', 'ia', 'je']}

extended_dict = {'а': ['a'], 'б': ['b'], 'в': ['v', 'w', 'l'], 'г': ['h', 'g'], 'ґ': ['g'], 'д': ['d', 'j', 'dz'], 'е': ['e', 'ye', 'ie'], 'є': ['ye', 'je', 'ie', 'e'], 'ж': ['zh', 'j', 'rz', 'ż'], 'з': ['z'], 'и': ['y', 'i', 'e'], 'і': ['i', 'o', 'y', 'e'], 'ї': ['yi', 'ji', 'ii', 'i', 'e', 'ye', 'ie', 'je'], 'й': ['y', 'j', 'i'], 'к': ['k', 'c', 'q'], 'л': ['l', 'ł'], 'м': ['m'], 'н': ['n'], 'о': ['o'], 'п': ['p'], 'р': ['r', 'rz'], 'с': ['s'], 'т': ['t'], 'у': ['u', 'ou', 'ó'], 'ф': ['f', 'ph'], 'х': ['kh', 'ch', 'h'], 'ц': ['ts', 'tz', 'z', 'c'], 'ч': ['ch', 'tch', 'tsch', 'cz'], 'ш': ['sh', 'sch', 'sz'], 'щ': ['shch', 'sch', 'sc', 'szcz'], 'ю': ['yu', 'ju', 'iu', 'u'], 'я': ['ya', 'ja', 'ia', 'a', 'ie']}

# defining a function to build regex pattern from a cyrillic string using the extended dictionary
def build_regex_pattern(query_cyr, ext_dict):
    pattern = ""
    for char in query_cyr:
        if char in ext_dict:
            alternatives = ext_dict[char]
            pattern += "(" + "|".join(re.escape(alt) for alt in alternatives) + ")"
        else:
            pattern += re.escape(char)
    return pattern

# defining file paths
final_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/final_places_mach_transl_with_fixed.xlsx"
sapir_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/SAPIR_TABLES/t_SC_Places_Items(ENG_HEB)_Values.xlsx"

# loading excel files
display(Markdown("**loading excel files...**"))
df_final = pd.read_excel(final_path)
df_sapir = pd.read_excel(sapir_path)

# filtering sapir dataframe to include only rows where both 'place' and 'district' equal "<>"
df_sapir = df_sapir[(df_sapir["place"] == "<>") & (df_sapir["district"] == "<>")]
df_sapir = df_sapir.reset_index(drop=True)
# using the 'region' column for matching instead of 'district'
df_sapir["region_norm"] = df_sapir["region"].astype(str).str.lower().str.strip()

# creating normalized columns for matching
df_final["final_places_eng_norm"] = df_final["norm_admun_name_eng"].astype(str).str.lower().str.strip()
df_final["final_places_regex"] = df_final["norm_admun_name_cyr"].astype(str).str.lower().str.strip().map(lambda x: build_regex_pattern(str(x), extended_dict))

# filtering final file for region category
df_final_region = df_final[df_final["sapir_category"].astype(str).str.lower() == "region"].copy()

output_rows = []
match_counts = {}
regex_match_count = 0
fuzzy_match_count = 0
direct_match_count = 0
unique_regex_matches = set()
unique_fuzzy_matches = set()
unique_direct_matches = set()

# iterating over each row in final region file
for idx, row in df_final_region.iterrows():
    # selecting candidate matching value based on priority: sapir_name, norm_admun_name_cyr, then final_places
    sapir_name = row["sapir_name"] if "sapir_name" in row else ""
    norm_cyr = row["norm_admun_name_cyr"] if "norm_admun_name_cyr" in row else ""
    final_places_val = row["final_places"] if "final_places" in row else ""

    if pd.notna(sapir_name) and str(sapir_name).strip() != "":
        query_value = str(sapir_name).strip().lower()
        match_method = "direct"
    elif pd.notna(norm_cyr) and str(norm_cyr).strip() != "":
        query_value = str(norm_cyr).strip().lower()
        query_regex = build_regex_pattern(query_value, extended_dict)
        query_eng = row["norm_admun_name_eng"] if pd.notna(row["norm_admun_name_eng"]) else query_value
        query_eng = str(query_eng).strip().lower()
        match_method = "regex"
    elif pd.notna(final_places_val) and str(final_places_val).strip() != "":
        query_value = str(final_places_val).strip().lower()
        query_regex = build_regex_pattern(query_value, extended_dict)
        query_eng = query_value
        match_method = "regex"
    else:
        match_method = None
        query_value = ""

    candidate_indices = []
    if match_method == "direct":
        # direct exact matching on region_norm
        candidate_indices = df_sapir.index[df_sapir["region_norm"] == query_value].tolist()
        direct_match_count += len(candidate_indices)
        method_used = "direct"
    elif match_method == "regex":
        # base filtering using first character from query_value
        source_first = query_value[0].lower() if query_value else ""
        filtered_candidates_base = []
        for match_index, sapir_row in df_sapir.iterrows():
            candidate_val = sapir_row["region_norm"]
            candidate_first = candidate_val[0] if candidate_val else ""
            if source_first in base_dict and candidate_first in base_dict[source_first]:
                filtered_candidates_base.append(match_index)
        filtered_candidates_regex = []
        for match_index in filtered_candidates_base:
            candidate_norm = df_sapir.loc[match_index]["region_norm"]
            if re.fullmatch(query_regex, candidate_norm):
                filtered_candidates_regex.append(match_index)
        if filtered_candidates_regex:
            candidate_indices = filtered_candidates_regex
            regex_match_count += len(filtered_candidates_regex)
            method_used = "regex"
        else:
            matches = process.extract(query_eng, df_sapir["region_norm"].tolist(), scorer=fuzz.token_sort_ratio, score_cutoff=similarity_threshold, limit=None)
            candidate_indices = [match[2] for match in matches]
            fuzzy_match_count += len(candidate_indices)
            method_used = "fuzzy"
    else:
        candidate_indices = []
        method_used = None

    match_counts[row["norm_admun_name_eng"]] = len(candidate_indices)

    final_eng_val = row["norm_admun_name_eng"]
    if candidate_indices:
        if method_used == "regex":
            unique_regex_matches.add(final_eng_val)
            unique_fuzzy_matches.discard(final_eng_val)
            unique_direct_matches.discard(final_eng_val)
        elif method_used == "fuzzy":
            if final_eng_val not in unique_regex_matches and final_eng_val not in unique_fuzzy_matches and final_eng_val not in unique_direct_matches:
                unique_fuzzy_matches.add(final_eng_val)
        elif method_used == "direct":
            unique_direct_matches.add(final_eng_val)
            unique_regex_matches.discard(final_eng_val)
            unique_fuzzy_matches.discard(final_eng_val)

    # if no matches found, append row with empty match fields
    if not candidate_indices:
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "ORIG_REGION_STRING_CYR": row["final_places"],
            "NORM_REGION_NAME_CYR": row["norm_admun_name_cyr"],
            "NORM_REGION_TYPE_CYR": row["norm_admun_type_cyr"],
            "NORM_REGION_NAME_ENG": row["norm_admun_name_eng"],
            "NORM_REGION_TYPE_ENG": row["norm_admun_type_eng"],
            "SAPIR_REGION_MATCH": "",
            "SAPIR_REGION_ID": "",
            "SAPIR_REGION_VALUE": "",
            "COUNT_MATCHES_BEF_MAN": 0,
            "CORRECTED": "",
            "0_MATCHES": "",
            "1_MISSMATCH": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "MISSING_GPS": ""
        })

    # for each match found, append row with match details
    for match_index in candidate_indices:
        sapir_row = df_sapir.loc[match_index]
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "ORIG_REGION_STRING_CYR": row["final_places"],
            "NORM_REGION_NAME_CYR": row["norm_admun_name_cyr"],
            "NORM_REGION_TYPE_CYR": row["norm_admun_type_cyr"],
            "NORM_REGION_NAME_ENG": row["norm_admun_name_eng"],
            "NORM_REGION_TYPE_ENG": row["norm_admun_type_eng"],
            # using the 'region' column for matching instead of 'district'
            "SAPIR_REGION_MATCH": sapir_row["region"],
            "SAPIR_REGION_ID": sapir_row["book_id"],
            "SAPIR_REGION_VALUE": sapir_row["TIENG"],
            "COUNT_MATCHES_BEF_MAN": len(candidate_indices),
            "CORRECTED": "",
            "0_MATCHES": "",
            "1_MISSMATCH": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "MISSING_GPS": ""
        })

df_output = pd.DataFrame(output_rows)
# saving the output file without adding an empty 'id' column
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_sapir_regex _&_fuzzy_match_region_with_man.xlsx"
df_output.to_excel(output_file, index=False)

# computing summary statistics for norm_admun_name_eng values (unique values in final region file)
total_values = df_final_region["norm_admun_name_eng"].nunique()
values_with_matches = sum(1 for count in match_counts.values() if count > 0)
values_with_no_matches = sum(1 for count in match_counts.values() if count == 0)

# building distribution of match counts
match_distribution = {}
for count in match_counts.values():
    match_distribution[count] = match_distribution.get(count, 0) + 1

# preparing summary report string for overall norm_region_name_eng matching
report_lines = []
report_lines.append(f"total unique norm_region_name_eng values processed: {total_values}")
report_lines.append(f"values with one or more matches: {values_with_matches} ({(values_with_matches/total_values*100):.2f}%)")
report_lines.append(f"values with no matches: {values_with_no_matches} ({(values_with_no_matches/total_values*100):.2f}%)")
for count in sorted(match_distribution.keys()):
    report_lines.append(f"values with exactly {count} match(es): {match_distribution[count]} ({(match_distribution[count]/total_values*100):.2f}%)")
report_text = "\n\n".join(report_lines)
display(Markdown("**summary report:**"))
display(Markdown(report_text))

# computing unique matching method statistics for norm_admun_name_eng (without duplicates)
unique_total = len(unique_regex_matches.union(unique_fuzzy_matches).union(unique_direct_matches))
if unique_total > 0:
    unique_regex_percentage = len(unique_regex_matches) / unique_total * 100
    unique_fuzzy_percentage = len(unique_fuzzy_matches) / unique_total * 100
    unique_direct_percentage = len(unique_direct_matches) / unique_total * 100
else:
    unique_regex_percentage = 0
    unique_fuzzy_percentage = 0
    unique_direct_percentage = 0
display(Markdown("**unique match method statistics (by norm_admun_name_eng):**"))
display(Markdown(f"unique regex matches: {len(unique_regex_matches)} ({unique_regex_percentage:.2f}%)"))
display(Markdown(f"unique fuzzy matches: {len(unique_fuzzy_matches)} ({unique_fuzzy_percentage:.2f}%)"))
display(Markdown(f"unique direct matches: {len(unique_direct_matches)} ({unique_direct_percentage:.2f}%)"))

# displaying 20 random sample rows with matches (unique region values)
df_matches = df_output[df_output["COUNT_MATCHES_BEF_MAN"] > 0].copy()
df_unique_regions = df_matches.drop_duplicates(subset=["ORIG_REGION_STRING_CYR", "NORM_REGION_NAME_ENG"])
if len(df_unique_regions) >= 20:
    sample_df = df_unique_regions.sample(n=20, random_state=42)
else:
    sample_df = df_unique_regions
display(Markdown("**20 random sample rows with matches (unique region values):**"))
display(sample_df[["ORIG_REGION_STRING_CYR", "NORM_REGION_NAME_CYR", "NORM_REGION_NAME_ENG", "SAPIR_REGION_MATCH", "SAPIR_REGION_ID", "SAPIR_REGION_VALUE", "COUNT_MATCHES_BEF_MAN"]])

display(Markdown(f"**output file saved at:** {output_file}"))
display(Markdown("**region matching process completed successfully.**"))


**loading excel files...**

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


**summary report:**

total unique norm_region_name_eng values processed: 18

values with one or more matches: 5 (27.78%)

values with no matches: 14 (77.78%)

values with exactly 0 match(es): 14 (77.78%)

values with exactly 1 match(es): 5 (27.78%)

**unique match method statistics (by norm_admun_name_eng):**

...

unique direct matches: 2 (40.00%)

**20 random sample rows with matches (unique region values):**

,ORIG_REGION_STRING_CYR,NORM_REGION_NAME_CYR,NORM_REGION_NAME_ENG,SAPIR_REGION_MATCH,SAPIR_REGION_ID,SAPIR_REGION_VALUE,COUNT_MATCHES_BEF_MAN
0,Буковина,NaN,NaN,Bukovina,5700112,"<>,<>,Bukovina,Romania",1
18,Закарпаття,NaN,NaN,Carpathian Ruthenia,5720537,"<>,<>,Carpathian Ruthenia,Czechoslovakia",1
19,Закарпатье,NaN,NaN,Carpathian Ruthenia,5720537,"<>,<>,Carpathian Ruthenia,Czechoslovakia",1
22,Львівська область,Львів,Lviv,Lwow,5703942,"<>,<>,Lwow,Poland",1
24,Львівщина,NaN,NaN,Lwow,5703942,"<>,<>,Lwow,Poland",1
25,Львовська область,Львів,Lviv,Lwow,5703942,"<>,<>,Lwow,Poland",1
28,Николаевская область,Николаев,Nikolaev,Nikolayev,5703971,"<>,<>,Nikolayev,Ukraine (USSR)",1
30,Одесская область,Одесса,Odessa,Odessa,7599998,"<>,<>,Odessa,Ukraine (USSR)",1
32,Одеська губернія,Одеса,Odesa,Odessa,7599998,"<>,<>,Odessa,Ukraine (USSR)",1


**output file saved at:** /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_sapir_regex _&_fuzzy_match_region_with_man.xlsx

**region matching process completed successfully.**

In [ ]:
#@title UNPIVOTED REGIONS WITH SAPIR


# loading the directory path for the files
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# defining filenames for manual region data and mapping data
manual_file = "places_sapir_regex _&_fuzzy_match_region_with_man.xlsx"
mapping_file = "t_Master_table_draft_dana_withdates_mapped.xlsx"

# reading the excel files
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(file_dir, mapping_file))

# converting id columns to string types in both dataframes;
# for mapping file, converting numeric values to int then to string to remove decimals
df_manual["source_ids"] = df_manual["source_ids"].astype(str)
df_mapping["old_id"] = df_mapping["old_id"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")
df_mapping["ID_master"] = df_mapping["ID_master"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")

# filtering out rows where status equals 'to remove'
df_manual = df_manual[df_manual["STATUS"] != "to remove"].copy()

# creating new columns for target region output based on manual file;
# region_cyr from ORIG_REGION_STRING_CYR
df_manual["region_cyr"] = df_manual["ORIG_REGION_STRING_CYR"]
# norm_region_name_cyr from NORM_REGION_NAME_CYR
df_manual["norm_region_name_cyr"] = df_manual["NORM_REGION_NAME_CYR"]
# norm_region_type_cyr from NORM_REGION_TYPE_CYR
df_manual["norm_region_type_cyr"] = df_manual["NORM_REGION_TYPE_CYR"]
# region_sapir from SAPIR_REGION_MATCH
df_manual["region_sapir"] = df_manual["SAPIR_REGION_MATCH"]
# region_missing as concatenation of NORM_REGION_NAME_ENG and NORM_REGION_TYPE_ENG with a space,
# but only for rows where STATUS equals "to export"
df_manual["region_missing"] = df_manual.apply(lambda row: str(row["NORM_REGION_NAME_ENG"]) + " " + str(row["NORM_REGION_TYPE_ENG"]) if row["STATUS"]=="to export" else "", axis=1)
# parachronistic_region_sapir: directly taken from parachronistic_region_sapir column
df_manual["parachronistic_region_sapir"] = df_manual["parachronistic_region_sapir"]
# parachronistic_id_sapir: directly taken from parachronistic_id_sapir column
df_manual["parachronistic_id_sapir"] = df_manual["parachronistic_id_sapir"]

# splitting source_ids by comma into a list
df_manual["id_old_list"] = df_manual["source_ids"].apply(lambda x: [item.strip() for item in x.split(",") if item.strip() != ""])

# exploding the source_ids into separate rows and renaming the column to id_old
df_exploded = df_manual.explode("id_old_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"id_old_list": "id_old"})
df_exploded["id_old"] = df_exploded["id_old"].astype(str)

# defining a function to convert id values: first to float, then to int, then to string
def conv_id(x):
    try:
        return str(int(float(x)))
    except:
        return x

# converting id_old to integer then to string to avoid decimals/extra zeros
df_exploded["id_old"] = df_exploded["id_old"].apply(conv_id)

# merging to bring in id_master from mapping file based on id_old matching old_id
df_merged = pd.merge(df_exploded, df_mapping[["old_id", "ID_master"]], left_on="id_old", right_on="old_id", how="left")
df_merged["ID_master"] = df_merged["ID_master"].fillna("missing")
df_merged = df_merged.rename(columns={"ID_master": "id_master"})
if "old_id_y" in df_merged.columns:
    df_merged = df_merged.drop(columns=["old_id_y"])

# converting id_master to integer then to string
df_merged["id_master"] = df_merged["id_master"].apply(conv_id)

# preparing final unpivoted output dataframe with required columns
# including sapir_id from SAPIR_REGION_ID (from manual file) – converting it via conv_id
cols_output = [
    "source",
    "region_cyr",
    "norm_region_name_cyr",
    "norm_region_type_cyr",
    "region_sapir",
    "region_missing",
    "parachronistic_region_sapir",
    "parachronistic_id_sapir",
    "id_old",
    "id_master",
    "SAPIR_REGION_ID",
    "SAPIR_REGION_VALUE",
    "MISS_CENTER_GPS",
    "MISS_REF1"
]
df_output = df_merged[cols_output].copy()
# renaming SAPIR_REGION_ID to sapir_id and SAPIR_REGION_VALUE to full_sapir
df_output = df_output.rename(columns={"SAPIR_REGION_ID": "sapir_id", "SAPIR_REGION_VALUE": "full_sapir"})

# converting sapir_id to integer then to string
df_output["sapir_id"] = df_output["sapir_id"].apply(conv_id)

# preparing the missing in sapir region output file;
# including only rows where STATUS equals "to export"
cols_missing = [
    "source_ids",
    "source",
    "ORIG_REGION_STRING_CYR",
    "NORM_REGION_NAME_CYR",
    "NORM_REGION_TYPE_CYR",
    "NORM_REGION_NAME_ENG",
    "NORM_REGION_TYPE_ENG",
    "MISS_CENTER_GPS",
    "MISS_REF1",
    "parachronistic_id_sapir",
    "parachronistic_region_sapir"
]
df_missing = df_manual[df_manual["STATUS"] == "to export"][cols_missing].drop_duplicates()

# computing statistics
total_manual = df_manual.shape[0]
total_exploded = df_output.shape[0]
perc_explosion = (total_exploded / total_manual) * 100 if total_manual > 0 else 0
source_counts = df_output["source"].value_counts()
source_perc = (source_counts / source_counts.sum()) * 100
unique_region_by_source = df_output.groupby("source")["region_cyr"].nunique()
total_unique_region = df_output["region_cyr"].nunique()
unique_region_perc = (unique_region_by_source / total_unique_region) * 100 if total_unique_region > 0 else 0
status_counts = df_manual["STATUS"].value_counts()
missing_id_master_count = (df_output["id_master"] == "missing").sum()
perc_missing_id_master = (missing_id_master_count / total_exploded) * 100 if total_exploded > 0 else 0
total_missing_rows = df_missing.shape[0]
missing_gps_dashes = df_missing["MISS_CENTER_GPS"].astype(str).str.contains("---").sum()
perc_missing_gps_dashes = (missing_gps_dashes / total_missing_rows) * 100 if total_missing_rows > 0 else 0

# printing summary report in a condensed format
print("\n--- summary report ---\n")
print("total rows in manual file (status != 'to remove'): {} rows".format(total_manual))
print("total rows in output file after explosion: {} rows ({:.2f}%)".format(total_exploded, perc_explosion))
print("source breakdown in output file: " + ", ".join(["{}: {} rows ({:.2f}%)".format(src, cnt, source_perc[src]) for src, cnt in source_counts.items()]))
print("unique region_cyr count by source: " + ", ".join(["{}: {} unique values ({:.2f}%)".format(src, cnt, unique_region_perc[src]) for src, cnt in unique_region_by_source.items()]))
print("manual file status counts: " + ", ".join(["{}: {} rows ({:.2f}%)".format(status, cnt, (cnt/total_manual*100)) for status, cnt in status_counts.items()]))
print("count of 'missing' in id_master in output file: {} rows ({:.2f}%)".format(missing_id_master_count, perc_missing_id_master))
print("missing sapir region file statistics: total rows: {} rows, rows where MISS_CENTER_GPS contains '---': {} rows ({:.2f}%)".format(total_missing_rows, missing_gps_dashes, perc_missing_gps_dashes))

# saving output files with updated filenames
output_file = os.path.join(file_dir, "places_unpivoted_with_sapir_region.xlsx")
missing_file = os.path.join(file_dir, "places_missing_in_sapir_region.xlsx")
df_output.to_excel(output_file, index=False)
df_missing.to_excel(missing_file, index=False)
print("\noutput files saved: unpivoted file -> {}, missing region file -> {}".format(output_file, missing_file))

# displaying random samples from the output files
sample_output = df_output.sample(n=20, random_state=42) if total_exploded >= 20 else df_output.copy()
sample_missing = df_missing.sample(n=20, random_state=42) if total_missing_rows >= 20 else df_missing.copy()
print("\n--- sample of 20 random rows from unpivoted region file ---")
display(sample_output)
print("\n--- sample of 20 random rows from missing region file ---")
display(sample_missing)



--- summary report ---

total rows in manual file (status != 'to remove'): 51 rows
total rows in output file after explosion: 4945 rows (9696.08%)
source breakdown in output file: fond: 4140 rows (83.72%), title: 805 rows (16.28%)
unique region_cyr count by source: fond: 15 unique values (42.86%), title: 34 unique values (97.14%)
manual file status counts: to export: 36 rows (70.59%), to use: 15 rows (29.41%)
count of 'missing' in id_master in output file: 161 rows (3.26%)
missing sapir region file statistics: total rows: 36 rows, rows where MISS_CENTER_GPS contains '---': 0 rows (0.00%)

output files saved: unpivoted file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_unpivoted_with_sapir_region.xlsx, missing region file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_missing_in_sapir_region.xlsx

--- sample of 20 random rows from unpivoted region file ---


,source,region_cyr,norm_region_name_cyr,norm_region_type_cyr,region_sapir,region_missing,parachronistic_region_sapir,parachronistic_id_sapir,id_old,id_master,sapir_id,full_sapir,MISS_CENTER_GPS,MISS_REF1
151,fond,Волинське воєводство,Волинь,воєводство,Wolyn,,NaN,NaN,9404,1852,5703791,"<>,<>,Wolyn,Poland",NaN,NaN
807,fond,Волынское воеводство,Волынь,воеводство,Wolyn,,NaN,NaN,243,15708,5703791,"<>,<>,Wolyn,Poland",NaN,NaN
621,title,Волинь,NaN,NaN,Wolyn,,NaN,NaN,1557,7185,5703791,"<>,<>,Wolyn,Poland",NaN,NaN
3976,fond,Херсонська округа,Херсон,округа,NaN,Kherson okruha,NaN,NaN,18812,14753,NaN,NaN,"46.654020677463755, 32.599544743751984",https://uk.wikipedia.org/wiki/%D0%A5%D0%B5%D1%...
3996,fond,Херсонська округа,Херсон,округа,NaN,Kherson okruha,NaN,NaN,18832,14773,NaN,NaN,"46.654020677463755, 32.599544743751984",https://uk.wikipedia.org/wiki/%D0%A5%D0%B5%D1%...
1747,fond,Волынское воеводство,Волынь,воеводство,Wolyn,,NaN,NaN,1270,6909,5703791,"<>,<>,Wolyn,Poland",NaN,NaN
555,title,Волинь,NaN,NaN,Wolyn,,NaN,NaN,1440,7078,5703791,"<>,<>,Wolyn,Poland",NaN,NaN
4544,fond,Николаевская область,Николаев,область,Nikolayev,,NaN,NaN,2772,17221,5703971,"<>,<>,Nikolayev,Ukraine (USSR)",NaN,NaN
1536,fond,Волынское воеводство,Волынь,воеводство,Wolyn,,NaN,NaN,972,7439,5703791,"<>,<>,Wolyn,Poland",NaN,NaN
4110,title,Закарпатье,NaN,NaN,NaN,nan nan,"<>,<>,Carpathian Ruthenia,Czechoslovakia",5720537.0,7577,8089,NaN,NaN,"48.62242699013335, 22.287959651054354",https://rb.gy/b9u39e



--- sample of 20 random rows from missing region file ---


,source_ids,source,ORIG_REGION_STRING_CYR,NORM_REGION_NAME_CYR,NORM_REGION_TYPE_CYR,NORM_REGION_NAME_ENG,NORM_REGION_TYPE_ENG,MISS_CENTER_GPS,MISS_REF1,parachronistic_id_sapir,parachronistic_region_sapir
50,12763,title,Одеська губернія,Одеса,губернія,Odesa,governorate,"46.47303918330332, 30.707506664064738",https://uk.wikipedia.org/wiki/%D0%9E%D0%B4%D0%...,7599998.0,"<>,<>,Odessa,Ukraine (USSR)"
20,"15875, 15879, 16566",title,Миколаївська губернія,Миколаїв,губернія,Mykolaiv,governorate,"46.966612924913555, 31.997253807374218",https://uk.wikipedia.org/wiki/%D0%9C%D0%B8%D0%...,5703971.0,"<>,<>,Nikolayev,Ukraine (USSR)"
36,12837,title,Черновицька область,Чернівці,область,Chernivtsi,oblast,"48.3041239232254, 25.946253932524098",https://uk.wikipedia.org/wiki/%D0%A7%D0%B5%D1%...,NaN,NaN
41,"10971, 10972, 10973, 10974, 10975, 10976, 1097...",fond,Львівська область,Львів,область,Lviv,oblast,"49.84018871051206, 24.025971188495223",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,5703942.0,"<>,<>,Lwow,Poland"
24,6624,title,Станіславська область,Станіслав,область,Stanyslav,oblast,"48.92067960181313, 24.71105360010301",https://uk.wikipedia.org/wiki/%D0%86%D0%B2%D0%...,5704853.0,"<>,<>,Stanislawow,Poland"
42,"12917, 12924, 12926",title,Львівська область,Львів,область,Lviv,oblast,"49.84018871051206, 24.025971188495223",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,5703942.0,"<>,<>,Lwow,Poland"
30,"12168, 12169, 12170, 12171, 12172, 12173, 1217...",fond,Херсонська область,Херсон,область,Kherson,oblast,"46.654020677463755, 32.599544743751984",https://uk.wikipedia.org/wiki/%D0%A5%D0%B5%D1%...,NaN,NaN
19,6624,title,Ізмаїльська область,Ізмаїл,область,Izmail,oblast,"45.351388584254074, 28.839995604852234",https://uk.wikipedia.org/wiki/%D0%86%D0%B7%D0%...,NaN,NaN
14,"7317, 7318, 7319, 7320, 7321, 7322, 7323, 7324...",fond,Закарпатская область,Закарпатье,область,Transcarpathian,oblast,"48.62242699013335, 22.287959651054354",https://uk.wikipedia.org/wiki/%D0%97%D0%B0%D0%...,5720537.0,"<>,<>,Carpathian Ruthenia,Czechoslovakia"
25,"10149, 10150",fond,Тернопільська губернія,Тернопіль,губернія,Ternopil,governorate,"49.55179976993447, 25.596639722047172",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,5703951.0,"<>,<>,Tarnopol,Poland"


In [ ]:
#@title COUNTRY MATCHING WITH SAPIR TABLE - FUZZY MATCH

# define similarity threshold for fuzzy matching
similarity_threshold = 85

# define file paths (no changes in file paths)
final_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/final_places_mach_transl_with_fixed.xlsx"
sapir_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/SAPIR_TABLES/t_SC_Places_Items(ENG_HEB)_Values.xlsx"

# display loading message
display(Markdown("**loading excel files...**"))

# load final file and sapir file
df_final = pd.read_excel(final_path)
df_sapir = pd.read_excel(sapir_path)

# filter sapir file: keep only rows where columns 'place', 'district' and 'region' equal "<>"
df_sapir = df_sapir[(df_sapir["place"] == "<>") & (df_sapir["district"] == "<>") & (df_sapir["region"] == "<>")]
df_sapir = df_sapir.reset_index(drop=True)
# create normalized column for matching in sapir file (using column 'country' as country)
df_sapir["country_norm"] = df_sapir["country"].astype(str).str.lower().str.strip()

# filter final file for country category (keep only rows where sapir_category equals "country")
df_final_country = df_final[df_final["sapir_category"].astype(str).str.lower() == "country"].copy()

#@title matching process for country
# initialize counters and storage variables
output_rows = []
match_counts = {}
direct_match_count = 0
fuzzy_match_count = 0
unique_direct_matches = set()
unique_fuzzy_matches = set()

# iterate over each row in final file filtered for country
for idx, row in df_final_country.iterrows():
    # determine matching candidate based on available values:
    # if sapir_name is not empty, use it for direct match; otherwise use final_places_eng for fuzzy match
    sapir_name = row["sapir_name"] if "sapir_name" in row else ""
    final_places_eng = row["final_places_eng"] if "final_places_eng" in row else ""

    if pd.notna(sapir_name) and str(sapir_name).strip() != "":
        query_value = str(sapir_name).strip().lower()
        match_method = "direct"
    elif pd.notna(final_places_eng) and str(final_places_eng).strip() != "":
        query_value = str(final_places_eng).strip().lower()
        match_method = "fuzzy"
    else:
        match_method = None
        query_value = ""

    candidate_indices = []
    if match_method == "direct":
        # direct exact matching on country_norm
        candidate_indices = df_sapir.index[df_sapir["country_norm"] == query_value].tolist()
        direct_match_count += len(candidate_indices)
        method_used = "direct"
    elif match_method == "fuzzy":
        # fuzzy matching on country_norm using final_places_eng
        matches = process.extract(query_value, df_sapir["country_norm"].tolist(), scorer=fuzz.token_sort_ratio, score_cutoff=similarity_threshold, limit=None)
        candidate_indices = [match[2] for match in matches]
        fuzzy_match_count += len(candidate_indices)
        method_used = "fuzzy"
    else:
        candidate_indices = []
        method_used = None

    # use final_places (cyrillic) and final_places_eng (english) for output
    # record match count for summary (using final_places_eng as key)
    key_val = row["final_places_eng"] if pd.notna(row["final_places_eng"]) else ""
    match_counts[key_val] = len(candidate_indices)

    # update unique matching sets
    if candidate_indices:
        if method_used == "direct":
            unique_direct_matches.add(key_val)
            unique_fuzzy_matches.discard(key_val)
        elif method_used == "fuzzy":
            if key_val not in unique_direct_matches and key_val not in unique_fuzzy_matches:
                unique_fuzzy_matches.add(key_val)

    # if no matches found, append row with empty match fields
    if not candidate_indices:
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "orig_country_string_cyr": row["final_places"],
            "country_eng": row["final_places_eng"],
            "sapir_country_match": "",
            "sapir_country_id": "",
            "sapir_country_value": "",
            "count_matches_bef_man": 0,
            "corrected": "",
            "status": "",
            "missing_in_sapir": "",
            "sapir_historically_incompatible": "",
            "miss_center_gps": "",
            "miss_ref1": "",
            "parachronistic_sapir_name": "",
            "parachronistic_id_sapir": "",
            "parachronistic_country_sapir": ""
        })

    # for each match found, append row with match details
    for match_index in candidate_indices:
        sapir_row = df_sapir.loc[match_index]
        output_rows.append({
            "source_ids": row["source_ids"],
            "source": row["source"],
            "orig_country_string_cyr": row["final_places"],
            "country_eng": row["final_places_eng"],
            "sapir_country_match": sapir_row["country"],
            "sapir_country_id": sapir_row["book_id"],
            "sapir_country_value": sapir_row["TIENG"],
            "count_matches_bef_man": len(candidate_indices),
            "corrected": "",
            "status": "",
            "missing_in_sapir": "",
            "sapir_historically_incompatible": "",
            "miss_center_gps": "",
            "miss_ref1": "",
            "parachronistic_sapir_name": "",
            "parachronistic_id_sapir": "",
            "parachronistic_country_sapir": ""
        })

# convert output rows to dataframe
df_output = pd.DataFrame(output_rows)

# define output file path with updated name (country instead of region)
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_sapir_fuzzy_match_country_with_man.xlsx"
df_output.to_excel(output_file, index=False)

#@title summary report and sample output for country matching
# compute summary statistics based on final_places_eng values (used as country_eng)
total_values = df_final_country["final_places_eng"].nunique()
values_with_matches = sum(1 for count in match_counts.values() if count > 0)
values_with_no_matches = sum(1 for count in match_counts.values() if count == 0)

# build distribution of match counts
match_distribution = {}
for count in match_counts.values():
    match_distribution[count] = match_distribution.get(count, 0) + 1

# prepare summary report string
report_lines = []
report_lines.append(f"total unique country_eng values processed: {total_values}")
report_lines.append(f"values with one or more matches: {values_with_matches} ({(values_with_matches/total_values*100):.2f}%)")
report_lines.append(f"values with no matches: {values_with_no_matches} ({(values_with_no_matches/total_values*100):.2f}%)")
for count in sorted(match_distribution.keys()):
    report_lines.append(f"values with exactly {count} match(es): {match_distribution[count]} ({(match_distribution[count]/total_values*100):.2f}%)")
report_text = "\n\n".join(report_lines)
display(Markdown("**summary report:**"))
display(Markdown(report_text))

# compute unique matching method statistics (by country_eng) without duplicates
unique_total = len(unique_direct_matches.union(unique_fuzzy_matches))
if unique_total > 0:
    unique_direct_percentage = len(unique_direct_matches) / unique_total * 100
    unique_fuzzy_percentage = len(unique_fuzzy_matches) / unique_total * 100
else:
    unique_direct_percentage = 0
    unique_fuzzy_percentage = 0
display(Markdown("**unique match method statistics (by country_eng):**"))
display(Markdown(f"unique direct matches: {len(unique_direct_matches)} ({unique_direct_percentage:.2f}%)"))
display(Markdown(f"unique fuzzy matches: {len(unique_fuzzy_matches)} ({unique_fuzzy_percentage:.2f}%)"))

# display 20 random sample rows with matches (unique country values)
df_matches = df_output[df_output["count_matches_bef_man"] > 0].copy()
df_unique_countries = df_matches.drop_duplicates(subset=["orig_country_string_cyr", "country_eng"])
if len(df_unique_countries) >= 20:
    sample_df = df_unique_countries.sample(n=20, random_state=42)
else:
    sample_df = df_unique_countries
display(Markdown("**20 random sample rows with matches (unique country values):**"))
display(sample_df[["orig_country_string_cyr", "country_eng", "sapir_country_match", "sapir_country_id", "sapir_country_value", "count_matches_bef_man"]])

display(Markdown(f"**output file saved at:** {output_file}"))
display(Markdown("**country matching process completed successfully.**"))


**loading excel files...**

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


**summary report:**

total unique country_eng values processed: 31

values with one or more matches: 28 (90.32%)

values with no matches: 3 (9.68%)

values with exactly 0 match(es): 3 (9.68%)

values with exactly 1 match(es): 22 (70.97%)

values with exactly 2 match(es): 6 (19.35%)

**unique match method statistics (by country_eng):**

...

unique fuzzy matches: 17 (60.71%)

**20 random sample rows with matches (unique country values):**

,orig_country_string_cyr,country_eng,sapir_country_match,sapir_country_id,sapir_country_value,count_matches_bef_man
48,Чехословацька республіка,Czechoslovak Republic,Czechoslovakia,5706532,"<>,<>,<>,Czechoslovakia",1
15,Палестина,Palestine,British Mandate for Palestine,5700529,"<>,<>,<>,British Mandate for Palestine",1
33,США,USA,United States of America,5708293,"<>,<>,<>,United States of America",1
41,Українська РСРС,Ukrainian RSRS,Ukraine (USSR),5459048,"<>,<>,<>,Ukraine (USSR)",2
18,Польща,Poland,Poland,5696006,"<>,<>,<>,Poland",1
43,УРСР,Ukrainian SSR,Ukraine (USSR),5459048,"<>,<>,<>,Ukraine (USSR)",2
26,Чехословаччина,Czechoslovakia,Czechoslovakia,5706532,"<>,<>,<>,Czechoslovakia",1
14,Німеччина,Germany,Germany,5696329,"<>,<> (<>),<>,Germany",1
10,Іспанія,Spain,Spain,5706282,"<>,<>,<>,Spain",1
20,Радянська Росія,Soviet Russia,Russia (USSR),5447421,"<>,<>,<>,Russia (USSR)",2


**output file saved at:** /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_sapir_fuzzy_match_country_with_man.xlsx

**country matching process completed successfully.**

In [ ]:
#@title UNPIVOTED COUNTRY WITH SAPIR

# defining the directory where the files are located
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# defining filenames for manual data and mapping data
manual_file = "places_sapir_fuzzy_match_country_with_man.xlsx"
mapping_file = "t_Master_table_draft_dana_withdates_mapped.xlsx"

# reading the excel files
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(file_dir, mapping_file))

# converting id columns to string types in both dataframes
df_manual["source_ids"] = df_manual["source_ids"].astype(str)
df_mapping["old_id"] = df_mapping["old_id"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")
df_mapping["ID_master"] = df_mapping["ID_master"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")

# filtering rows where status equals 'to use'
df_manual = df_manual[df_manual["status"] == "to use"].copy()

# for country_cyr, taking orig_country_string_cyr
df_manual["country_cyr"] = df_manual["orig_country_string_cyr"]

# for country_sapir, taking sapir_country_match
df_manual["country_sapir"] = df_manual["sapir_country_match"]

# full_sapir from sapir_country_value
df_manual["full_sapir"] = df_manual["sapir_country_value"]

# splitting source_ids by comma into a list
df_manual["id_old_list"] = df_manual["source_ids"].apply(lambda x: [item.strip() for item in x.split(",") if item.strip() != ""])

# exploding the source_ids into separate rows and renaming the column to id_old
df_exploded = df_manual.explode("id_old_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"id_old_list": "id_old"})
df_exploded["id_old"] = df_exploded["id_old"].astype(str)

# function to convert id values: first to float, then to int, then to string
def conv_id(x):
    try:
        return str(int(float(x)))
    except:
        return x

# converting id_old to integer then to string to avoid decimals/extra zeros
df_exploded["id_old"] = df_exploded["id_old"].apply(conv_id)

# merging to bring in id_master based on id_old matching old_id
df_merged = pd.merge(df_exploded, df_mapping[["old_id", "ID_master"]], left_on="id_old", right_on="old_id", how="left")
df_merged["ID_master"] = df_merged["ID_master"].fillna("missing")
df_merged = df_merged.rename(columns={"ID_master": "id_master"})
if "old_id_y" in df_merged.columns:
    df_merged = df_merged.drop(columns=["old_id_y"])

# converting id_master to integer then to string
df_merged["id_master"] = df_merged["id_master"].apply(conv_id)

# renaming sapir_country_id to sapir_id for output
df_merged = df_merged.rename(columns={"sapir_country_id": "sapir_id"})

# preparing final unpivoted output dataframe with selected columns
cols_output = ["source", "country_cyr", "country_sapir", "id_old", "id_master", "sapir_id", "full_sapir"]
df_output = df_merged[cols_output].copy()
df_output["sapir_id"] = df_output["sapir_id"].apply(conv_id)

# computing statistics
total_manual = df_manual.shape[0]
total_exploded = df_output.shape[0]
perc_explosion = (total_exploded / total_manual) * 100 if total_manual > 0 else 0
source_counts = df_output["source"].value_counts()
source_perc = (source_counts / source_counts.sum()) * 100
unique_country_by_source = df_output.groupby("source")["country_cyr"].nunique()
total_unique_country = df_output["country_cyr"].nunique()
unique_country_perc = (unique_country_by_source / total_unique_country) * 100 if total_unique_country > 0 else 0
status_counts = df_manual["status"].value_counts()
missing_id_master_count = (df_output["id_master"] == "missing").sum()
perc_missing_id_master = (missing_id_master_count / total_exploded) * 100 if total_exploded > 0 else 0

# printing summary report in a condensed format
print("\n--- summary report ---\n")
print("total rows in manual file (status == 'to use'): {} rows".format(total_manual))
print("total rows in output file after explosion: {} rows ({:.2f}%)".format(total_exploded, perc_explosion))
print("source breakdown in output file: " + ", ".join(["{}: {} rows ({:.2f}%)".format(src, cnt, source_perc[src]) for src, cnt in source_counts.items()]))
print("unique country_cyr count by source: " + ", ".join(["{}: {} unique values ({:.2f}%)".format(src, cnt, unique_country_perc[src]) for src, cnt in unique_country_by_source.items()]))
print("manual file status counts: " + ", ".join(["{}: {} rows ({:.2f}%)".format(status, cnt, (cnt/total_manual*100)) for status, cnt in status_counts.items()]))
print("count of 'missing' in id_master in output file: {} rows ({:.2f}%)".format(missing_id_master_count, perc_missing_id_master))

# saving output file
output_file = os.path.join(file_dir, "places_unpivoted_with_sapir_countries.xlsx")
df_output.to_excel(output_file, index=False)
print("\noutput file saved: unpivoted file -> {}".format(output_file))

# displaying random samples
sample_output = df_output.sample(n=100, random_state=42) if total_exploded >= 100 else df_output.copy()
print("\n--- sample of 100 random rows from output file ---")
display(sample_output)



--- summary report ---

total rows in manual file (status == 'to use'): 43 rows
total rows in output file after explosion: 965 rows (2244.19%)
source breakdown in output file: title: 795 rows (82.38%), fond: 170 rows (17.62%)
unique country_cyr count by source: fond: 3 unique values (7.50%), title: 39 unique values (97.50%)
manual file status counts: to use: 43 rows (100.00%)
count of 'missing' in id_master in output file: 509 rows (52.75%)

output file saved: unpivoted file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_unpivoted_with_sapir_countries.xlsx

--- sample of 100 random rows from output file ---


,source,country_cyr,country_sapir,id_old,id_master,sapir_id,full_sapir
889,fond,Україна,Ukraine (USSR),18511,14411,5459048,"<>,<>,<>,Ukraine (USSR)"
468,title,Польща,Poland,4313,missing,5696006,"<>,<>,<>,Poland"
168,title,Польша,Poland,372,6877,5696006,"<>,<>,<>,Poland"
405,title,Польща,Poland,4250,missing,5696006,"<>,<>,<>,Poland"
70,title,Канада,Canada,7747,7615,5707227,"<>,<>,<>,Canada"
...,...,...,...,...,...,...,...
506,title,Польща,Poland,4364,missing,5696006,"<>,<>,<>,Poland"
247,fond,Польща,Poland,7950,482,5696006,"<>,<>,<>,Poland"
759,title,Чехословацька республіка,Czechoslovakia,7845,7713,5706532,"<>,<>,<>,Czechoslovakia"
342,title,Польща,Poland,4187,missing,5696006,"<>,<>,<>,Poland"


In [ ]:
#@title UNPIVOTED חלוקה_אדמיניסטרטיבית

# defining the directory where the files are located
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# defining filenames for manual data and mapping data
manual_file = "places_missing_in_sapir_administrative_division.xlsx"
mapping_file = "t_Master_table_draft_dana_withdates_mapped.xlsx"

# reading the excel files
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(file_dir, mapping_file))

# converting id columns to string types in both dataframes
df_manual["source_ids"] = df_manual["source_ids"].astype(str)
df_mapping["old_id"] = df_mapping["old_id"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")
df_mapping["ID_master"] = df_mapping["ID_master"].apply(lambda x: str(int(x)) if pd.notnull(x) else "")

# splitting source_ids by comma into a list
df_manual["id_old_list"] = df_manual["source_ids"].apply(lambda x: [item.strip() for item in x.split(",") if item.strip() != ""])

# exploding the source_ids into separate rows and renaming the column to id_old
df_exploded = df_manual.explode("id_old_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"id_old_list": "id_old"})
df_exploded["id_old"] = df_exploded["id_old"].astype(str)

# function to convert id values: first to float, then to int, then to string
def conv_id(x):
    try:
        return str(int(float(x)))
    except:
        return x

# converting id_old to integer then to string to avoid decimals/extra zeros
df_exploded["id_old"] = df_exploded["id_old"].apply(conv_id)

# merging to bring in id_master based on id_old matching old_id
df_merged = pd.merge(df_exploded, df_mapping[["old_id", "ID_master"]], left_on="id_old", right_on="old_id", how="left")
df_merged["ID_master"] = df_merged["ID_master"].fillna("missing")
df_merged = df_merged.rename(columns={"ID_master": "id_master"})
if "old_id_y" in df_merged.columns:
    df_merged = df_merged.drop(columns=["old_id_y"])

# converting id_master to integer then to string
df_merged["id_master"] = df_merged["id_master"].apply(conv_id)

# preparing final unpivoted output dataframe with selected columns
df_output = pd.DataFrame({
    "source": df_merged["source"],
    "admdiv_cyr": df_merged["ADMDIV_STRING_CYR"],
    "norm_admdiv_name_cyr": df_merged["NORM_ADMDIV_NAME_CYR"],
    "norm_admdiv_type_cyr": df_merged["NORM_ADMDIV_TYPE_CYR"],
    "admdiv_sapir": "",  # empty string
    "admdiv_missing": df_merged["ADMDIV_STRING_ENG"],
    "id_old": df_merged["id_old"],
    "id_master": df_merged["id_master"],
    "sapir_id": "",  # empty string
    "full_sapir": "",  # empty string
    "MISS_CENTER_GPS": df_merged["MISS_CENTER_GPS"],
    "MISS_REF1": df_merged["MISS_REF1"],
    "MISS_REF2": df_merged["MISS_REF2"]
})

# computing statistics
total_manual = df_manual.shape[0]
total_exploded = df_output.shape[0]
perc_explosion = (total_exploded / total_manual) * 100 if total_manual > 0 else 0
source_counts = df_output["source"].value_counts()
source_perc = (source_counts / source_counts.sum()) * 100
unique_admdiv_by_source = df_output.groupby("source")["admdiv_cyr"].nunique()
total_unique_admdiv = df_output["admdiv_cyr"].nunique()
unique_admdiv_perc = (unique_admdiv_by_source / total_unique_admdiv) * 100 if total_unique_admdiv > 0 else 0
missing_id_master_count = (df_output["id_master"] == "missing").sum()
perc_missing_id_master = (missing_id_master_count / total_exploded) * 100 if total_exploded > 0 else 0

# printing summary report in a condensed format
print("\n--- summary report ---\n")
print("total rows in manual file: {} rows".format(total_manual))
print("total rows in output file after explosion: {} rows ({:.2f}%)".format(total_exploded, perc_explosion))
print("source breakdown in output file: " + ", ".join(["{}: {} rows ({:.2f}%)".format(src, cnt, source_perc[src]) for src, cnt in source_counts.items()]))
print("unique admdiv_cyr count by source: " + ", ".join(["{}: {} unique values ({:.2f}%)".format(src, cnt, unique_admdiv_perc[src]) for src, cnt in unique_admdiv_by_source.items()]))
print("count of 'missing' in id_master in output file: {} rows ({:.2f}%)".format(missing_id_master_count, perc_missing_id_master))

# saving output file
output_file = os.path.join(file_dir, "places_unpivoted_with_sapir_admdiv.xlsx")
df_output.to_excel(output_file, index=False)
print("\noutput file saved: unpivoted file -> {}".format(output_file))

# displaying random samples
sample_output = df_output.sample(n=100, random_state=42) if total_exploded >= 100 else df_output.copy()
print("\n--- sample of 100 random rows from output file ---")
display(sample_output)



--- summary report ---

total rows in manual file: 13 rows
total rows in output file after explosion: 95 rows (730.77%)
source breakdown in output file: title: 95 rows (100.00%)
unique admdiv_cyr count by source: title: 13 unique values (100.00%)
count of 'missing' in id_master in output file: 94 rows (98.95%)

output file saved: unpivoted file -> /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_unpivoted_with_sapir_admdiv.xlsx

--- sample of 100 random rows from output file ---


,source,admdiv_cyr,norm_admdiv_name_cyr,norm_admdiv_type_cyr,admdiv_sapir,admdiv_missing,id_old,id_master,sapir_id,full_sapir,MISS_CENTER_GPS,MISS_REF1,MISS_REF2
0,title,Бережанська округа ОУН,Бережани,округа ОУН,,Berezhany OUN okruha,6794,missing,,,"49.44478430280668, 24.93776785937474",https://uk.wikipedia.org/wiki/%D0%90%D0%B4%D0%...,https://litopysupa.com/wp-content/uploads/2019...
1,title,Бережанський виборчий округ,Бережани,виборчий округ,,Berezhany electoral district,6770,missing,,,"49.44478430280668, 24.93776785937474",---,NaN
2,title,Бережанський надрайон ОУН,Бережани,надрайон ОУН,,Berezhany OUN supra-raion,6636,missing,,,"49.44478430280668, 24.93776785937474",https://uk.wikipedia.org/wiki/%D0%90%D0%B4%D0%...,https://litopysupa.com/wp-content/uploads/2019...
3,title,Бережанський надрайон ОУН,Бережани,надрайон ОУН,,Berezhany OUN supra-raion,6637,missing,,,"49.44478430280668, 24.93776785937474",https://uk.wikipedia.org/wiki/%D0%90%D0%B4%D0%...,https://litopysupa.com/wp-content/uploads/2019...
4,title,Бережанський надрайон ОУН,Бережани,надрайон ОУН,,Berezhany OUN supra-raion,6647,missing,,,"49.44478430280668, 24.93776785937474",https://uk.wikipedia.org/wiki/%D0%90%D0%B4%D0%...,https://litopysupa.com/wp-content/uploads/2019...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,title,Чортківсько-Бережанська округа ОУН,Чортків-Бережани,округа ОУН,,Chortkiv-Berezhany OUN okruha,6674,missing,,,"49.01206277042099, 25.80049258712418",https://uk.wikipedia.org/wiki/%D0%90%D0%B4%D0%...,https://litopysupa.com/wp-content/uploads/2019...
91,title,Чортківсько-Бережанська округа ОУН,Чортків-Бережани,округа ОУН,,Chortkiv-Berezhany OUN okruha,6697,missing,,,"49.01206277042099, 25.80049258712418",https://uk.wikipedia.org/wiki/%D0%90%D0%B4%D0%...,https://litopysupa.com/wp-content/uploads/2019...
92,title,Чортківсько-Бережанська округа ОУН,Чортків-Бережани,округа ОУН,,Chortkiv-Berezhany OUN okruha,6698,missing,,,"49.01206277042099, 25.80049258712418",https://uk.wikipedia.org/wiki/%D0%90%D0%B4%D0%...,https://litopysupa.com/wp-content/uploads/2019...
93,title,Чортківсько-Бережанська округа ОУН,Чортків-Бережани,округа ОУН,,Chortkiv-Berezhany OUN okruha,6714,missing,,,"49.01206277042099, 25.80049258712418",https://uk.wikipedia.org/wiki/%D0%90%D0%B4%D0%...,https://litopysupa.com/wp-content/uploads/2019...


In [ ]:
#@title MERGING UNPIVOTED

# base directory where all files are located
base_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# path to the coordination (union) file
coordination_file = os.path.join(base_dir, "unpivoted_union_table.xlsx")

# reading the coordination file that contains the mapping configuration
df_mapping = pd.read_excel(coordination_file)

# final union columns are those in the coordination file except 'FILE_NAME'
final_columns = [col for col in df_mapping.columns if col != "FILE_NAME"]

# list to collect processed dataframes
df_list = []

# dictionary to store row counts per file
file_row_counts = {}

# iterate over each mapping row (each corresponds to one source file)
for idx, row in df_mapping.iterrows():
    file_name = row["FILE_NAME"]
    # fixed value for category_sapir for this source file from the mapping
    category_value = row["CATEGORY_SAPIR"] if pd.notnull(row["CATEGORY_SAPIR"]) else ""
    source_file_path = os.path.join(base_dir, file_name)
    print(f"processing file: {file_name}")
    try:
        # reading the source file
        df_source = pd.read_excel(source_file_path)
    except Exception as e:
        print(f"error reading {file_name}: {e}")
        continue

    # create a new dataframe with the same index as the source file and with columns as final_columns
    df_new = pd.DataFrame(index=df_source.index, columns=final_columns)

    # iterate over each final union column
    for col in final_columns:
        # if the column is fixed (e.g. category_sapir), fill it with the value from the mapping row
        if col == "CATEGORY_SAPIR":
            df_new[col] = row[col] if pd.notnull(row[col]) else ""
        else:
            # get the mapping value: the name of the source column that should populate this final column
            mapping_val = row[col] if pd.notnull(row[col]) else ""
            if mapping_val:
                # if the mapping value exists in the source dataframe, use its data
                if mapping_val in df_source.columns:
                    df_new[col] = df_source[mapping_val]
                else:
                    # if the expected column is not present in the source file, fill with empty string
                    df_new[col] = ""
            else:
                # if mapping value is empty, fill the final column with empty string
                df_new[col] = ""
    print(f"finished processing file: {file_name} with {len(df_new)} rows")
    file_row_counts[file_name] = len(df_new)
    df_list.append(df_new)

# concatenate all processed dataframes into one final union dataframe
if df_list:
    df_union = pd.concat(df_list, ignore_index=True)
else:
    df_union = pd.DataFrame(columns=final_columns)

# convert id columns from float to integer then to string (to hide decimals and extra zeros)
for col in ["ID_OLD", "ID_MASTER", "ID_SAPIR"]:
    if col in df_union.columns:
        df_union[col] = df_union[col].apply(lambda x: str(int(float(x))) if pd.notnull(x) and x != "" and str(x).replace('.', '', 1).isdigit() else x if pd.notnull(x) else "")

# define output file path for the merged union table
output_file = os.path.join(base_dir, "places_unpivoted_with_sapir_unified.xlsx")
df_union.to_excel(output_file, index=False)
print(f"merged file saved to: {output_file}")

# statistical output

total_rows = len(df_union)
print("\n--- statistical output ---\n")
print(f"total rows in merged file: {total_rows}")

# rows per file and percentage
print("\nrows per file:")
for file_name, count in file_row_counts.items():
    percent = (count / total_rows * 100) if total_rows > 0 else 0
    print(f"file: {file_name} - rows: {count} ({percent:.2f}%)")

# source column stats for 'fond' and 'title'
fond_count = (df_union["SOURCE"] == "fond").sum()
title_count = (df_union["SOURCE"] == "title").sum()
fond_percent = (fond_count / total_rows * 100) if total_rows > 0 else 0
title_percent = (title_count / total_rows * 100) if total_rows > 0 else 0
print("\nsource column stats:")
print(f"'fond' count: {fond_count} ({fond_percent:.2f}%)")
print(f"'title' count: {title_count} ({title_percent:.2f}%)")

# create an effective name: use name_sapir if not empty (or not NaN), otherwise use name_missing_sapir
df_union["effective_name"] = df_union["NAME_SAPIR"].where(df_union["NAME_SAPIR"].fillna("") != "", df_union["NAME_MISSING_SAPIR"])

# stats per category for effective names with breakdown by source and by which field provided the name
print("\nstats per category (unique effective names):")
categories = df_union["CATEGORY_SAPIR"].unique()
for cat in categories:
    df_cat = df_union[df_union["CATEGORY_SAPIR"] == cat]
    unique_effective = df_cat["effective_name"].nunique()
    # breakdown: count unique names from name_sapir (non-empty) and from name_missing_sapir (when name_sapir is empty)
    unique_from_name_sapir = df_cat[df_cat["NAME_SAPIR"].fillna("") != ""]["NAME_SAPIR"].nunique()
    unique_from_name_missing = df_cat[df_cat["NAME_SAPIR"].fillna("") == ""]["NAME_MISSING_SAPIR"].nunique()
    # breakdown by source
    fond_unique = df_cat[df_cat["SOURCE"] == "fond"]["effective_name"].nunique()
    title_unique = df_cat[df_cat["SOURCE"] == "title"]["effective_name"].nunique()
    print(f"category: {cat} - total unique effective names: {unique_effective}, from name_sapir: {unique_from_name_sapir}, from name_missing_sapir: {unique_from_name_missing}, fond unique: {fond_unique}, title unique: {title_unique}")

# distribution of id_sapir per id_old (consider only non-empty id_sapir)
df_id_old = df_union[df_union["ID_SAPIR"] != ""].groupby("ID_OLD")["ID_SAPIR"].nunique()
dist_id_old = df_id_old.value_counts().sort_index()
total_id_old = df_id_old.count()
print("\ndistribution of unique id_sapir per id_old:\n")
for num, count in dist_id_old.items():
    percent = (count / total_id_old * 100) if total_id_old > 0 else 0
    print(f"id_old with {num} unique id_sapir: {count} ({percent:.2f}%)")

# distribution of id_sapir per id_master (exclude id_master with 'missing' or empty)
df_id_master = df_union[(df_union["ID_MASTER"] != "missing") & (df_union["ID_MASTER"] != "") & (df_union["ID_SAPIR"] != "")].groupby("ID_MASTER")["ID_SAPIR"].nunique()
dist_id_master = df_id_master.value_counts().sort_index()
total_id_master = df_id_master.count()
print("\ndistribution of unique id_sapir per id_master (excluding 'missing'):\n")
for num, count in dist_id_master.items():
    percent = (count / total_id_master * 100) if total_id_master > 0 else 0
    print(f"id_master with {num} unique id_sapir: {count} ({percent:.2f}%)")

# id_sapir filled vs empty
id_sapir_filled = (df_union["ID_SAPIR"].fillna("") != "").sum()
id_sapir_empty = (df_union["ID_SAPIR"].fillna("") == "").sum()
filled_percent = (id_sapir_filled / total_rows * 100) if total_rows > 0 else 0
empty_percent = (id_sapir_empty / total_rows * 100) if total_rows > 0 else 0
print("\nid_sapir filled vs empty:")
print(f"non-empty id_sapir: {id_sapir_filled} ({filled_percent:.2f}%)")
print(f"empty id_sapir: {id_sapir_empty} ({empty_percent:.2f}%)")

# top 50 combinations of name_cyr and id_sapir (both non-empty)
df_name_combo = df_union[(df_union["NAME_CYR"] != "") & (df_union["ID_SAPIR"] != "")].copy()
df_name_combo["name_cyr_id_sapir"] = df_name_combo["NAME_CYR"].astype(str) + " " + df_name_combo["ID_SAPIR"].astype(str)
combo_counts = df_name_combo["name_cyr_id_sapir"].value_counts().head(50)
total_combos = df_name_combo["name_cyr_id_sapir"].count()
print("\n*top 50 combinations of name_cyr and id_sapir:*\n")
print(f"total combinations: {total_combos} (100%)\n")
for combo, count in combo_counts.items():
    name_only = combo.rsplit(" ", 1)[0]  # hide the ID part
    percent = (count / total_combos * 100) if total_combos > 0 else 0
    # breakdown by source for this combination
    df_combo = df_name_combo[df_name_combo["name_cyr_id_sapir"] == combo]
    fond_count_combo = (df_combo["SOURCE"] == "fond").sum()
    title_count_combo = (df_combo["SOURCE"] == "title").sum()
    print(f"combination: {name_only} - count: {count} ({percent:.2f}%), fond: {fond_count_combo}, title: {title_count_combo}")

# unique non-empty name_missing_sapir count and top 30 values
print("\n*top 30 name_missing_sapir values:*\n")
df_name_missing = df_union[df_union["NAME_MISSING_SAPIR"] != ""]
unique_name_missing_count = df_name_missing["NAME_MISSING_SAPIR"].nunique()
print("unique non-empty name_missing_sapir count:", unique_name_missing_count)
top30_name_missing = df_name_missing["NAME_MISSING_SAPIR"].value_counts().head(30)
total_name_missing = df_name_missing["NAME_MISSING_SAPIR"].count()
print(f"total name_missing_sapir values: {total_name_missing} (100%)\n")
for name, count in top30_name_missing.items():
    percent = (count / total_name_missing * 100) if total_name_missing > 0 else 0
    print(f"{name} - count: {count} ({percent:.2f}%)")

# unique name_parachr_sapir count and top 10 values
print("\n*top 10 name_parachr_sapir values:*\n")
df_name_parachr = df_union[df_union["NAME_PARACHR_SAPIR"] != ""]
unique_name_parachr_count = df_name_parachr["NAME_PARACHR_SAPIR"].nunique()
print("unique name_parachr_sapir count:", unique_name_parachr_count)
top10_name_parachr = df_name_parachr["NAME_PARACHR_SAPIR"].value_counts().head(10)
total_name_parachr = df_name_parachr["NAME_PARACHR_SAPIR"].count()
print(f"total name_parachr_sapir values: {total_name_parachr} (100%)\n")
for name, count in top10_name_parachr.items():
    percent = (count / total_name_parachr * 100) if total_name_parachr > 0 else 0
    print(f"{name} - count: {count} ({percent:.2f}%)")

# distribution of unique combinations of miss_center_gps and name_missing_sapir
df_miss_combo = df_union[["MISS_CENTER_GPS", "NAME_MISSING_SAPIR"]].drop_duplicates()
non_dash_count = df_miss_combo[df_miss_combo["MISS_CENTER_GPS"] != "---"].shape[0]
dash_count = df_miss_combo[df_miss_combo["MISS_CENTER_GPS"] == "---"].shape[0]
total_miss_combo = df_miss_combo.shape[0]
non_dash_percent = (non_dash_count / total_miss_combo * 100) if total_miss_combo > 0 else 0
dash_percent = (dash_count / total_miss_combo * 100) if total_miss_combo > 0 else 0
print("\ndistribution of provided vs missing GPS coordinates for missing in sapir:")
print(f"with GPS coordinates': {non_dash_count} ({non_dash_percent:.2f}%)")
print(f"GPS coordinates not found: {dash_count} ({dash_percent:.2f}%)")


processing file: places_unpivoted_with_sapir_admdiv.xlsx
finished processing file: places_unpivoted_with_sapir_admdiv.xlsx with 95 rows
processing file: places_unpivoted_with_sapir_countries.xlsx
finished processing file: places_unpivoted_with_sapir_countries.xlsx with 965 rows
processing file: places_unpivoted_with_sapir_district.xlsx
finished processing file: places_unpivoted_with_sapir_district.xlsx with 5623 rows
processing file: places_unpivoted_with_sapir_places.xlsx
finished processing file: places_unpivoted_with_sapir_places.xlsx with 10549 rows
processing file: places_unpivoted_with_sapir_region.xlsx
finished processing file: places_unpivoted_with_sapir_region.xlsx with 4945 rows
processing file: places_unpivoted_with_sapir_sub-district.xlsx
finished processing file: places_unpivoted_with_sapir_sub-district.xlsx with 808 rows
merged file saved to: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/places_unpivoted_with_sapir_unified.xlsx

--- statistical 

In [ ]:
#@title MERGING MISSING

# define base directory for files
base_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS"

# read the coordination (union) mapping file
coord_file = os.path.join(base_dir, "missing_union_table.xlsx")
df_mapping = pd.read_excel(coord_file)

# define final union columns (all columns except 'FILE_NAME')
final_columns = [col for col in df_mapping.columns if col != "FILE_NAME"]

# list to collect processed dataframes and dictionary to store row counts per file
df_list = []
file_row_counts = {}

# iterate over each mapping row (each corresponds to one source file)
for idx, mapping_row in df_mapping.iterrows():
    # get source file name and fixed category value from mapping
    file_name = mapping_row["FILE_NAME"]
    category_fixed = mapping_row["CATEGORY_SAPIR"] if pd.notnull(mapping_row["CATEGORY_SAPIR"]) else ""
    file_path = os.path.join(base_dir, file_name)
    print(f"processing file: {file_name}")
    try:
        # read the source excel file
        df_source = pd.read_excel(file_path)
    except Exception as e:
        print(f"error reading {file_name}: {e}")
        continue

    # create a new dataframe with final union columns
    df_new = pd.DataFrame(index=df_source.index, columns=final_columns)

    # for each final union column, use mapping to assign source column value (or fill empty string)
    for col in final_columns:
        if col == "CATEGORY_SAPIR":
            df_new[col] = category_fixed
        else:
            mapping_val = mapping_row[col] if pd.notnull(mapping_row[col]) else ""
            if mapping_val:
                if mapping_val in df_source.columns:
                    df_new[col] = df_source[mapping_val]
                else:
                    df_new[col] = ""
            else:
                df_new[col] = ""
    file_row_counts[file_name] = len(df_new)
    print(f"finished processing file: {file_name} with {len(df_new)} rows")
    df_list.append(df_new)

# concatenate all processed dataframes into one final union dataframe
if df_list:
    df_union = pd.concat(df_list, ignore_index=True)
else:
    df_union = pd.DataFrame(columns=final_columns)

# load the mapping file for id conversion (old id -> master id)
mapping_id_file = os.path.join(base_dir, "t_Master_table_draft_dana_withdates_mapped.xlsx")
try:
    df_id_map = pd.read_excel(mapping_id_file)
except Exception as e:
    print(f"error reading mapping id file: {e}")
    df_id_map = pd.DataFrame()

# build a dictionary mapping old_id to id_master (convert numeric values to int then to string)
id_mapping = {}
if not df_id_map.empty:
    for _, row in df_id_map.iterrows():
        if pd.notnull(row.get("old_id")) and pd.notnull(row.get("id_master")):
            try:
                old_id_str = str(int(float(row["old_id"])))
            except:
                old_id_str = str(row["old_id"]).strip()
            try:
                id_master_str = str(int(float(row["id_master"])))
            except:
                id_master_str = str(row["id_master"]).strip()
            id_mapping[old_id_str] = id_master_str

# define a function to map comma-separated old ids to corresponding master ids
def map_ids(id_old_val):
    # if value is missing or empty, return empty string
    if pd.isna(id_old_val) or str(id_old_val).strip() == "":
        return ""
    id_old_str = str(id_old_val)
    # split by comma and trim each part
    ids = [part.strip() for part in id_old_str.split(",") if part.strip() != ""]
    mapped_ids = []
    for id_val in ids:
        try:
            id_int = int(float(id_val))
            id_clean = str(id_int)
        except:
            id_clean = id_val
        # if mapping exists, append the master id; if not, skip this id
        if id_clean in id_mapping:
            mapped_ids.append(id_mapping[id_clean])
    return ", ".join(mapped_ids)

# update the ID_MASTER column by mapping values from ID_OLD using the id_mapping
if "ID_OLD" in df_union.columns and "ID_MASTER" in df_union.columns:
    df_union["ID_MASTER"] = df_union["ID_OLD"].apply(map_ids)

# define a helper function to convert id cells (which may be comma-separated numbers) to cleaned string format
def convert_id_cell(cell):
    if pd.isna(cell) or str(cell).strip() == "":
        return ""
    cell_str = str(cell)
    parts = [part.strip() for part in cell_str.split(",") if part.strip() != ""]
    converted_parts = []
    for part in parts:
        try:
            num = int(float(part))
            converted_parts.append(str(num))
        except:
            converted_parts.append(part)
    return ", ".join(converted_parts)

# process all columns with names starting with 'ID_' to ensure proper string formatting
id_cols = [col for col in df_union.columns if col.upper().startswith("ID_")]
for col in id_cols:
    df_union[col] = df_union[col].apply(convert_id_cell)

# if the column NAME_FUL_ENG exists, rename it to NAME_ENG so the concatenation can be applied
if "NAME_FUL_ENG" in df_union.columns:
    df_union.rename(columns={"NAME_FUL_ENG": "NAME_ENG"}, inplace=True)

# update NAME_ENG column: if empty, fill with NAME_NORM_ENG + " " + TYPE_NORM_ENG
df_union["NAME_ENG"] = df_union.apply(lambda row: row["NAME_ENG"] if pd.notnull(row["NAME_ENG"]) and str(row["NAME_ENG"]).strip() != "" else ((str(row["NAME_NORM_ENG"]).strip() if pd.notnull(row["NAME_NORM_ENG"]) else "") + " " + (str(row["TYPE_NORM_ENG"]).strip() if pd.notnull(row["TYPE_NORM_ENG"]) else "")).strip(), axis=1)

# ---------------------- generating statistical output ----------------------

total_rows = len(df_union)
print("\n--- statistical output ---\n")
print(f"total number of rows in merged file: {total_rows}")

# print the number of rows per category and percentage of overall rows
print("\nrows per category:")
category_counts = df_union["CATEGORY_SAPIR"].value_counts()
for cat, count in category_counts.items():
    perc = (count / total_rows * 100) if total_rows > 0 else 0
    print(f"{cat} - {count} ({perc:.2f}%)")

# compute top 10 of name_cyr per category based on count of comma-separated id numbers in ID_OLD
print("\ntop 10 frequent missing items per category (based on id_old):")
# add helper column that counts the number of id numbers in ID_OLD
df_union["id_old_count"] = df_union["ID_OLD"].apply(lambda x: len([p for p in str(x).split(",") if p.strip() != ""]) if pd.notna(x) else 0)
# group by CATEGORY_SAPIR and NAME_CYR and sum the counts
grouped = df_union.groupby(["CATEGORY_SAPIR", "NAME_CYR"])["id_old_count"].sum().reset_index()
# for each category, compute and print top 10 entries
for cat in category_counts.index:
    df_cat = grouped[grouped["CATEGORY_SAPIR"] == cat]
    df_top = df_cat.sort_values("id_old_count", ascending=False).head(10)
    print(f"\n{cat} (total rows: {category_counts[cat]}):\n")
    for _, row in df_top.iterrows():
        print(f"{row['NAME_CYR']} - {row['id_old_count']}")

# distribution of source for overall data (by 'SOURCE' column)
print("\ndistribution by source for entire data:")
source_counts = df_union["SOURCE"].value_counts()
for src, count in source_counts.items():
    perc = (count / total_rows * 100) if total_rows > 0 else 0
    print(f"source: {src} - count: {count} ({perc:.2f}%)")

# distribution of source for each category
print("\ndistribution by source per category:")
for cat in category_counts.index:
    df_cat = df_union[df_union["CATEGORY_SAPIR"] == cat]
    cat_total = len(df_cat)
    src_counts_cat = df_cat["SOURCE"].value_counts()
    print(f"\n{cat} (total rows: {cat_total}):")
    for src, count in src_counts_cat.items():
        perc = (count / cat_total * 100) if cat_total > 0 else 0
        print(f"{src} - count: {count} ({perc:.2f}%)")

# stats for miss_center_gps: count and percentage of filled (not equal to '---') and empty ('---') values
print("\nmiss_center_gps stats:")
gps_full = (df_union["MISS_CENTER_GPS"] != "---").sum()
gps_empty = (df_union["MISS_CENTER_GPS"] == "---").sum()
perc_full = (gps_full / total_rows * 100) if total_rows > 0 else 0
perc_empty = (gps_empty / total_rows * 100) if total_rows > 0 else 0
print(f"miss_center_gps found: {gps_full} ({perc_full:.2f}%)")
print(f"miss_center_gps missing: {gps_empty} ({perc_empty:.2f}%)")

# stats for miss_ref1: count and percentage of filled (not equal to '---') and empty ('---') values
print("\nmiss_ref1 stats:")
ref1_full = (df_union["MISS_REF1"] != "---").sum()
ref1_empty = (df_union["MISS_REF1"] == "---").sum()
perc_ref1_full = (ref1_full / total_rows * 100) if total_rows > 0 else 0
perc_ref1_empty = (ref1_empty / total_rows * 100) if total_rows > 0 else 0
print(f"reference found: {ref1_full} ({perc_ref1_full:.2f}%)")
print(f"reference not found: {ref1_empty} ({perc_ref1_empty:.2f}%)")

# stats for paradchronistic values (name_parachr_sapir) in categories district and region
print("\nparachronistic values stats for district and region:")
for cat in ["district", "region"]:
    df_cat = df_union[df_union["CATEGORY_SAPIR"] == cat]
    total_cat = len(df_cat)
    non_empty_para = df_cat[df_cat["NAME_PARACHR_SAPIR"].fillna("").str.strip() != ""].shape[0]
    perc_non_empty_para = (non_empty_para / total_cat * 100) if total_cat > 0 else 0
    print(f"{cat} - total rows: {total_cat}")
    print(f"parachronistic: {non_empty_para} ({perc_non_empty_para:.2f}%)")

# stats for lacking_dedicated_sapir in sub-district category
print("\nlacking_dedicated_sapir stats for sub-district category:")
df_sub = df_union[df_union["CATEGORY_SAPIR"] == "sub-district"]
total_sub = len(df_sub)
non_empty_lacking = df_sub[df_sub["LACKING_DEDICATED_SAPIR"].fillna("").str.strip() != ""].shape[0]
perc_non_empty_lacking = (non_empty_lacking / total_sub * 100) if total_sub > 0 else 0
print(f"sub-district - total rows: {total_sub}")
print(f"lacking_dedicated_sapir: {non_empty_lacking} ({perc_non_empty_lacking:.2f}%)")

# drop helper column
df_union.drop(columns=["id_old_count"], inplace=True)

# save the merged union dataframe to output file
output_file = os.path.join(base_dir, "places_missing_in_sapir_unified.xlsx")
df_union.to_excel(output_file, index=False)
print(f"\nmerged file saved to: {output_file}")


processing file: places_missing_in_sapir_administrative_division.xlsx
finished processing file: places_missing_in_sapir_administrative_division.xlsx with 13 rows
processing file: places_missing_in_sapir_district.xlsx
finished processing file: places_missing_in_sapir_district.xlsx with 98 rows
processing file: places_missing_in_sapir_places.xlsx
finished processing file: places_missing_in_sapir_places.xlsx with 284 rows
processing file: places_missing_in_sapir_region.xlsx
finished processing file: places_missing_in_sapir_region.xlsx with 36 rows
processing file: places_missing_in_sapir_sub-district.xlsx
finished processing file: places_missing_in_sapir_sub-district.xlsx with 59 rows

--- statistical output ---

total number of rows in merged file: 490

rows per category:
place - 284 (57.96%)
district - 98 (20.00%)
sub-district - 59 (12.04%)
region - 36 (7.35%)
חלוקה_אדמיניסטרטיבית - 13 (2.65%)

top 10 frequent missing items per category (based on id_old):

place (total rows: 284):

Воро